In [1]:
#Prints **all** console output, not just last item in cell 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

**Notebook author:** emeinhardt@ucsd.edu

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Overview" data-toc-modified-id="Overview-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Overview</a></span><ul class="toc-item"><li><span><a href="#Motivation" data-toc-modified-id="Motivation-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Motivation</a></span><ul class="toc-item"><li><span><a href="#What-we-want-to-calculate" data-toc-modified-id="What-we-want-to-calculate-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>What we <em>want</em> to calculate</a></span></li><li><span><a href="#What-we-can-calculate" data-toc-modified-id="What-we-can-calculate-1.1.2"><span class="toc-item-num">1.1.2&nbsp;&nbsp;</span>What we <em>can</em> calculate</a></span></li><li><span><a href="#The-structure-of-calculations" data-toc-modified-id="The-structure-of-calculations-1.1.3"><span class="toc-item-num">1.1.3&nbsp;&nbsp;</span>The structure of calculations</a></span></li><li><span><a href="#The-structure-of-the-pipeline" data-toc-modified-id="The-structure-of-the-pipeline-1.1.4"><span class="toc-item-num">1.1.4&nbsp;&nbsp;</span>The structure of the pipeline</a></span></li></ul></li><li><span><a href="#Requirements" data-toc-modified-id="Requirements-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Requirements</a></span><ul class="toc-item"><li><span><a href="#Python-environment" data-toc-modified-id="Python-environment-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Python environment</a></span></li><li><span><a href="#Hardware-/-runtime-expectations" data-toc-modified-id="Hardware-/-runtime-expectations-1.2.2"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>Hardware / runtime expectations</a></span></li></ul></li><li><span><a href="#Todo" data-toc-modified-id="Todo-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Todo</a></span></li></ul></li><li><span><a href="#Imports" data-toc-modified-id="Imports-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Imports</a></span></li><li><span><a href="#Permitted-steps-/-control-flow" data-toc-modified-id="Permitted-steps-/-control-flow-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Permitted steps / control flow</a></span></li><li><span><a href="#Step-0:-Import/check-for-foundational-files" data-toc-modified-id="Step-0:-Import/check-for-foundational-files-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Step 0: Import/check for foundational files</a></span><ul class="toc-item"><li><span><a href="#Importing-existing-data-and-creating-directories" data-toc-modified-id="Importing-existing-data-and-creating-directories-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Importing existing data and creating directories</a></span></li><li><span><a href="#Step-0a:-Check-for-gating-data" data-toc-modified-id="Step-0a:-Check-for-gating-data-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Step 0a: Check for gating data</a></span></li><li><span><a href="#Step-0b:-Check-for-transcribed-lexicons" data-toc-modified-id="Step-0b:-Check-for-transcribed-lexicons-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Step 0b: Check for transcribed lexicons</a></span></li><li><span><a href="#Step-0c:-Check-for-n-gram-contexts" data-toc-modified-id="Step-0c:-Check-for-n-gram-contexts-4.4"><span class="toc-item-num">4.4&nbsp;&nbsp;</span>Step 0c: Check for n-gram contexts</a></span></li><li><span><a href="#Step-0d:-Check-for-language-model(s)" data-toc-modified-id="Step-0d:-Check-for-language-model(s)-4.5"><span class="toc-item-num">4.5&nbsp;&nbsp;</span>Step 0d: Check for language model(s)</a></span></li></ul></li><li><span><a href="#Step-1:-Segment-inventory-alignment" data-toc-modified-id="Step-1:-Segment-inventory-alignment-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Step 1: Segment inventory alignment</a></span><ul class="toc-item"><li><span><a href="#Step-1a:-Define-inventory-alignment-projections" data-toc-modified-id="Step-1a:-Define-inventory-alignment-projections-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Step 1a: Define inventory alignment projections</a></span></li><li><span><a href="#Step-1b:-Apply-inventory-alignment-projections" data-toc-modified-id="Step-1b:-Apply-inventory-alignment-projections-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Step 1b: Apply inventory alignment projections</a></span><ul class="toc-item"><li><span><a href="#Check-for-projection-definitions" data-toc-modified-id="Check-for-projection-definitions-5.2.1"><span class="toc-item-num">5.2.1&nbsp;&nbsp;</span>Check for projection definitions</a></span></li><li><span><a href="#How-are-inventory-alignment-projections-actually-applied?" data-toc-modified-id="How-are-inventory-alignment-projections-actually-applied?-5.2.2"><span class="toc-item-num">5.2.2&nbsp;&nbsp;</span>How are inventory alignment projections actually applied?</a></span></li><li><span><a href="#Apply-projection-definitions" data-toc-modified-id="Apply-projection-definitions-5.2.3"><span class="toc-item-num">5.2.3&nbsp;&nbsp;</span>Apply projection definitions</a></span></li></ul></li></ul></li><li><span><a href="#Step-2:-Generating-channel-and-(orthographic)-lexicon-distributions" data-toc-modified-id="Step-2:-Generating-channel-and-(orthographic)-lexicon-distributions-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Step 2: Generating channel and (orthographic) lexicon distributions</a></span><ul class="toc-item"><li><span><a href="#Step-2a:-Generating-channel-distributions-and-associated-metadata" data-toc-modified-id="Step-2a:-Generating-channel-distributions-and-associated-metadata-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Step 2a: Generating channel distributions and associated metadata</a></span><ul class="toc-item"><li><span><a href="#Metadata" data-toc-modified-id="Metadata-6.1.1"><span class="toc-item-num">6.1.1&nbsp;&nbsp;</span>Metadata</a></span></li><li><span><a href="#Channel-distributions" data-toc-modified-id="Channel-distributions-6.1.2"><span class="toc-item-num">6.1.2&nbsp;&nbsp;</span>Channel distributions</a></span></li></ul></li><li><span><a href="#Step-2b:-Generating-(contextual)-lexicon-distributions-(over-orthographic-vocabularies)" data-toc-modified-id="Step-2b:-Generating-(contextual)-lexicon-distributions-(over-orthographic-vocabularies)-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>Step 2b: Generating (contextual) lexicon distributions (over orthographic vocabularies)</a></span></li></ul></li><li><span><a href="#Step-3:-Creating-combinable-models" data-toc-modified-id="Step-3:-Creating-combinable-models-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Step 3: Creating combinable models</a></span><ul class="toc-item"><li><span><a href="#Step-3a:-Filter-transcription-lexicons-to-only-include-words-that-can-be-modeled-by-a-given-channel-distribution" data-toc-modified-id="Step-3a:-Filter-transcription-lexicons-to-only-include-words-that-can-be-modeled-by-a-given-channel-distribution-7.1"><span class="toc-item-num">7.1&nbsp;&nbsp;</span>Step 3a: Filter transcription lexicons to only include words that can be modeled by a given channel distribution</a></span></li><li><span><a href="#Step-3b:-Filter-transcription-lexicons-to-only-include-words-that-are-in-a-language-model's-vocabulary" data-toc-modified-id="Step-3b:-Filter-transcription-lexicons-to-only-include-words-that-are-in-a-language-model's-vocabulary-7.2"><span class="toc-item-num">7.2&nbsp;&nbsp;</span>Step 3b: Filter transcription lexicons to only include words that are in a language model's vocabulary</a></span></li><li><span><a href="#Step-3c:-Filter-the-conditioning-events-of-channel-distributions-to-only-include-$k$-factors-contained-in-elements-of-a-transcription-lexicon's-segmental-wordforms" data-toc-modified-id="Step-3c:-Filter-the-conditioning-events-of-channel-distributions-to-only-include-$k$-factors-contained-in-elements-of-a-transcription-lexicon's-segmental-wordforms-7.3"><span class="toc-item-num">7.3&nbsp;&nbsp;</span>Step 3c: Filter the conditioning events of channel distributions to only include $k$-factors contained in elements of a transcription lexicon's segmental wordforms</a></span></li><li><span><a href="#Step-3d:-For-each-(filtered)-transcribed-lexicon-relation,-define-the-relevant-contextual-lexicon-distributions-over-orthographic-wordforms" data-toc-modified-id="Step-3d:-For-each-(filtered)-transcribed-lexicon-relation,-define-the-relevant-contextual-lexicon-distributions-over-orthographic-wordforms-7.4"><span class="toc-item-num">7.4&nbsp;&nbsp;</span>Step 3d: For each (filtered) transcribed lexicon relation, define the relevant contextual lexicon distributions over orthographic wordforms</a></span></li><li><span><a href="#Step-3e:-For-each-(filtered)-transcribed-lexicon-relation,-define-a-conditional-distribution-on-segmental-wordforms-given-an-orthographic-wordform" data-toc-modified-id="Step-3e:-For-each-(filtered)-transcribed-lexicon-relation,-define-a-conditional-distribution-on-segmental-wordforms-given-an-orthographic-wordform-7.5"><span class="toc-item-num">7.5&nbsp;&nbsp;</span>Step 3e: For each (filtered) transcribed lexicon relation, define a conditional distribution on segmental wordforms given an orthographic wordform</a></span></li></ul></li><li><span><a href="#Step-4:-Pre-calculate-remaining-forward-model-components-and-meta-data" data-toc-modified-id="Step-4:-Pre-calculate-remaining-forward-model-components-and-meta-data-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Step 4: Pre-calculate remaining forward model components and meta-data</a></span><ul class="toc-item"><li><span><a href="#Step-4a:-Generate-triphone-lexicon-distributions-for-every-triphone-channel-model" data-toc-modified-id="Step-4a:-Generate-triphone-lexicon-distributions-for-every-triphone-channel-model-8.1"><span class="toc-item-num">8.1&nbsp;&nbsp;</span>Step 4a: Generate triphone lexicon distributions for every triphone channel model</a></span></li><li><span><a href="#Step-4b:-Pre-calculate-prefix-relation,-$k$-cousins,-and-$k$-spheres-for-each-segmental-lexicon" data-toc-modified-id="Step-4b:-Pre-calculate-prefix-relation,-$k$-cousins,-and-$k$-spheres-for-each-segmental-lexicon-8.2"><span class="toc-item-num">8.2&nbsp;&nbsp;</span>Step 4b: Pre-calculate prefix relation, $k$-cousins, and $k$-spheres for each segmental lexicon</a></span></li><li><span><a href="#Step-4c:-Calculate-the-marginal-probability-$p(W|C)$-of-each-segmental-wordform-$w$-given-$n$-gram-contexts-$C$" data-toc-modified-id="Step-4c:-Calculate-the-marginal-probability-$p(W|C)$-of-each-segmental-wordform-$w$-given-$n$-gram-contexts-$C$-8.3"><span class="toc-item-num">8.3&nbsp;&nbsp;</span>Step 4c: Calculate the marginal probability $p(W|C)$ of each segmental wordform $w$ given $n$-gram contexts $C$</a></span></li><li><span><a href="#Step-4d:-Define-observation-distributions" data-toc-modified-id="Step-4d:-Define-observation-distributions-8.4"><span class="toc-item-num">8.4&nbsp;&nbsp;</span>Step 4d: Define observation distributions</a></span></li><li><span><a href="#Step-4e:-Define-channel-distributions-on-a-set-of-segmental-wordforms(+prefixes)" data-toc-modified-id="Step-4e:-Define-channel-distributions-on-a-set-of-segmental-wordforms(+prefixes)-8.5"><span class="toc-item-num">8.5&nbsp;&nbsp;</span>Step 4e: Define channel distributions on a set of segmental wordforms(+prefixes)</a></span></li></ul></li><li><span><a href="#Step-5:-Calculate-posterior-probabilities" data-toc-modified-id="Step-5:-Calculate-posterior-probabilities-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Step 5: Calculate posterior probabilities</a></span><ul class="toc-item"><li><span><a href="#Step-5a:-Calculate-$p(V|W,-C)$" data-toc-modified-id="Step-5a:-Calculate-$p(V|W,-C)$-9.1"><span class="toc-item-num">9.1&nbsp;&nbsp;</span>Step 5a: Calculate $p(V|W, C)$</a></span></li><li><span><a href="#Step-5b:-Calculate-$p(\hat{X}_0^f|X_0^f,-C)$" data-toc-modified-id="Step-5b:-Calculate-$p(\hat{X}_0^f|X_0^f,-C)$-9.2"><span class="toc-item-num">9.2&nbsp;&nbsp;</span>Step 5b: Calculate $p(\hat{X}_0^f|X_0^f, C)$</a></span></li><li><span><a href="#Step-5c:-Calculate-$p(\hat{V}-=-v^*|-V-=-v^*,-C)$" data-toc-modified-id="Step-5c:-Calculate-$p(\hat{V}-=-v^*|-V-=-v^*,-C)$-9.3"><span class="toc-item-num">9.3&nbsp;&nbsp;</span>Step 5c: Calculate $p(\hat{V} = v^*| V = v^*, C)$</a></span></li></ul></li><li><span><a href="#Step-6:-Generating-analysis-measures" data-toc-modified-id="Step-6:-Generating-analysis-measures-10"><span class="toc-item-num">10&nbsp;&nbsp;</span>Step 6: Generating analysis measures</a></span></li></ul></div>

# Overview

This notebook describes the processing pipeline from 
 - gating data
 - transcribed lexicon
 - a language model and (possibly empty) n-gram contexts

to 
 - channel distribution
 - lexicon distribution(s) (distributions over wordforms)
 - expected posterior distribution over intended wordform given what has been produced of what was intended.
 
 
It describes what happens at each step, checks some pre- and post-conditions, describes what you, the user must do (if anything), and scripts some commands to automatically do the necessary processing.

## Motivation

There are about one-and-a-half calculations that this processing pipeline enables.

Let 
 - $C = \{c_0, c_1 ... c_{n-1}\}$ denote a set of $n$-gram contexts (sequences of $\leq (n-1)$ orthographic wordforms) drawn from a speech corpus (e.g. Buckeye or Switchboard).
 - $V = \{v_0, v_1 ... v_V\}$ denote a set of orthographic wordforms (a 'vocabulary') drawn from a speech corpus (the same corpus $C$ is drawn from).
 - $W = \{w_0, w_1 ... w_W\}$ denote a set of segmental wordforms ('transcribed wordforms'). Each element $w \in W$ consists of a sequence $x_0 x_1 ... x_f = x_0^f$ of segments ('phonemes') drawn from a set of symbols $\Sigma_x$.
 
(All sets are finite, unless otherwise noted.)

 - A language model describes $p(V|C)$.
 - A lexical database or speech corpus describes a relation $r$ between orthographic wordforms $V$ and segmental wordforms $W$, where $(v,w) \in r$ iff $v$ can be produced as $w$.
 - Let $p(W|V)$ be defined as follows: $p(w|v) = \frac{1}{r_{v}}$, where $r_v = |\{w' | (v, w') \in r\}|$
 - Given that the $i$th segment that a speaker intended to produce is $x_i$, a triphone channel model describes $p(Y_i|x_{i-1}, x_i; x_{i+1})$, a distribution over received/perceived segments $\Sigma_y$. This is estimated from diphone gating data. Note that it doesn't permit modeling insertions or deletions.
 - $p(Y_0^f|W) = p(Y_0^f|X_0^f)$ is a (channel) distribution over perceived (segmental) wordforms given an intended (segmental) wordform. It is completely defined by a choice of $p(W|V)$ and a choice of $p(Y_i|x_{i-1}, x_i; x_{i+1})$.

The *forward model*, then defines a distribution $p(Y_0^f, X_0^f, V|C) = p(Y_0^f|X_0^f)p(X_0^f|V)p(V|C)$.

### What we *want* to calculate

We are interested in calculating the *expected posterior of a listener* over what orthographic wordform the speaker intended given what the speaker actually intended (taking the expectation over the speaker's actual intended segmental wordform, the perceived segmental wordform, and the listener's estimate of the speaker's intended segmental wordform):

$$p(\hat{V} = v^*|V = v^*, c) = \sum\limits_{x_0^{*f}, y_0^f, x_0^{'f}} p(\hat{V} = v^*, \hat{X}_0^f = x_0^{'f}, y_0^f, X_0^f = x_0^{*f}|V = v^*, c)$$

$$p(\hat{V} = v^*|V = v^*, c) = \sum\limits_{x_0^{*f}, y_0^f, x_0^{'f}} p(\hat{V} = v^*|\hat{X}_0^f = x_0^{'f}, c) p(\hat{X}_0^f = x_0^{'f}|Y_0^f = y_0^f, c) p(Y_0^f = y_0^f | X_0^f = x_0^{*f})p(X_0^f = x_0^{*f}|V = v^*)$$

where 

1. $p(\hat{X}_0^f = x_0^{'f}|Y_0^f = y_0^f, c) = \frac{p(y_0^f|x_0^{'f})p(x_0^{'f}|c)}{p(y_0^f | c)}$
2. $p(x_0^{'f}|c) = \sum\limits_{v'} p(x_0^{'f}|v')p(v'|c)$
3. $p(y_0^f| c) = \sum\limits_{v', x_0^{''f}} p(y_0^f|x_0^{''f})p(x_0^{''f}|v')p(v'|c) = \sum\limits_{x_0^{''f}} p(y_0^f|x_0^{''f})p(x_0^{''f}|c)$
4. $p(\hat{V} = v^*|\hat{X}_0^f = x_0^{'f}, c) = \frac{p(x_0^{'f}|v^*)p(v^*|c)}{p(x_0^{'f}|c)}$ 


### What we *can* calculate

 1. Unfortunately, because of the enormous size of $Y_0^f$, exact marginalization over all $y_0^f$ is *not* feasible.
 2. Fortunately, because, for any given $x_0^{*f}$, the fraction of $Y_0^f$ with non-negligible mass in $p(Y_0^f|x_0^{*f})$ is small, we can construct a Monte Carlo estimate of 
 $$p(\hat{X}_0^f = x_0^{'f}|x_0^{*f}, c) = \sum\limits_{y_0^f} p(\hat{X}_0^f = x_0^{'f}|y_0^f, c) p(y_0^f|x_0^{*f})$$
 as
 $$\hat{p}(\hat{X}_0^f = x_0^{'f}|x_0^{*f}, c) = \frac{1}{n} \sum\limits_{y_0^f \in S} p(\hat{X}_0^f = x_0^{'f}|y_0^f, c)$$
 where $S = $ a set of $n$ samples from $p(Y_0^f|x_0^{*f})$. In practice an $n \approx 50$ seems to result in estimates that are within $10^{-6}$ of the true estimate. 
 
 3. Unfortunately, even with this approximation $p(\hat{X}_0^f|X_0^{*f}, c)$ has about $10^8 - 10^9$ entries: too many to feasibly calculate exactly, especially across *all choices of $c$* as well.
 4. However, because of the relative dispersion of wordforms, the relatively low overall rate of channel noise, and the information provided by sentential context, the fraction of $\hat{X}_0^f$ assigned non-negligible mass in $p(\hat{X}_0^f|x_0^{*f}, c)$ for any given $x_0^{*f}$ is relatively small, and largely concentrated on $x_0^{'f}$ that are within $k$ edits (substitutions) of $x_0^{*f}$ for small $k$. Accordingly, we can get an arbitrarily good approximation $\hat{p}^{k}$ of $p(\hat{X}_0^f|x_0^{*f}, c)$ by 
  - choosing an arbitrarily small threshold $\epsilon$
  - calculating
$$p^k = \{p(\hat{X}_0^f = x_0^{'f}|x_0^{*f}, c) | x_0^{'f} \text{ is within } k \text{ edits of }x_0^{*f}\}$$ for progressively increasing $k$, stopping when $1 - \sum p_i^k \leq \epsilon$ and assigning $0$ probability to all $x_0^{'f}$ not associated with $p^k$.

Note that this means that if we choose the same $\epsilon$ for all $(x_0^{*f}, c)$ pairs, some segmental wordforms $x_0^f$ will have approximations involving higher or lower $k$ values than others, but that all will have distributions that are at least as accurate as some same minimum (determined by $\epsilon$).

### The structure of calculations

$$p(\hat{V} = v^*|V = v^*, c) = \sum\limits_{x_0^{*f}, x_0^{'f}} p(\hat{V} = v^*|\hat{X}_0^f = x_0^{'f}, c) p(\hat{X}_0^f = x_0^{'f}| X_0^f = x_0^{*f}, c) p(X_0^f = x_0^{*f}|V = v^*)$$

so $$p(\hat{V} = v^*|V = v^*, c) \approx \sum\limits_{x_0^{*f}, x_0^{'f}. d(x_0^{*f}, x_0^{'f}) \leq k} p(\hat{V} = v^*|\hat{X}_0^f = x_0^{'f}, c) \hat{p}(\hat{X}_0^f = x_0^{'f}| X_0^f = x_0^{*f}, c) p(X_0^f = x_0^{*f}|V = v^*)$$

We want to have the following distributions pre-computed (or as close to that as possible) as `numpy` arrays
 1. $p(W|V) = p(X_0^f|V)$
 2. $\hat{p}(\hat{X}_0^f|X_0^f, C)$
 3. $p(Y_0^f|W)$
 4. $p(W|C)$
 5. $p(V|W, C)$
 
so that we can efficiently calculate $p(\hat{V} = v^*| V = v^*, c)$ for all pairs of $(v^*, c)$ and
where 
 - $p(\hat{V} = v^*| V = v^*, c)$
 - $\hat{p}(\hat{X}_0^f|X_0^f, C)$, and 
 - $p(V|W,C)$ 
 
involve decreasing amounts of non-trivial computation.

### The structure of the pipeline

Given foundational files:
 - a language model $m$ over some orthographic vocabulary $V_m$
 - $n$-gram contexts $C$ taken from one or more speech corpora
 - a target orthographic vocabulary $V$ taken from each of the speech corpora that contexts are taken from
 - diphone gating data
 - one or more transcription relations $r$
 
the first step in the processing pipeline involves aligning segmental inventories of gating data and transcription relations. Once that's been done, we can define triphone channel distributions on transcription-relation aligned gating data that can be applied to gating-data aligned transcribed wordforms. We also need to define $p(V_m|C)$ for each choice of $C$.

Next, we create mutually combinable versions of transcription relations, channel distributions, and distributions over orthographic vocabularies - e.g.
 - we need to remove words from each $r$ that contain triphones that can't be modeled by the aligned triphone channel distribution
 - we need to remove words from each $r$ that aren't in the language model vocabulary $V_m$
 - $p(V_m|C)$ needs to be projected down to remove orthographic words that aren't in transcription relations and context sets $C$ need to be scrubbed of contexts containing orthographic wordforms not in the language model

With all the atomic components of the model(/each possible combined model) constructed, we then pre-calculate remaining components of the forward model(s). Finally, we calculate components of the posterior. (The separation of this last step from *everything previous* facilitates parallel computation.)

## Requirements

### Python environment
This repository was developed using Python 3.6 with the following package requirements (upstream dependencies not included):
 - **Jupyter/notebook related packages**: `jupyter` `jupyter_contrib_nbextensions` `papermill`
    - Notable notebook extensions used include `ExecuteTime` and `Table of Contents (2)`.
 - **Numeric computing and data processing**: `scipy` `numpy` `pandas` `tqdm` `joblib` `numba` `sparse` `pytorch` `torchvision`
 - **Language modeling**: `kenlm`, `arpa`
 - **Plotting**: `matplotlib` `plotnine`
 - **Misc**: `funcy` `more_itertools`

See `dev_environment.sh` for `conda` and `pip` commands to create an environment that has all of these packages. (Not all are available on conda, and not all of those that are available on conda are available from the main channel.)


### Hardware / runtime expectations

The last machine this repository was developed on (`wittgenstein`) has 32 cores. `joblib` is used extensively to parallelize data processing. On this machine, using `joblib` and often using nearly all of those cores:
1. Step 1 takes about a minute.
2. Step 2 takes about 3 hours.
3. Step 3 takes about 15-20 minutes.
4. Step 4 takes about 4.5 hours.

To run all of these scripts comfortably and without modification, you will need about 128GB of memory and ≈1 TB of hard disk space. Little or no attempt has been made to compress or avoid unnecessary file outputs except insofar as it creates representations that lead to usefully smaller memory loads or facilitate matrix-based compuations. Some unused or older versions of scripts in this repository were developed on machines with 180-190GB of RAM and use somewhere between 140 and 160GB of RAM on the largest inputs (invariably something related to the CMU dictionary - the largest lexicon by far).

In [2]:
import watermark
%load_ext watermark

In [3]:
%watermark -ihmuv

last updated: 2019-09-12T16:13:36-07:00

CPython 3.7.4
IPython 7.8.0

compiler   : GCC 7.3.0
system     : Linux
release    : 4.15.0-62-generic
machine    : x86_64
processor  : x86_64
CPU cores  : 32
interpreter: 64bit
host name  : wittgenstein


In [4]:
!lscpu

Architecture:        x86_64
CPU op-mode(s):      32-bit, 64-bit
Byte Order:          Little Endian
CPU(s):              32
On-line CPU(s) list: 0-31
Thread(s) per core:  2
Core(s) per socket:  16
Socket(s):           1
NUMA node(s):        1
Vendor ID:           AuthenticAMD
CPU family:          23
Model:               8
Model name:          AMD Ryzen Threadripper 2950X 16-Core Processor
Stepping:            2
CPU MHz:             1895.432
CPU max MHz:         3500.0000
CPU min MHz:         2200.0000
BogoMIPS:            6986.04
Virtualization:      AMD-V
L1d cache:           32K
L1i cache:           64K
L2 cache:            512K
L3 cache:            8192K
NUMA node0 CPU(s):   0-31
Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ht syscall nx mmxext fxsr_opt pdpe1gb rdtscp lm constant_tsc rep_good nopl nonstop_tsc cpuid extd_apicid amd_dcm aperfmperf pni pclmulqdq monitor ssse3 fma cx16 sse4_1 sse4_2 movbe popcnt ae

In [5]:
!free -h

              total        used        free      shared  buff/cache   available
Mem:           125G        1.0G        6.8G        105M        118G        123G
Swap:          2.0G        104M        1.9G


In [6]:
!modinfo nvidia | grep version

version:        430.40
srcversion:     B9960A7E9AD113F15C3277E


## Todo

0. **Extensibility**: Step 3b (filtering transcribed lexicons against language model vocabularies) uses output filenames that won't scale if any transcription lexicon is used with more than one language model. (This also affects 3c.)
1. **Extensibility/Modularity/Maintainability**: Every notebook that depends on the behavior or output of another notebook being a certain way should have that assumption flagged at the top of the notebook. 
   - **Most common example**: one of notebook $B$'s arguments is a *directory path* $d$, where it expects to find a specific set of files with certain fixed filenames (or with filenames that are derived in some way from one of $B$'s arguments - often $d$); this assumption is predicated on the behavior of some notebook $A$ earlier in the pipeline producing exactly some set of files in a common directory all with certain filenames.
2. **Portability/Reproducibility**: For every file that this repository depends on that *isn't* tracked by the repository (e.g. processed versions of swbd2003, Buckeye, etc.), there should be *something* (e.g. a cell in this script) that lets the user identify where those files are located, and then said something copies them to wherever this repository is expecting to find them.
3. **Portability/Reproducibility**: Check for and remove absolute paths in this and other files.
4. **Portability/Reproducibility**: For platform independence (read: supporting windows users, I guess?) use `path.join` instead of manually choosing directory slashes...
5. **Documentation**: 
   1. Make sure motivation section is up to date.
   2. Math-y documentation in channel distribution and posterior distribution notebooks probably needs to be updated / at least have notation overhauled.
   3. Go through notebooks used here and make sure `Overview` cells are accurate.
   4. Go through notebooks used here and make sure Papermill-related `Usage` cells are filled in / accurate.

# Imports

In [7]:
import papermill as pm

In [8]:
from tqdm import tqdm

In [9]:
from os import getcwd, chdir, listdir, path, mkdir, makedirs

import json
import csv

In [10]:
from copy import deepcopy

In [11]:
from collections import OrderedDict

In [12]:
from itertools import product
from boilerplate import ensure_dir_exists, union, endNote, startNote, stampedNote, stamp

In [13]:
def progress_report(nb_fp, arg_dict):
    startNote()
    output_dir = path.dirname(nb_fp)
    nb_fn = path.basename(nb_fp)
    print(f"Running notebook:\n\t{nb_fn}")
    print(f"Output directory:\n\t{output_dir}")
    print("Arguments:")
    print(json.dumps(arg_dict, indent=1))

In [14]:
from funcy import *

In [15]:
repo_dir = getcwd()
repo_dir

'/mnt/cube/home/AD/emeinhar/wr'

In [16]:
repo_contents = listdir()
repo_contents

['MeasBasAnalysis.ipynb',
 'probdist.py',
 'LD_Fisher_vocab_in_NXT_swbd_preceding_contexts_2gram_model',
 'GD_AmE-diphones - LTR_NXT_swbd_destressed alignment application to LTR_NXT_swbd_destressed.ipynb',
 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0',
 'Calculate prefix data, k-cousins, and k-spheres (vec-dev).ipynb',
 'Calculate orthographic posterior given segmental wordform + context (sparse + dask + tiledb).ipynb',
 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.05',
 'LTR_NXT_swbd_destressed',
 'CM_AmE_destressed_aligned_w_LTR_Buckeye_pseudocount0',
 'LD_Fisher_vocab_in_NXT_swbd_preceding_contexts_3gram_model',
 'buckeye_vocab_excluded_by_xlnet.txt',
 'Generate triphone lexicon distribution from channel model.ipynb',
 'swbd2003_contexts.txt',
 'Calculate orthographic posterior given segmental wordform + context (sparse tensor calculations + memory issues).ipynb',
 'boilerplate.py',
 'buckeye_contexts_filtered_against_fisher_vocabulary_main.txt

# Permitted steps / control flow

To facilitate quick (re)running of just the steps you want to (re)do and to avoid tediously (re)running steps you do not want to calculate, this list here controls what calculation steps will actually be done. (Combine with 'Run All' to facilitate quick calculation.)

In [17]:
permittedSteps = [
#     '0',
#     '1a',
#     '1b',
#     '2ai',
#     '2aii',
#     '2b',
#     '3a',
#     '3b',
#     '3c',
#     '3d',
#     '3e',
#     '4a',
#     '4b',
#     '4c',
#     '4d',
#     '4e',
    '5a',
#     '5b',
#     '5c'
]

If the flag below is `False`, then (in selected places where I've added support in this notebook) the driver notebook will check for and not overwrite existing output notebooks (evidence of a previous and assumed-to-be-successful run).

In [18]:
overwrite = False

If the flag below is `True`, then this machine will skip (*only* in Step 4e) processing 'trim' inputs. (See parameters in the Step 3e notebook.)

In [19]:
skip_trim = False

# Step 0: Import/check for foundational files

**NOTE:**
 - I assume all relevant transcriptions have been converted to Unicode IPA characters and that segment sequences have `.` as a separator. For each data source used here, the IPA alignment step is documented in a GitHub repository elsewhere. (While I don't *think* any script here depends on use of Unicode IPA symbols, I haven't - and won't - test that idea, and it is *absolutely required* that contiguous sequences of segments be separated by `.` in data files.)
 - Where language models and n-gram contexts (drawn from speech corpora) are referenced, each of these is assumed to have come from as is from other GitHub repositories/from executing the notebooks in those repositories.

## Importing existing data and creating directories

In [20]:
from shutil import copy

In [21]:
# AmE_gating_data_dir = 'GD_AmE'
# AmE_gating_data_fn = 'AmE-diphones-IPA-annotated-columns.csv'
# AmE_GD_fp = path.join(AmE_gating_data_dir, AmE_gating_data_fn)

# ensure_dir_exists(AmE_gating_data_dir)

# if not path.exists(AmE_GD_fp):
#     'Gating data file {0} does not exist. Attempting to copy from repository in sister folder.'.format(AmE_GD_fp)
    
#     wmc2014ipa_repo_dir = '../wmc2014-ipa'

#     assert path.exists(wmc2014ipa_repo_dir), "Cannot move gating data file from repository for you because it cannot be found.\n For automatic placement, you must download the wmc2014-ipa repository (from https://github.com/emeinhardt/wmc2014-ipa) and place it in the same folder as 'wr'"

#     copy(path.join(wmc2014ipa_repo_dir, AmE_gating_data_fn),
#          path.join(repo_dir, AmE_gating_data_dir, AmE_gating_data_fn))

In [22]:
# newdic_destressed_ltr_folder = 'LTR_newdic_destressed'
# cmu_destressed_ltr_folder = 'LTR_CMU_destressed'
# cmu_stressed_ltr_folder = 'LTR_CMU_stressed'
# buckeye_ltr_folder = 'LTR_Buckeye'
# nxt_swbd_ltr_folder = 'LTR_NXT_swbd_destressed'

# LTR_folders = (newdic_destressed_ltr_folder, cmu_destressed_ltr_folder, cmu_stressed_ltr_folder, buckeye_ltr_folder, nxt_swbd_ltr_folder)
# LTR_folders_to_process = (newdic_destressed_ltr_folder, cmu_destressed_ltr_folder, buckeye_ltr_folder, nxt_swbd_ltr_folder)

# LTR_folders_to_repo_dir = {
#     newdic_destressed_ltr_folder:'../newdic-nettalk-ipa',
#     cmu_destressed_ltr_folder:'../cmu-ipa',
#     cmu_stressed_ltr_folder:'../cmu-ipa',
#     buckeye_ltr_folder:'../buckeye-lm',
#     nxt_swbd_ltr_folder:'../switchboard-lm'
# }

# for LTR_dir in LTR_folders_to_process:
#     ensure_dir_exists(LTR_dir)
#     LTR_fp = path.join(LTR_dir, LTR_dir + '.tsv')
    
#     if not path.exists(LTR_fp):
#         'Transcribed lexicon relation {0} does not exist. Attempting to copy from repository in sister folder.'.format(LTR_fp)
    
#     LTR_repo_dir = LTR_folders_to_repo_dir[LTR_dir]
#     assert path.exists(LTR_repo_dir), "Cannot move transcribed lexicon relation .tsv from repository for you because it cannot be found.\n For automatic placement, you must download the {0} repository (from https://github.com/emeinhardt/{0}) and place it in the same folder as 'wr'".format(LTR_repo_dir)
    
    

The four steps below verify that foundational files assumed to be present by downstream notebooks are, in fact, present in the repository directory. If for some reason those files are not present, the processing pipeline will be aborted.

## Step 0a: Check for gating data

In [23]:
AmE_gating_data_dir = 'GD_AmE'
AmE_gating_data_fn = 'AmE-diphones-IPA-annotated-columns.csv'
AmE_GD_fp = path.join(AmE_gating_data_dir, AmE_gating_data_fn)

ensure_dir_exists(AmE_gating_data_dir)

if not path.exists(AmE_GD_fp):
    'Gating data file {0} does not exist. Attempting to copy from repository in sister folder.'.format(AmE_GD_fp)
    
    wmc2014ipa_repo_dir = '../wmc2014-ipa'

    assert path.exists(wmc2014ipa_repo_dir), "Cannot move gating data file from repository for you because it cannot be found.\n For automatic placement, you must download the wmc2014-ipa repository (from https://github.com/emeinhardt/wmc2014-ipa) and place it in the same folder as 'wr'"

    copy(path.join(wmc2014ipa_repo_dir, AmE_gating_data_fn),
         path.join(repo_dir, AmE_gating_data_dir, AmE_gating_data_fn))

In [24]:
# AmE_gating_data_dir = 'GD_AmE'
# AmE_gating_data_fn = 'AmE-diphones-IPA-annotated-columns.csv'
# AmE_GD_fp = path.join(AmE_gating_data_dir, AmE_gating_data_fn)
assert path.exists(AmE_gating_data_dir), 'Gating data directory {0} does not exist.'.format(AmE_gating_data_dir)
assert path.exists(AmE_GD_fp), 'Gating data file {0} does not exist.'.format(AmE_GD_fp)

The processed gating data used here come from 
 - https://github.com/emeinhardt/wmc2014-ipa
 
See those repositories for information on how they were produced.

## Step 0b: Check for transcribed lexicons

 - Each transcribed lexicon `LEXNAME` should be in a folder (e.g. `LTR_LEXNAME`) containing a file `LTR_LEXNAME.tsv`. For documentation purposes, the source file and a notebook documenting the production of the `.tsv` file should, if practicable be included in the folder as well.
   - A transcribed lexicon `LTR_....tsv` file should have two columns: `Orthographic_Wordform` and `Transcription`.
   - NB: The `LTR_` prefix on transcribed lexicon data files and containing folders is simply a convention for organization and readability, but is not required or expected by any file or script in this repository.

The assertions in the code below will only succeed if step 1 is complete for all transcribed lexicons listed for checking below.

In [25]:
newdic_destressed_ltr_folder = 'LTR_newdic_destressed'
cmu_destressed_ltr_folder = 'LTR_CMU_destressed'
cmu_stressed_ltr_folder = 'LTR_CMU_stressed'
buckeye_ltr_folder = 'LTR_Buckeye'
nxt_swbd_ltr_folder = 'LTR_NXT_swbd_destressed'

LTR_folders = (newdic_destressed_ltr_folder, cmu_destressed_ltr_folder, cmu_stressed_ltr_folder, buckeye_ltr_folder, nxt_swbd_ltr_folder)
LTR_folders_to_process = (newdic_destressed_ltr_folder, cmu_destressed_ltr_folder, buckeye_ltr_folder, nxt_swbd_ltr_folder)

# LTR_folders_to_repo_dir = {
#     newdic_destressed_ltr_folder:'../newdic-nettalk-ipa',
#     cmu_destressed_ltr_folder:'../cmu-ipa',
#     cmu_stressed_ltr_folder:'../cmu-ipa',
#     buckeye_ltr_folder:'../buckeye-lm',
#     nxt_swbd_ltr_folder:'../switchboard-lm'
# }

# for LTR_dir in LTR_folders_to_process:
#     ensure_dir_exists(LTR_dir)
#     LTR_fp = path.join(LTR_dir, LTR_dir + '.tsv')
    
#     if not path.exists(LTR_fp):
#         'Transcribed lexicon relation {0} does not exist.'.format(LTR_fp)
    
#         if LTR_dir in {buckeye_ltr_folder, nxt_swbd_ltr_folder}:
#             print('Attempting to copy transcribed lexicon relation .tsv from repository folder...')
            
#             LTR_repo_dir = LTR_folders_to_repo_dir[LTR_dir]
            
#             assert path.exists(LTR_repo_dir), "Cannot move transcribed lexicon relation .tsv from repository for you because it cannot be found.\n For automatic placement, you must download the {0} repository (from https://github.com/emeinhardt/{0}) and place it in the same folder as 'wr'".format(LTR_repo_dir)
            
#             copy(path.join(LTR_repo_dir, FIXME), 
#                  path.join(repo_dir, LTR_dir, LTR_dir + '.tsv'))
    
    

for dirname in tqdm(LTR_folders_to_process):
    assert path.exists(dirname), 'Transcribed lexicon directory {0} not found in repo directory'.format(dirname)
    fname = path.join(dirname, dirname + '.tsv')
    assert path.exists(fname), 'Transcribed lexicon {0} not found in repo directory'.format(fname)

100%|██████████| 4/4 [00:00<00:00, 314.90it/s]


**How are transcribed lexicon relations made?**

Each was created by processing a transcription source (a lexical database, an annotated corpus, etc.). The processing step is described in other repositories:
 - https://github.com/emeinhardt/newdic-nettalk-ipa
 - https://github.com/emeinhardt/cmu-ipa
 - https://github.com/emeinhardt/buckeye-lm
 - https://github.com/emeinhardt/switchboard-lm

Given the processed outputs of these repositories, the `Making a Transcribed Lexicon Relation - <LEXNAME>.ipynb` notebook in each `LTR_LEXNAME` folder describes how the homogeneous `.tsv` files downstream steps are created.

In [26]:
for dirname in LTR_folders_to_process:
    listdir(dirname)

['LTR_newdic_destressed.tsv',
 'Making a Transcribed Lexicon Relation - newdic_destressed.ipynb',
 'newdic_IPA.tsv',
 '.ipynb_checkpoints']

['LTR_CMU_destressed.pW_V.npz_metadata.json',
 'Making a Transcribed Lexicon Relation - CMU_destressed.ipynb',
 'LTR_CMU_destressed.pW_V.json',
 'LTR_CMU_destressed.tsv',
 '.ipynb_checkpoints',
 'LTR_CMU_destressed.pW_V.npz',
 'cmudict-0.7b_IPA_destressed.tsv',
 'LTR_CMU_destressed_Orthographic_Wordforms.txt',
 'LTR_CMU_destressed_Transcriptions.txt']

['LTR_Buckeye.tsv',
 'buckeye_orthography_phonemic_transcription_relation.tsv',
 'Making a Transcribed Lexicon Relation - Buckeye-old.ipynb',
 'buckeye_words_analysis_relation.json',
 '.ipynb_checkpoints',
 'Making a Transcribed Lexicon Relation - Buckeye.ipynb']

['nxt_swbd_orthography_transcription_relation.tsv',
 '.ipynb_checkpoints',
 'Making a Transcribed Lexicon Relation - NXT_swbd.ipynb',
 'LTR_NXT_swbd_destressed.tsv']

## Step 0c: Check for n-gram contexts

In [27]:
from funcy import str_join, walk_values

In [28]:
context_fns = {
    #swbd2003 wordforms need to be POS tagged and 
    # contexts need to be organized by size + correctly constructed and
    # exclusion criteria need to be applied
#     'swbd2003':FIXME,
    'Buckeye':{'preceding':{l:str_join('_', ['buckeye', 'contexts', 'preceding', str(l), 'filtered']) + '.txt' 
                            for l in (1,2,3,4)},
               'following':{l:str_join('_', ['buckeye', 'contexts', 'following', str(l), 'filtered']) + '.txt' 
                            for l in (1,2,3,4)},
               'bidirectional':'buckeye_contexts_bidirectional_filtered.json'},
    'NXT_swbd':{'preceding':{l:str_join('_', ['nxt_swbd', 'contexts', 'preceding', str(l), 'filtered']) + '.txt' 
                            for l in (1,2,3,4)},
                'following':{l:str_join('_', ['nxt_swbd', 'contexts', 'following', str(l), 'filtered']) + '.txt' 
                            for l in (1,2,3,4)},
                'bidirectional':'nxt_swbd_contexts_bidirectional_filtered.json'}
}

context_dirs = tuple([str_join('_', ('C', corpus_name)) 
                      for corpus_name in context_fns])

for context_dir in context_dirs:
    ensure_dir_exists(context_dir)

context_dir_to_repo_dir = {
#     swbd2003_ltr_folder:'../switchboard-lm',
    'Buckeye':'../buckeye-lm',
    'NXT_swbd':'../switchboard-lm'
}

corpus_to_contexts = {
    'Buckeye':list(sorted({context_fns['Buckeye'][direction][l] 
                           for direction in ('preceding', 'following') for l in (1,2,3,4)})) + ['buckeye_contexts_bidirectional_filtered.json'],
    'NXT_swbd':list(sorted({context_fns['NXT_swbd'][direction][l] 
                           for direction in ('preceding', 'following') for l in (1,2,3,4)})) + ['nxt_swbd_contexts_bidirectional_filtered.json'],
}
corpus_to_contexts = walk_values(tuple, corpus_to_contexts)
corpus_to_contexts

for context_dir in context_dirs:
    context_name = context_dir[2:]
    for direction in context_fns[context_name]:
        if direction == 'bidirectional':
            context_fn = context_fns[context_name][direction]
            
            if not path.exists(path.join(context_dir, context_fn)):
                print('{0} not found in {1}.'.format(context_fn, context_dir))
                repo_dir = context_dir_to_repo_dir[context_name]
                print('Attempting to copy context file {0} from {1}.'.format(context_fn, repo_dir))
                
                copy(path.join(repo_dir, context_fn),
                     path.join(context_dir, context_fn))
        else:
            for l in context_fns[context_name][direction]:
                context_fn = context_fns[context_name][direction][l]
                
                if not path.exists(path.join(context_dir, context_fn)):
                    print('{0} not found in {1}.'.format(context_fn, context_dir))
                    repo_dir = context_dir_to_repo_dir[context_name]
                    print('Attempting to copy context file {0} from {1}.'.format(context_fn, repo_dir))
                
                    copy(path.join(repo_dir, context_fn),
                         path.join(context_dir, context_fn))
                

# buckeye_contexts = 'buckeye_contexts.txt'
# swbd2003_contexts = 'swbd2003_contexts.txt'

# contexts = (buckeye_contexts, swbd2003_contexts)

# for c_fn in contexts:
#     assert path.exists(c_fn), "N-gram contexts file {0} does not exist.".format(c_fn)

{'Buckeye': ('buckeye_contexts_following_1_filtered.txt',
  'buckeye_contexts_following_2_filtered.txt',
  'buckeye_contexts_following_3_filtered.txt',
  'buckeye_contexts_following_4_filtered.txt',
  'buckeye_contexts_preceding_1_filtered.txt',
  'buckeye_contexts_preceding_2_filtered.txt',
  'buckeye_contexts_preceding_3_filtered.txt',
  'buckeye_contexts_preceding_4_filtered.txt',
  'buckeye_contexts_bidirectional_filtered.json'),
 'NXT_swbd': ('nxt_swbd_contexts_following_1_filtered.txt',
  'nxt_swbd_contexts_following_2_filtered.txt',
  'nxt_swbd_contexts_following_3_filtered.txt',
  'nxt_swbd_contexts_following_4_filtered.txt',
  'nxt_swbd_contexts_preceding_1_filtered.txt',
  'nxt_swbd_contexts_preceding_2_filtered.txt',
  'nxt_swbd_contexts_preceding_3_filtered.txt',
  'nxt_swbd_contexts_preceding_4_filtered.txt',
  'nxt_swbd_contexts_bidirectional_filtered.json')}

The n-gram context files are taken from
 - https://github.com/emeinhardt/buckeye-lm
 - https://github.com/emeinhardt/switchboard-lm
 
See those repositories for more information on how the contexts were extracted. (*NB*: Like the transcription lexicons, the context files are not included in this repository both to avoid duplication and because of licensing restrictions: to recreate these contexts, you will need access to your own copy of the Buckeye and (various) Switchboard corpora.)

## Step 0d: Check for language model(s)

In [29]:
fisher_lm_dir = 'LM_Fisher'

ensure_dir_exists(fisher_lm_dir)

LM_fn_stem = 'fisher_utterances_main'
LM_fns = {
    '':{l:LM_fn_stem + '_' + str(l) + 'gram.mmap'
        for l in (2,3,4,5)},
    'rev':{l:LM_fn_stem + '_' + 'rev' + '_' + str(l) + 'gram.mmap'
           for l in (2,3,4,5)}
}

fisher_lm_repo_dir = '../fisher-lm'

for direction in LM_fns:
    for l in LM_fns[direction]:
        lm_fn = LM_fns[direction][l]
        
        if not path.exists(path.join(fisher_lm_dir, lm_fn)):
            print('{0} not found in {1}'.format(lm_fn, fisher_lm_dir))
            print('Attempting to copy from repository directory...')
            copy(path.join(fisher_lm_repo_dir, lm_fn),
                 path.join(fisher_lm_dir, lm_fn))
            

# fisher_lm_fn = 'fisher_utterances_main_4gram.mmap'
# fisher_lm_fp = path.join(fisher_lm_dir, fisher_lm_fn)

fisher_lm_vocab_fn = 'fisher_vocabulary_main.txt'
if not path.exists(path.join(fisher_lm_dir, fisher_lm_vocab_fn)):
    print('{0} not found in {1}'.format(fisher_lm_vocab_fn, fisher_lm_dir))
    print('Attempting to copy from repository directory...')
    copy(path.join(fisher_lm_repo_dir, fisher_lm_vocab_fn),
         path.join(fisher_lm_dir, fisher_lm_vocab_fn))

# fisher_lm_vocab_fp = path.join(fisher_lm_dir, fisher_lm_vocab_fn)

# assert path.exists(fisher_lm_fp), 'Language model {0} not found'.format(fisher_lm_fp)
# assert path.exists(fisher_lm_vocab_fp), 'Language model vocabulary {0} not found'.format(fisher_lm_vocab_fp)

# fisher_lm_fps = {'lm':fisher_lm_fp, 
#                  'vocab':fisher_lm_vocab_fp}

The (memory mapped) n-gram language models are copied from the output of this repository:
 - https://github.com/emeinhardt/fisher-lm
 
See that repository for more information. (*NB* Again, the language model file is not included in this repository both to avoid duplication and because of licensing restrictions: to recreate these contexts, you will need access to your own copy of the Buckeye and Switchboard corpora.

# Step 1: Segment inventory alignment

## Step 1a: Define inventory alignment projections

The segment inventory of any given transcribed lexicon and the segment inventory of the gating data often do not line up. For the gating data to be usefully applied to a given lexicon of transcriptions, the strings in the (segmental) lexicon must contain only segments found in the gating data stimuli inventory.

To ensure this happens, the notebook `Gating Data - Transcription Lexicon Alignment Maker.ipynb` 
 - takes as inputs 
     - a transcribed lexicon file path and a gating data file path
     - a lexicon projection file path and a gating data projection file path
 - identifies the inventories of each and what symbols are relatively unique to the lexicon and the gating data
 - produces 
   - *a Jupyter notebook* for **you to open and finish by defining two projection functions** (i.e. Python dictionaries) to be applied to strings in the transcribed lexicon and to the gating data (one function for each). When you finish doing this (and set an export flag in the notebook to True and run the remainder of the notebook), this notebook will produce
     - two *.json files storing these projections* according to the previously provided output file paths.

The cell below will clear all existing alignment folders created using the code in this subsection:

In [30]:
# %rm -rf LTR*_aligned_w_*
# %rm -rf *" alignment definition"*

The cell below will only succeed if the American English gating data of Warner, McQueen, and Cutler (2014) is contained in the repo directory with a particular directory and filename.

In [31]:
gating_data_folder = 'GD_AmE'
gating_data_fn = 'AmE-diphones-IPA-annotated-columns.csv'
gating_data_fp = path.join(gating_data_folder, gating_data_fn)

assert path.exists(gating_data_folder), 'AmE gating data folder {0} not found in repo directory'.format(gating_data_folder)
assert path.exists(gating_data_fp), 'AmE gating data {0} not found in repo directory'.format(gating_data_fp)

The third cell below will create a notebook for alignment projection definitions for each of the transcribed lexicons from the previous step and the AmE gating data.

In [32]:
#FIXME replace usage with path.splitext
def removeExtension(fp):
    dir_name = path.dirname(fp)
    file_name = path.basename(fp)
    ext = file_name.split('.')[-1]
    rest = '.'.join( file_name.split('.')[:-1] )
    return path.join(dir_name, rest)

In [33]:
alignment_arg_bundles = []
for LTR_dirname in LTR_folders_to_process:
    LTR_fn = LTR_dirname + '.tsv'
    LTR_fp = path.join(LTR_dirname, LTR_fn)
    
    nb_output_name = 'GD_AmE-diphones - ' + LTR_dirname + ' alignment definition' + '.ipynb'
    my_g = gating_data_fp
    my_l = LTR_fp
    my_s = 'destressed'
    
    gd_alignment_dn = 'GD_AmE_' + my_s + '_' + 'aligned_w_' + LTR_dirname
    gd_alignment_fn = 'alignment_of_' + removeExtension(gating_data_fn) + '_w_' + LTR_dirname + '.json'
    gd_alignment_fp = path.join(gd_alignment_dn, gd_alignment_fn)
    if not path.exists(gd_alignment_dn):
        makedirs(gd_alignment_dn)
    my_gp = gd_alignment_fp
    
    ltr_alignment_dn = LTR_dirname + '_aligned_w_' + 'GD_AmE_' + my_s
    ltr_alignment_fn = 'alignment_of_' + LTR_dirname + '_w_' + removeExtension(gating_data_fn) + '.json'
    ltr_alignment_fp = path.join(ltr_alignment_dn, ltr_alignment_fn)
    if not path.exists(ltr_alignment_dn):
        makedirs(ltr_alignment_dn)
    my_lp = ltr_alignment_fp
    
    
    my_arg_bundle = OrderedDict({
        'LTR_dirname':LTR_dirname,
        'LTR_fn':LTR_fn,
        'LTR_fp':LTR_fp,
        'gd_alignment_dn':gd_alignment_dn,
        'gd_alignment_fn':gd_alignment_fn,
        'gd_alignment_fp':gd_alignment_fp,
        'ltr_alignment_dn':ltr_alignment_dn,
        'ltr_alignment_fn':ltr_alignment_fn,
        'ltr_alignment_fp':ltr_alignment_fp,
        'align_def_nb_output_name':nb_output_name,
        'my_g':my_g,
        'my_l':my_l,
        'my_s':my_s,
        'my_gp':my_gp,
        'my_lp':my_lp,
    })
    alignment_arg_bundles.append(my_arg_bundle)

In [34]:
if '1a' in permittedSteps:
    # takes ~30s on wittgenstein
    for arg_bundle in tqdm(alignment_arg_bundles):
        nb = pm.execute_notebook(
            'Gating Data - Transcription Lexicon Alignment Maker.ipynb',
            arg_bundle['align_def_nb_output_name'],
            parameters=dict(g = arg_bundle['my_g'], 
                            l = arg_bundle['my_l'], 
                            s = arg_bundle['my_s'], 
                            gp = arg_bundle['my_gp'], 
                            lp = arg_bundle['my_lp'])
        )
    #     pm.execute_notebook(
    #        'Gating Data - Transcription Lexicon Alignment Maker.ipynb',
    #        nb_output_name,
    #        parameters=dict(g = my_g, l = my_l, s = my_s, gp = my_gp, lp = my_lp)
    #     )
        print("Finished creating alignment definition notebook '{0}'.\nOpen and run the notebook, complete the projection definition, and run the remainder of the notebook (remembering to change the export flag to 'True').\n".format(arg_bundle['align_def_nb_output_name']))

## Step 1b: Apply inventory alignment projections

The cell below will clear all existing alignment folders created using the code in this subsection:

In [35]:
# %rm -rf *" alignment application "*

### Check for projection definitions

The cell below will succeed if you have run each of the previously produced notebooks correctly and produced a projection mapping file.

In [36]:
for arg_bundle in alignment_arg_bundles:
    args = arg_bundle
    assert path.exists(args['gd_alignment_fp']), 'Gating data alignment projection mapping not found:\n\t{0}'.format(args['gd_alignment_fp'])
    assert path.exists(args['ltr_alignment_fp']), 'Transcribed lexicon data alignment projection mapping not found:\n\t{0}'.format(args['ltr_alignment_fp'])

### How are inventory alignment projections actually applied?

See `Align transcriptions.ipynb`.

### Apply projection definitions

The cell below applies each pair of alignment projections to each matched pair of gating data and transcribed lexicon choice:

In [37]:
for arg_bundle in alignment_arg_bundles:
    args = arg_bundle
    LTR_fn = args['LTR_fn']
    
#     my_pg = args['my_gp']
#     my_g = args['my_g']
    my_o_fn = 'GD_AmE-diphones' + '_aligned_w_' + removeExtension(LTR_fn) + '.tsv'
    my_og = path.join(args['gd_alignment_dn'], my_o_fn)
    args['align_apply_gd_nb_output_name'] = 'GD_AmE-diphones - ' + removeExtension(LTR_fn) + ' alignment application to ' + 'AmE-diphones' + '.ipynb'
    args['my_og'] = my_og
    
#     my_pl = args['my_lp']
#     my_l = args['my_l']
    my_o_fn = removeExtension(LTR_fn) + '_aligned_w_' + 'GD_AmE-diphones' + '.tsv'
    my_ol = path.join(args['ltr_alignment_dn'], my_o_fn)
    args['align_apply_ltr_nb_output_name'] = 'GD_AmE-diphones - ' + removeExtension(LTR_fn) + ' alignment application to ' + removeExtension(LTR_fn) + '.ipynb'
    args['my_ol'] = my_ol

In [38]:
if '1b' in permittedSteps:
    # takes ~45s on wittgenstein
    for arg_bundle in alignment_arg_bundles:
        args = arg_bundle
    #     LTR_fn = args['LTR_fn']
        startNote()
        my_pg = args['my_gp']
        my_g = args['my_g']
    #     my_o_fn = 'GD_AmE-diphones' + '_aligned_w_' + removeExtension(LTR_fn) + '.tsv'
    #     my_og = path.join(args['gd_alignment_dn'], my_o_fn)
    #     args['align_apply_gd_nb_output_name'] = 'GD_AmE-diphones - ' + removeExtension(LTR_fn) + ' alignment application to ' + 'AmE-diphones' + '.ipynb'
    #     args['my_og'] = my_og
        my_og = args['my_og']
        print("Creating notebook '{0}' w/ args p, g, o = \n\t{1}\n\t{2}\n\t{3}".format(args['align_apply_gd_nb_output_name'], my_pg, my_g, my_og))
        nb = pm.execute_notebook(
            'Align transcriptions.ipynb',
            args['align_apply_gd_nb_output_name'],
            parameters=dict(p = my_pg,
                            g = my_g,
                            o = my_og)
        )
        print('Finished applying alignment projection\n\tp = {0}\nto\n\tg = {1}\nResult saved to\n\t{2}'.format(my_pg, my_g, my_og))
        print(' ')

        my_pl = args['my_lp']
        my_l = args['my_l']
    #     my_o_fn = removeExtension(LTR_fn) + '_aligned_w_' + 'GD_AmE-diphones' + '.tsv'
    #     my_ol = path.join(args['ltr_alignment_dn'], my_o_fn)
    #     args['align_apply_ltr_nb_output_name'] = 'GD_AmE-diphones - ' + removeExtension(LTR_fn) + ' alignment application to ' + removeExtension(LTR_fn) + '.ipynb'
    #     args['my_ol'] = my_ol
        my_ol = args['my_ol']
        print('Creating notebook {0} w/ args p, g, o = \n\t{1}\n\t{2}\n\t{3}'.format(args['align_apply_ltr_nb_output_name'], my_pg, my_l, my_ol))
        nb = pm.execute_notebook(
            'Align transcriptions.ipynb',
            args['align_apply_ltr_nb_output_name'],
            parameters=dict(p = my_pl,
                            l = my_l,
                            o = my_ol)
        )
        print('Finished applying alignment projection\n\tp = {0}\nto\n\tl = {1}\nResult saved to\n\t{2}'.format(my_pl, my_l, my_ol))
        endNote()
        print('\n')

# Step 2: Generating channel and (orthographic) lexicon distributions

## Step 2a: Generating channel distributions and associated metadata

In [39]:
%ls -d GD_*

 GD_AmE/
 GD_AmE_destressed_aligned_w_LTR_Buckeye/
 GD_AmE_destressed_aligned_w_LTR_CMU_destressed/
 GD_AmE_destressed_aligned_w_LTR_newdic_destressed/
 GD_AmE_destressed_aligned_w_LTR_NXT_swbd_destressed/
'GD_AmE-diphones - LTR_Buckeye alignment application to AmE-diphones.ipynb'
'GD_AmE-diphones - LTR_Buckeye alignment application to LTR_Buckeye.ipynb'
'GD_AmE-diphones - LTR_Buckeye alignment definition.ipynb'
'GD_AmE-diphones - LTR_CMU_destressed alignment application to AmE-diphones.ipynb'
'GD_AmE-diphones - LTR_CMU_destressed alignment application to LTR_CMU_destressed.ipynb'
'GD_AmE-diphones - LTR_CMU_destressed alignment definition.ipynb'
'GD_AmE-diphones - LTR_newdic_destressed alignment application to AmE-diphones.ipynb'
'GD_AmE-diphones - LTR_newdic_destressed alignment application to LTR_newdic_destressed.ipynb'
'GD_AmE-diphones - LTR_newdic_destressed alignment definition.ipynb'
'GD_AmE-diphones - LTR_NXT_swbd_destressed alignment application to AmE-diphones.ipynb'
'GD_AmE-

In [40]:
gating_data_folders = ('GD_AmE', ) + tuple(map(lambda ab: ab['gd_alignment_dn'], alignment_arg_bundles))
gating_data_folders

('GD_AmE',
 'GD_AmE_destressed_aligned_w_LTR_newdic_destressed',
 'GD_AmE_destressed_aligned_w_LTR_CMU_destressed',
 'GD_AmE_destressed_aligned_w_LTR_Buckeye',
 'GD_AmE_destressed_aligned_w_LTR_NXT_swbd_destressed')

In [41]:
gating_data_fps = ('GD_AmE/AmE-diphones-IPA-annotated-columns.csv',) + \
                  tuple(map(lambda ab: ab['my_og'], alignment_arg_bundles))
gating_data_fps

('GD_AmE/AmE-diphones-IPA-annotated-columns.csv',
 'GD_AmE_destressed_aligned_w_LTR_newdic_destressed/GD_AmE-diphones_aligned_w_LTR_newdic_destressed.tsv',
 'GD_AmE_destressed_aligned_w_LTR_CMU_destressed/GD_AmE-diphones_aligned_w_LTR_CMU_destressed.tsv',
 'GD_AmE_destressed_aligned_w_LTR_Buckeye/GD_AmE-diphones_aligned_w_LTR_Buckeye.tsv',
 'GD_AmE_destressed_aligned_w_LTR_NXT_swbd_destressed/GD_AmE-diphones_aligned_w_LTR_NXT_swbd_destressed.tsv')

### Metadata

First (for downstream convenience) we identify the $n$-phones (not) contained in and (not) constructible from each of the versions of the gating data.

**NB:** This is done by calling the notebook `Run n-phone analysis of gating data.ipynb`, passing it **the filepath to a gating data `.csv`/`.tsv` file** and **a path to an output directory** for the dozen or so files the notebook will produce.

In [42]:
if '2ai' in permittedSteps:
    # takes ~2m on wittgenstein
    for gating_data_fp in gating_data_fps:
        gd_dir = path.dirname(gating_data_fp)

        progress_report(path.join(gd_dir, gd_dir) + " n-phone analysis.ipynb",
                        dict(g = gating_data_fp,
                            o = gd_dir))
        nb = pm.execute_notebook(
            'Run n-phone analysis of gating data.ipynb',
    #         args['align_apply_ltr_nb_output_name'],
            path.join(gd_dir, gd_dir) + " n-phone analysis.ipynb",
            parameters=dict(g = gating_data_fp,
                            o = gd_dir)
        )
        listdir(gd_dir)
        endNote()
        print('\n')

### Channel distributions

Next, the notebook `Producing channel distributions.ipynb` will create `.json` files defining (among other things) a uniphone and triphone channel distribution. It requires the following arguments to specify information about what kind of channel model to build and where to put it:
 - **a filepath** to a gating data file
 - **a directory** containing metadata indicating possible/impossible $n$-phones
 - **a string argument** ("stressed" or "destressed") indicating whether the distribution will be over a segment inventory with or without stress information
 - **a real valued**  smoothing parameter (a pseudocount to add to every channel outcome)
 - **an output directory** to write the channel model to.

In [43]:
pseudocounts = (0, 0.01, 0.05, 0.1)

In [44]:
cm_arg_bundles = []
for gating_data_fp in gating_data_fps:
    metadata_dir = path.dirname(gating_data_fp)
    s = "destressed"
    channel_model_dir_stem = 'CM' + metadata_dir[2:]
    
    for pc in pseudocounts:
        channel_model_dir_suffix = '_pseudocount' + str(pc)
        if metadata_dir == 'GD_AmE':
            channel_model_dir = channel_model_dir_stem + '_' + s + '_unaligned' + channel_model_dir_suffix
        else:
            channel_model_dir = channel_model_dir_stem + channel_model_dir_suffix
        nb_output_name = 'Producing channel distributions from ' + metadata_dir + ', pc={0}'.format(pc) + '.ipynb'
        new_arg_bundle = {'gating_data_fp':gating_data_fp,
                          'metadata_dir':metadata_dir,
                          's':s,
                          'c':pc,
                          'cm_dir':channel_model_dir,
                          'nb_output_name':nb_output_name,
                          'nb_fp':path.join(channel_model_dir, nb_output_name)}
        cm_arg_bundles.append(new_arg_bundle)

In [45]:
cm_arg_bundles[0]

{'gating_data_fp': 'GD_AmE/AmE-diphones-IPA-annotated-columns.csv',
 'metadata_dir': 'GD_AmE',
 's': 'destressed',
 'c': 0,
 'cm_dir': 'CM_AmE_destressed_unaligned_pseudocount0',
 'nb_output_name': 'Producing channel distributions from GD_AmE, pc=0.ipynb',
 'nb_fp': 'CM_AmE_destressed_unaligned_pseudocount0/Producing channel distributions from GD_AmE, pc=0.ipynb'}

In [46]:
if '2aii' in permittedSteps:
    # takes ~110m on wittgenstein
    for ab in cm_arg_bundles:
        ensure_dir_exists(ab['cm_dir'])

        progress_report(ab['nb_fp'],
#                         path.join(ab['cm_dir'], ab['nb_output_name']),
                        dict(g = ab['gating_data_fp'],
                             m = ab['metadata_dir'],
                             s = ab['s'],
                             c = ab['c'],
                             o = ab['cm_dir']))

        if not overwrite and path.exists(ab['nb_fp']):
#         if not overwrite and path.exists(path.join(ab['cm_dir'], ab['nb_output_name'])):
            print('{0} already exists. Skipping...'.format(path.join(ab['nb_fp'])))
#             print('{0} already exists. Skipping...'.format(path.join(ab['cm_dir'], ab['nb_output_name'])))
            endNote()
            continue
        
        nb = pm.execute_notebook(
        'Producing channel distributions.ipynb',
        ab['nb_fp'],
#         path.join(ab['cm_dir'], ab['nb_output_name']),
        parameters=dict(g = ab['gating_data_fp'],
                        m = ab['metadata_dir'],
                        s = ab['s'],
                        c = ab['c'],
                        o = ab['cm_dir'])
        )
        endNote()
        print('\n')

**How are channel distributions created?**

Take a look at `Producing channel distributions.ipynb`. Besides removing stress information, there are some mathematically non-trivial details that go into defining both uniphone and triphone channel distributions.

## Step 2b: Generating (contextual) lexicon distributions (over orthographic vocabularies)

Given 
 - a language model $m$ (defined by a `.arpa` file or `kenlm` memory-mapped analogue) 
 - a choice of n-gram contexts $C$ (a `.txt` file with one context per line)
 - a vocabulary $V$ (a `.txt` file with one word per line)
 - a (partial) output filepath $o$ / output filepath prefix $o$
 
`Producing contextual distributions.ipynb` will write a serialized/memory-mapped `numpy` array to $o$.hV_C that defines $-log_2( p(V|C) )$ - slightly transformed output from `kenlm`. It will also write $p(V|C)$ to $o$.pV_C, and copy both $V$ and $C$ to the base directory specified by $o$. (In both cases, each column is associated with the distribution $p(V|c)$ for some $c$.)

In [47]:
context_fns
context_dirs
corpus_to_contexts
context_dir_to_repo_dir

{'Buckeye': {'preceding': {1: 'buckeye_contexts_preceding_1_filtered.txt',
   2: 'buckeye_contexts_preceding_2_filtered.txt',
   3: 'buckeye_contexts_preceding_3_filtered.txt',
   4: 'buckeye_contexts_preceding_4_filtered.txt'},
  'following': {1: 'buckeye_contexts_following_1_filtered.txt',
   2: 'buckeye_contexts_following_2_filtered.txt',
   3: 'buckeye_contexts_following_3_filtered.txt',
   4: 'buckeye_contexts_following_4_filtered.txt'},
  'bidirectional': 'buckeye_contexts_bidirectional_filtered.json'},
 'NXT_swbd': {'preceding': {1: 'nxt_swbd_contexts_preceding_1_filtered.txt',
   2: 'nxt_swbd_contexts_preceding_2_filtered.txt',
   3: 'nxt_swbd_contexts_preceding_3_filtered.txt',
   4: 'nxt_swbd_contexts_preceding_4_filtered.txt'},
  'following': {1: 'nxt_swbd_contexts_following_1_filtered.txt',
   2: 'nxt_swbd_contexts_following_2_filtered.txt',
   3: 'nxt_swbd_contexts_following_3_filtered.txt',
   4: 'nxt_swbd_contexts_following_4_filtered.txt'},
  'bidirectional': 'nxt_swbd_

('C_Buckeye', 'C_NXT_swbd')

{'Buckeye': ('buckeye_contexts_following_1_filtered.txt',
  'buckeye_contexts_following_2_filtered.txt',
  'buckeye_contexts_following_3_filtered.txt',
  'buckeye_contexts_following_4_filtered.txt',
  'buckeye_contexts_preceding_1_filtered.txt',
  'buckeye_contexts_preceding_2_filtered.txt',
  'buckeye_contexts_preceding_3_filtered.txt',
  'buckeye_contexts_preceding_4_filtered.txt',
  'buckeye_contexts_bidirectional_filtered.json'),
 'NXT_swbd': ('nxt_swbd_contexts_following_1_filtered.txt',
  'nxt_swbd_contexts_following_2_filtered.txt',
  'nxt_swbd_contexts_following_3_filtered.txt',
  'nxt_swbd_contexts_following_4_filtered.txt',
  'nxt_swbd_contexts_preceding_1_filtered.txt',
  'nxt_swbd_contexts_preceding_2_filtered.txt',
  'nxt_swbd_contexts_preceding_3_filtered.txt',
  'nxt_swbd_contexts_preceding_4_filtered.txt',
  'nxt_swbd_contexts_bidirectional_filtered.json')}

{'Buckeye': '../buckeye-lm', 'NXT_swbd': '../switchboard-lm'}

In [48]:
context_fns
context_dirs
corpus_to_contexts
context_dir_to_repo_dir

# fisher_lm_dir = 'LM_Fisher'

LM_fns
# LM_fn_stem = 'fisher_utterances_main'
# LM_fns = {
#     '':{l:LM_fn_stem + '_' + str(l) + 'gram.mmap'
#         for l in (2,3,4,5)},
#     'rev':{l:LM_fn_stem + '_' + 'rev' + '_' + str(l) + 'gram.mmap'
#            for l in (2,3,4,5)}
# }


# fisher_lm_vocab_fn = 'fisher_vocabulary_main.txt'

{'Buckeye': {'preceding': {1: 'buckeye_contexts_preceding_1_filtered.txt',
   2: 'buckeye_contexts_preceding_2_filtered.txt',
   3: 'buckeye_contexts_preceding_3_filtered.txt',
   4: 'buckeye_contexts_preceding_4_filtered.txt'},
  'following': {1: 'buckeye_contexts_following_1_filtered.txt',
   2: 'buckeye_contexts_following_2_filtered.txt',
   3: 'buckeye_contexts_following_3_filtered.txt',
   4: 'buckeye_contexts_following_4_filtered.txt'},
  'bidirectional': 'buckeye_contexts_bidirectional_filtered.json'},
 'NXT_swbd': {'preceding': {1: 'nxt_swbd_contexts_preceding_1_filtered.txt',
   2: 'nxt_swbd_contexts_preceding_2_filtered.txt',
   3: 'nxt_swbd_contexts_preceding_3_filtered.txt',
   4: 'nxt_swbd_contexts_preceding_4_filtered.txt'},
  'following': {1: 'nxt_swbd_contexts_following_1_filtered.txt',
   2: 'nxt_swbd_contexts_following_2_filtered.txt',
   3: 'nxt_swbd_contexts_following_3_filtered.txt',
   4: 'nxt_swbd_contexts_following_4_filtered.txt'},
  'bidirectional': 'nxt_swbd_

('C_Buckeye', 'C_NXT_swbd')

{'Buckeye': ('buckeye_contexts_following_1_filtered.txt',
  'buckeye_contexts_following_2_filtered.txt',
  'buckeye_contexts_following_3_filtered.txt',
  'buckeye_contexts_following_4_filtered.txt',
  'buckeye_contexts_preceding_1_filtered.txt',
  'buckeye_contexts_preceding_2_filtered.txt',
  'buckeye_contexts_preceding_3_filtered.txt',
  'buckeye_contexts_preceding_4_filtered.txt',
  'buckeye_contexts_bidirectional_filtered.json'),
 'NXT_swbd': ('nxt_swbd_contexts_following_1_filtered.txt',
  'nxt_swbd_contexts_following_2_filtered.txt',
  'nxt_swbd_contexts_following_3_filtered.txt',
  'nxt_swbd_contexts_following_4_filtered.txt',
  'nxt_swbd_contexts_preceding_1_filtered.txt',
  'nxt_swbd_contexts_preceding_2_filtered.txt',
  'nxt_swbd_contexts_preceding_3_filtered.txt',
  'nxt_swbd_contexts_preceding_4_filtered.txt',
  'nxt_swbd_contexts_bidirectional_filtered.json')}

{'Buckeye': '../buckeye-lm', 'NXT_swbd': '../switchboard-lm'}

{'': {2: 'fisher_utterances_main_2gram.mmap',
  3: 'fisher_utterances_main_3gram.mmap',
  4: 'fisher_utterances_main_4gram.mmap',
  5: 'fisher_utterances_main_5gram.mmap'},
 'rev': {2: 'fisher_utterances_main_rev_2gram.mmap',
  3: 'fisher_utterances_main_rev_3gram.mmap',
  4: 'fisher_utterances_main_rev_4gram.mmap',
  5: 'fisher_utterances_main_rev_5gram.mmap'}}

In [49]:
my_vocab_fn = fisher_lm_vocab_fn
LD_bundles = []

for corpus_name in corpus_to_contexts:
    for context_fn in corpus_to_contexts[corpus_name]:
        if 'bidirectional' in context_fn:
            continue #not supported for now
        elif 'preceding' in context_fn:
            context_direction = 'preceding'
            lm_direction = ''
        else:
            context_direction = 'following'
            lm_direction = 'rev'
        
        context_size = context_fn[-14]
        order = int(context_size) + 1
        
        LD_id = str_join('_', ['LD','Fisher','vocab','in',
                               corpus_name, context_direction, 'contexts',
                               str(order) + 'gram', 'model'])
        
        new_bundle = {
            'corpus':corpus_name,
            'context_fn':context_fn,
            'context_fp':path.join('C_' + corpus_name, context_fn),
            'lm_fn':LM_fns[lm_direction][order],
            'lm_fp':path.join(fisher_lm_dir, LM_fns[lm_direction][order]),
            'LD_dir':LD_id,
            'o_fn_stem':LD_id,
            'o':path.join(LD_id, LD_id),
            'm':path.join(fisher_lm_dir, LM_fns[lm_direction][order]),
            'v':path.join(fisher_lm_dir, my_vocab_fn),
            'c':path.join('C_' + corpus_name, context_fn),
            'nb_fp':path.join(LD_id, 
                              'Producing ' + LD_id.replace('_', ' ')[3:] + ' contextual distributions.ipynb')
        }
        
        LD_bundles.append(new_bundle)
LD_bundles
LDs = LD_bundles

[{'corpus': 'Buckeye',
  'context_fn': 'buckeye_contexts_following_1_filtered.txt',
  'context_fp': 'C_Buckeye/buckeye_contexts_following_1_filtered.txt',
  'lm_fn': 'fisher_utterances_main_rev_2gram.mmap',
  'lm_fp': 'LM_Fisher/fisher_utterances_main_rev_2gram.mmap',
  'LD_dir': 'LD_Fisher_vocab_in_Buckeye_following_contexts_2gram_model',
  'o_fn_stem': 'LD_Fisher_vocab_in_Buckeye_following_contexts_2gram_model',
  'o': 'LD_Fisher_vocab_in_Buckeye_following_contexts_2gram_model/LD_Fisher_vocab_in_Buckeye_following_contexts_2gram_model',
  'm': 'LM_Fisher/fisher_utterances_main_rev_2gram.mmap',
  'v': 'LM_Fisher/fisher_vocabulary_main.txt',
  'c': 'C_Buckeye/buckeye_contexts_following_1_filtered.txt',
  'nb_fp': 'LD_Fisher_vocab_in_Buckeye_following_contexts_2gram_model/Producing Fisher vocab in Buckeye following contexts 2gram model contextual distributions.ipynb'},
 {'corpus': 'Buckeye',
  'context_fn': 'buckeye_contexts_following_2_filtered.txt',
  'context_fp': 'C_Buckeye/buckeye_c

In [50]:
# lmap(partial(omit, keys=('v', 'nb_fp', 'corpus', 'context_fp', 'lm_fp', 'o_fn_stem')),
#      LDs)

In [51]:
# contexts
# fisher_lm_fps

In [52]:
# LDs = [{'LD_dir':'LD_Fisher_vocab_in_Buckeye_contexts',
#         'o_fn_stem':'LD_fisher_vocab_in_buckeye_contexts',
#         'o':'LD_Fisher_vocab_in_Buckeye_contexts' + '/' + 'LD_fisher_vocab_in_buckeye_contexts',
#         'm':fisher_lm_fps['lm'],
#         'v':fisher_lm_fps['vocab'],
#         'c':buckeye_contexts,
#         'nb_fp':path.join('LD_Fisher_vocab_in_Buckeye_contexts', 
#                           'Producing ' + 'LD_Fisher_vocab_in_Buckeye_contexts'.replace('_', ' ')[3:] + ' contextual distributions.ipynb')},
# #        {'LD_dir':'LD_Fisher_vocab_in_swbd2003_contexts',
# #         'o_fn_stem':'LD_fisher_vocab_in_swbd2003_contexts',
# #         'o':'LD_Fisher_vocab_in_swbd2003_contexts' + '/' + 'LD_fisher_vocab_in_swbd2003_contexts',
# #         'm':fisher_lm_fps['lm'],
# #         'v':fisher_lm_fps['vocab'],
# #         'c':swbd2003_contexts,
# #         'nb_fp':path.join('LD_Fisher_vocab_in_swbd2003_contexts', 
# #                           'Producing ' + 'LD_Fisher_vocab_in_swbd2003_contexts'.replace('_', ' ')[3:] + ' contextual distributions.ipynb')},
#        {'LD_dir':'LD_Fisher_vocab_in_NXT_swbd_contexts',
#         'o_fn_stem':'LD_Fisher_vocab_in_nxt_swbd_contexts',
#         'o':'LD_Fisher_vocab_in_NXT_swbd_contexts' + '/' + 'LD_Fisher_vocab_in_nxt_swbd_contexts',
#         'm':FIXME,
#         'v':,
#         'c':,
#         'nb_fp':path.join('LD_Fisher_vocab_in_NXT_swbd_contexts',
#                           'Producing ' + 'LD_Fisher_vocab_in_NXT_swbd_contexts'.replace('_', ' ')[3:] + ' contextual distributions.ipynb')}
#       ]

In [53]:
if '2b' in permittedSteps:
    # used to take ~80m on wittgenstein
    
    # timing data
    # corpus = Buckeye
    #     preceding contexts:
    #         n = 2
    #             pitts/1.3m
    #         n = 3
    #             montague/7.5m
    #         n = 4
    #             pitts/12m
    #         n = 5
    #             sidious/16m
    #     following contexts:
    #         n = 2r
    #             x
    #         n = 3r
    #             x
    #         n = 4r
    #             x
    #         n = 5r
    #             wittgenstein/14m
    # corpus = NXT_swbd
    #     preceding contexts:
    #         n = 2
    #             sidious/2.66m
    #         n = 3
    #             sidious/13.5m
    #         n = 4
    #             x
    #         n = 5
    #             wittgenstein/34m
    #     following contexts:
    #         n = 2r
    #             montague/3m
    #         n = 3r
    #             montague/16m
    #         n = 4r
    #             wittgenstein/28.5m
    #         n = 5r
    #             wittgenstein/32m
    for ld in LDs:
#         output_dir = path.dirname(ld['LD_dir'])
        output_dir = ld['LD_dir']
        ensure_dir_exists(output_dir)
        
        progress_report(ld['nb_fp'],
                        dict(m = ld['m'],
                        v = ld['v'],
                        c = ld['c'],
                        o = ld['o']))
        
        if not overwrite and path.exists(ld['nb_fp']):
            print('{0} already exists. Skipping...'.format(ld['nb_fp']))
            endNote()
            continue

        nb = pm.execute_notebook(
        'Producing contextual distributions.ipynb',
    #     'Producing ' + ld['LD_dir'].replace('_', ' ')[3:] + ' contextual distributions.ipynb',
        ld['nb_fp'],
        parameters=dict(m = ld['m'],
                        v = ld['v'],
                        c = ld['c'],
                        o = ld['o'])
        )
        endNote()
        print('\n')

In [54]:
%pwd

'/mnt/cube/home/AD/emeinhar/wr'

# Step 3: Creating combinable models

**The basic problem:**

1. **Channel model + transcribed lexicon relation**: Even after the gating data and a transcribed lexicon relation are defined over the same inventory, 
 - the lexicon may contain triphones or diphones that are not in the stimuli triphones/diphones of a channel model.
 - channel distributions will contain triphones/diphones that cannot be found in the transcribed lexicon relation. (While the other steps here are strictly necessary, this is simply a practical step for making downstream computation faster.)
2. **Language model + transcribed lexicon relation**: The orthographic vocabulary of a transcribed lexicon relation may contain wordforms not in an n-gram model's vocabulary. (We *don't* want to use the out-of-vocabulary estimate for those wordforms.)
3.  **Contextual distributions + transcribed lexicon relation**: The contextual distributions from Step 3b above are defined over the *language model's* orthographic vocabulary, which will likely include wordforms that are not in the transcribed lexicon relation. We want to create modified forms of these distributions where we condition on the choice of an orthographic wordform that is in the transcribed lexicon relation.

Once we have
 - a version $l'$ of the transcribed lexicon relation $l$ trimmed with respect to both the triphones of the channel model $c$ and the (orthographic) vocabulary of a language model $m$
 - a version $d'$ of the contextual distributions over $m$'s vocabulary (with respect to some set of n-gram contexts) $d$ trimmed to only define distributions over $l'$
 - a version $c'$ of the channel model $c$ trimmed with respect to a transcribed lexicon relation $l'$
 - a probability distribution over segmental wordforms given an orthographic wordform
 
we will be able to combine everything together to calculate confusability of wordforms in corpus contexts.

## Step 3a: Filter transcription lexicons to only include words that can be modeled by a given channel distribution

In [55]:
#gather relevant LTRs and their associated CMs

In [56]:
aligned_LTRs = lmap(lambda ab: {'LTR_fp':ab['my_ol'],
                                'GD_fp':ab['my_og']},
                    alignment_arg_bundles)
lmap(lambda d: d['LTR_fp'],
     aligned_LTRs)

['LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_w_GD_AmE-diphones.tsv',
 'LTR_CMU_destressed_aligned_w_GD_AmE_destressed/LTR_CMU_destressed_aligned_w_GD_AmE-diphones.tsv',
 'LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_w_GD_AmE-diphones.tsv',
 'LTR_NXT_swbd_destressed_aligned_w_GD_AmE_destressed/LTR_NXT_swbd_destressed_aligned_w_GD_AmE-diphones.tsv']

In [57]:
# CM_dirs = list(map(lambda cab: {'CM_fp':path.join(cab['cm_dir'],'pY1X0X1X2.json'),
#                                 'GD_fp':cab['gating_data_fp']},
#                    filter(lambda cab: cab['c'] != 0, cm_arg_bundles)))
# CM_dirs
# CM_dirs[5]
# # listdir(CM_dirs[0][])

aligned_CMs = list(map(lambda cab: {'CM_fp':path.join(cab['cm_dir'],'pY1X0X1X2.json'),
                                    'GD_fp':cab['gating_data_fp']},
                       filter(lambda cab: cab['c'] != 0 and 'aligned' in cab['gating_data_fp'], 
                              cm_arg_bundles)))
aligned_CMs

[{'CM_fp': 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.01/pY1X0X1X2.json',
  'GD_fp': 'GD_AmE_destressed_aligned_w_LTR_newdic_destressed/GD_AmE-diphones_aligned_w_LTR_newdic_destressed.tsv'},
 {'CM_fp': 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.05/pY1X0X1X2.json',
  'GD_fp': 'GD_AmE_destressed_aligned_w_LTR_newdic_destressed/GD_AmE-diphones_aligned_w_LTR_newdic_destressed.tsv'},
 {'CM_fp': 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.1/pY1X0X1X2.json',
  'GD_fp': 'GD_AmE_destressed_aligned_w_LTR_newdic_destressed/GD_AmE-diphones_aligned_w_LTR_newdic_destressed.tsv'},
 {'CM_fp': 'CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.01/pY1X0X1X2.json',
  'GD_fp': 'GD_AmE_destressed_aligned_w_LTR_CMU_destressed/GD_AmE-diphones_aligned_w_LTR_CMU_destressed.tsv'},
 {'CM_fp': 'CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.05/pY1X0X1X2.json',
  'GD_fp': 'GD_AmE_destressed_aligned_w_LTR_CMU_destressed/GD_AmE-diph

In [58]:
def get_aligned_CMs(ltr_bundle):
    matches = [cm_bundle for cm_bundle in aligned_CMs if cm_bundle['GD_fp'] == ltr_bundle['GD_fp']]
    return list(map(lambda d: d['CM_fp'],
                    matches))

In [59]:
aligned_LTRs[0]['LTR_fp']

#NB: all of these will have the same set of stimuli triphones
#    ...which is all we care about here
get_aligned_CMs(aligned_LTRs[0]) 

'LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_w_GD_AmE-diphones.tsv'

['CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.01/pY1X0X1X2.json',
 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.05/pY1X0X1X2.json',
 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.1/pY1X0X1X2.json']

In [60]:
aligned_LTRs_and_CM = [{'LTR_fp':ltr['LTR_fp'],
                        'matching_CMs':get_aligned_CMs(ltr)}
                       for ltr in aligned_LTRs]
aligned_LTRs_and_CM

[{'LTR_fp': 'LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_w_GD_AmE-diphones.tsv',
  'matching_CMs': ['CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.01/pY1X0X1X2.json',
   'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.05/pY1X0X1X2.json',
   'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.1/pY1X0X1X2.json']},
 {'LTR_fp': 'LTR_CMU_destressed_aligned_w_GD_AmE_destressed/LTR_CMU_destressed_aligned_w_GD_AmE-diphones.tsv',
  'matching_CMs': ['CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.01/pY1X0X1X2.json',
   'CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.05/pY1X0X1X2.json',
   'CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.1/pY1X0X1X2.json']},
 {'LTR_fp': 'LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_w_GD_AmE-diphones.tsv',
  'matching_CMs': ['CM_AmE_destressed_aligned_w_LTR_Buckeye_pseudocount0.01/pY1X0X1X2.json',
   'CM_AmE_destressed_aligned_w_LTR

In [61]:
for each in aligned_LTRs_and_CM:
    each['c'] = each['matching_CMs'][0]
    each['l'] = each['LTR_fp']
    o_dir = path.dirname(each['LTR_fp'])
    o_fn = path.basename(each['LTR_fp']).split('w_')[0] + 'CM_filtered' + '.tsv'
    each['o'] = path.join(o_dir, o_fn)
    
    nb_fn = 'Filter ' + o_fn.split('_aligned')[0] + ' against channel model.ipynb'
    each['nb_fp'] = path.join(o_dir, nb_fn)
    
    print('c = {0}\nl = {1}\no = {2}\nnb = {3}'.format(each['c'], each['l'], each['o'], each['nb_fp']))
    print(' ')

c = CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.01/pY1X0X1X2.json
l = LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_w_GD_AmE-diphones.tsv
o = LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_CM_filtered.tsv
nb = LTR_newdic_destressed_aligned_w_GD_AmE_destressed/Filter LTR_newdic_destressed against channel model.ipynb
 
c = CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.01/pY1X0X1X2.json
l = LTR_CMU_destressed_aligned_w_GD_AmE_destressed/LTR_CMU_destressed_aligned_w_GD_AmE-diphones.tsv
o = LTR_CMU_destressed_aligned_w_GD_AmE_destressed/LTR_CMU_destressed_aligned_CM_filtered.tsv
nb = LTR_CMU_destressed_aligned_w_GD_AmE_destressed/Filter LTR_CMU_destressed against channel model.ipynb
 
c = CM_AmE_destressed_aligned_w_LTR_Buckeye_pseudocount0.01/pY1X0X1X2.json
l = LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_w_GD_AmE-diphones.tsv
o = LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Bu

In [62]:
if '3a' in permittedSteps:
    # takes about 30s on wittgenstein
    for each in aligned_LTRs_and_CM:
        output_dir = path.dirname(each['o'])
        ensure_dir_exists(output_dir)

        progress_report(each['nb_fp'],
                        dict(l = each['l'],
                             c = each['c'],
                             o = each['o']))

        nb = pm.execute_notebook(
        'Filter transcription lexicon by channel model.ipynb',
        each['nb_fp'],
        parameters=dict(l = each['l'],
                        c = each['c'],
                        o = each['o'])
        )
        endNote()
        print('\n')

## Step 3b: Filter transcription lexicons to only include words that are in a language model's vocabulary

**Dependencies**
 - **Step 3a**: `LTR_..._aligned_CM_filtered....tsv`

In [63]:
# fisher_lm_fps
# fisher_lm_vocab_fp

In [64]:
LTR_fps = list(map(lambda pair: pair['LTR_fp'],
                   aligned_LTRs))
LTR_fps

LTR_CM_filtered = list(map(lambda d: d['o'],
                           aligned_LTRs_and_CM))
LTR_CM_filtered

['LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_w_GD_AmE-diphones.tsv',
 'LTR_CMU_destressed_aligned_w_GD_AmE_destressed/LTR_CMU_destressed_aligned_w_GD_AmE-diphones.tsv',
 'LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_w_GD_AmE-diphones.tsv',
 'LTR_NXT_swbd_destressed_aligned_w_GD_AmE_destressed/LTR_NXT_swbd_destressed_aligned_w_GD_AmE-diphones.tsv']

['LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_CM_filtered.tsv',
 'LTR_CMU_destressed_aligned_w_GD_AmE_destressed/LTR_CMU_destressed_aligned_CM_filtered.tsv',
 'LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_CM_filtered.tsv',
 'LTR_NXT_swbd_destressed_aligned_w_GD_AmE_destressed/LTR_NXT_swbd_destressed_aligned_CM_filtered.tsv']

How many lexicon entries have unmodelable triphones? (We'll next check how many such lexicon entries aren't in the language model vocabulary.)

In [65]:
!wc -l LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_w_GD_AmE-diphones.tsv
!wc -l LTR_CMU_destressed_aligned_w_GD_AmE_destressed/LTR_CMU_destressed_aligned_w_GD_AmE-diphones.tsv
!wc -l LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_w_GD_AmE-diphones.tsv
!wc -l LTR_NXT_swbd_destressed_aligned_w_GD_AmE_destressed/LTR_NXT_swbd_destressed_aligned_w_GD_AmE-diphones.tsv

19529 LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_w_GD_AmE-diphones.tsv
133855 LTR_CMU_destressed_aligned_w_GD_AmE_destressed/LTR_CMU_destressed_aligned_w_GD_AmE-diphones.tsv
7999 LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_w_GD_AmE-diphones.tsv
15814 LTR_NXT_swbd_destressed_aligned_w_GD_AmE_destressed/LTR_NXT_swbd_destressed_aligned_w_GD_AmE-diphones.tsv


In [66]:
!wc -l LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_CM_filtered.tsv
!wc -l LTR_CMU_destressed_aligned_w_GD_AmE_destressed/LTR_CMU_destressed_aligned_CM_filtered.tsv
!wc -l LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_CM_filtered.tsv
!wc -l LTR_NXT_swbd_destressed_aligned_w_GD_AmE_destressed/LTR_NXT_swbd_destressed_aligned_CM_filtered.tsv

17079 LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_CM_filtered.tsv
127799 LTR_CMU_destressed_aligned_w_GD_AmE_destressed/LTR_CMU_destressed_aligned_CM_filtered.tsv
7011 LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_CM_filtered.tsv
15318 LTR_NXT_swbd_destressed_aligned_w_GD_AmE_destressed/LTR_NXT_swbd_destressed_aligned_CM_filtered.tsv


Word type loss for newdic:
 - $19529 - 17079 = 2450$ word types lost due to unmodelable triphones

Word type loss for CMU_destressed:
 - $133855 - 127799 = 6056$ word types lost due to unmodelable triphones

Word type loss for Buckeye:
 - $7999 - 7011 = 988$ word types lost due to unmodelable triphones
 
Word type loss for NXT_swbd_destressed:
 - $15834 - 15338 = 496$ word types lost due to unmodelable triphones

In [67]:
LTR_CM_filtered

['LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_CM_filtered.tsv',
 'LTR_CMU_destressed_aligned_w_GD_AmE_destressed/LTR_CMU_destressed_aligned_CM_filtered.tsv',
 'LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_CM_filtered.tsv',
 'LTR_NXT_swbd_destressed_aligned_w_GD_AmE_destressed/LTR_NXT_swbd_destressed_aligned_CM_filtered.tsv']

In [68]:
fisher_lm_vocab_fp = path.join(fisher_lm_dir, fisher_lm_vocab_fn)

In [69]:
LTR_LM_filter_bundles = []
for each_LTR_fp in LTR_CM_filtered:
    bundle = dict()
    LTR_descr = path.basename(each_LTR_fp)[:-4]
    LM_V_descr = path.basename(fisher_lm_vocab_fp)[:-4]
    bundle['l'] = each_LTR_fp
    bundle['v'] = fisher_lm_vocab_fp
    bundle['o'] = each_LTR_fp[:-4] + '_LM_filtered' + '.tsv'
    bundle['nb_fp'] = path.join(path.dirname(each_LTR_fp),
                                f'Filter {LTR_descr} against {LM_V_descr}' + '.ipynb')
    bundle
    LTR_LM_filter_bundles.append(bundle)
    print('')

{'l': 'LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_CM_filtered.tsv',
 'v': 'LM_Fisher/fisher_vocabulary_main.txt',
 'o': 'LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered.tsv',
 'nb_fp': 'LTR_newdic_destressed_aligned_w_GD_AmE_destressed/Filter LTR_newdic_destressed_aligned_CM_filtered against fisher_vocabulary_main.ipynb'}

{'l': 'LTR_CMU_destressed_aligned_w_GD_AmE_destressed/LTR_CMU_destressed_aligned_CM_filtered.tsv',
 'v': 'LM_Fisher/fisher_vocabulary_main.txt',
 'o': 'LTR_CMU_destressed_aligned_w_GD_AmE_destressed/LTR_CMU_destressed_aligned_CM_filtered_LM_filtered.tsv',
 'nb_fp': 'LTR_CMU_destressed_aligned_w_GD_AmE_destressed/Filter LTR_CMU_destressed_aligned_CM_filtered against fisher_vocabulary_main.ipynb'}

{'l': 'LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_CM_filtered.tsv',
 'v': 'LM_Fisher/fisher_vocabulary_main.txt',
 'o': 'LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_CM_filtered_LM_filtered.tsv',
 'nb_fp': 'LTR_Buckeye_aligned_w_GD_AmE_destressed/Filter LTR_Buckeye_aligned_CM_filtered against fisher_vocabulary_main.ipynb'}

{'l': 'LTR_NXT_swbd_destressed_aligned_w_GD_AmE_destressed/LTR_NXT_swbd_destressed_aligned_CM_filtered.tsv',
 'v': 'LM_Fisher/fisher_vocabulary_main.txt',
 'o': 'LTR_NXT_swbd_destressed_aligned_w_GD_AmE_destressed/LTR_NXT_swbd_destressed_aligned_CM_filtered_LM_filtered.tsv',
 'nb_fp': 'LTR_NXT_swbd_destressed_aligned_w_GD_AmE_destressed/Filter LTR_NXT_swbd_destressed_aligned_CM_filtered against fisher_vocabulary_main.ipynb'}

In [70]:
if '3b' in permittedSteps:
    # takes about ~1m on wittgenstein
    for each in LTR_LM_filter_bundles:
        output_dir = path.dirname(each['o'])
        ensure_dir_exists(output_dir)

        progress_report(each['nb_fp'],
                        dict(l = each['l'],
                             v = each['v'],
                             o = each['o']))

        nb = pm.execute_notebook(
        'Filter transcription lexicon by language model vocabulary.ipynb',
        each['nb_fp'],
        parameters=dict(l = each['l'],
                        v = each['v'],
                        o = each['o'])
        )
        endNote()
        print('\n')

**A bit less than half** of the triphone channel model-modelable `newdic` lexicon **isn't** in the LM vocab:

In [71]:
!wc -l LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_CM_filtered.tsv
!wc -l LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered.tsv

17079 LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_CM_filtered.tsv
9412 LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered.tsv


$17079 - 9412 = 7667$ lost

$7667 / 17079 ≈ 0.45$ proportionally

Word type loss for newdic:
 - $19529 - 17079 = 2450$ word types lost due to unmodelable triphones
 - $17079 - 9412 = 7667$ further word types lost due orthographic wordforms not being in the language model vocabulary
 - $\frac{2450+7667}{19529} ≈ 51.8\%$ of word types lost.

**About three quarters** of the triphone channel model-modelable `CMU_destressed` lexicon **isn't** in the LM vocab:

In [72]:
!wc -l LTR_CMU_destressed_aligned_w_GD_AmE_destressed/LTR_CMU_destressed_aligned_CM_filtered.tsv
!wc -l LTR_CMU_destressed_aligned_w_GD_AmE_destressed/LTR_CMU_destressed_aligned_CM_filtered_LM_filtered.tsv

127799 LTR_CMU_destressed_aligned_w_GD_AmE_destressed/LTR_CMU_destressed_aligned_CM_filtered.tsv
33125 LTR_CMU_destressed_aligned_w_GD_AmE_destressed/LTR_CMU_destressed_aligned_CM_filtered_LM_filtered.tsv


$127799 - 33125 = 94674$ lost

$94674 / 127799 ≈ 0.74$ proportionally

Word type loss for CMU_destressed:
 - $133855 - 127799 = 6056$ word types lost due to unmodelable triphones
 - $127799 - 33125 = 94674$ further word types lost due orthographic wordforms not being in the language model vocabulary
 - $\frac{6056+94674}{133855} ≈ 75.25\%$ of word types lost.

**A bit less than 20%** of the triphone channel model-modelable `Buckeye` lexicon **isn't** in the LM vocab:

In [73]:
!wc -l LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_CM_filtered.tsv
!wc -l LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_CM_filtered_LM_filtered.tsv

7011 LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_CM_filtered.tsv
6576 LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_CM_filtered_LM_filtered.tsv


$7011 - 6576 = 435$ lost

$435 / 7011 ≈ .06$ proportionally

Word type loss for Buckeye:
 - $7999 - 7011 = 988$ word types lost due to unmodelable triphones
 - $7011 - 6576 = 435$ further word types lost due orthographic wordforms not being in the language model vocabulary
 - $\frac{435+988}{7999} ≈ 17.8\%$ of word types lost.

**About 15%** of the triphone channel model-modelable `NXT_swbd_destressed` lexicon **isn't** in the LM vocab:

In [74]:
!wc -l LTR_NXT_swbd_destressed_aligned_w_GD_AmE_destressed/LTR_NXT_swbd_destressed_aligned_CM_filtered.tsv
!wc -l LTR_NXT_swbd_destressed_aligned_w_GD_AmE_destressed/LTR_NXT_swbd_destressed_aligned_CM_filtered_LM_filtered.tsv

15318 LTR_NXT_swbd_destressed_aligned_w_GD_AmE_destressed/LTR_NXT_swbd_destressed_aligned_CM_filtered.tsv
13246 LTR_NXT_swbd_destressed_aligned_w_GD_AmE_destressed/LTR_NXT_swbd_destressed_aligned_CM_filtered_LM_filtered.tsv


$15338 - 13266 = 2072$ lost

$2072 / 15338 ≈ 0.14$ proportionally

Word type loss for NXT_swbd_destressed:
 - $15834 - 15338 = 496$ word types lost due to unmodelable triphones
 - $15338 - 13266 = 2072$ further word types lost due orthographic wordforms not being in the language model vocabulary
 - $\frac{496+2072}{15834} ≈ 16.2\%$ of word types lost.

Collecting the filtered transcribed lexicon relations...

In [75]:
LTR_CM_filtered_LM_filtered = list(map(lambda bundle: bundle['o'],
                                       LTR_LM_filter_bundles))
LTR_CM_filtered_LM_filtered

['LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered.tsv',
 'LTR_CMU_destressed_aligned_w_GD_AmE_destressed/LTR_CMU_destressed_aligned_CM_filtered_LM_filtered.tsv',
 'LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_CM_filtered_LM_filtered.tsv',
 'LTR_NXT_swbd_destressed_aligned_w_GD_AmE_destressed/LTR_NXT_swbd_destressed_aligned_CM_filtered_LM_filtered.tsv']

## Step 3c: Filter the conditioning events of channel distributions to only include $k$-factors contained in elements of a transcription lexicon's segmental wordforms

**Dependencies**
 - **Step 3b**: `LTR_..._aligned_CM_filtered_LM_filtered.tsv`

In [76]:
#what channel models might you want to use with what lexicons?
# there are 3x3 triphone CMs that are aligned with one of 3 LTRs and have one of 3 relevant pseudocount levels
# For each of the 3 LTRs aligned with the gating data, there's exactly 1 `LTR...CM_filtered_LM_filtered.tsv` file
# ∴ there are 3x3 triphone channel models to trim

# Also, for each triphone CM, there are preview and postview distributions to trim

In [77]:
for each in aligned_LTRs_and_CM:
    LTR_dir = path.dirname(each['LTR_fp'])
    LTR_trimmed_fn = path.basename(each['LTR_fp']).split('_w_')[0] + '_CM_filtered_LM_filtered.tsv'
    each['LTR_trimmed_fp'] = path.join(LTR_dir, LTR_trimmed_fn)
    each['matching_trimmed_CMs'] = [path.join(path.dirname(fp),
                                              LTR_trimmed_fn[:-4] + '_' + path.basename(fp))
                                    for fp in each['matching_CMs']]
    
#     each['LTR_fp']
    each['LTR_trimmed_fp']
    each['matching_CMs']
    each['matching_trimmed_CMs']
#     each['trimmed LTR_fp']
    print('')

'LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered.tsv'

['CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.01/pY1X0X1X2.json',
 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.05/pY1X0X1X2.json',
 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.1/pY1X0X1X2.json']

['CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.01/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered_pY1X0X1X2.json',
 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.05/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered_pY1X0X1X2.json',
 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.1/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered_pY1X0X1X2.json']

'LTR_CMU_destressed_aligned_w_GD_AmE_destressed/LTR_CMU_destressed_aligned_CM_filtered_LM_filtered.tsv'

['CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.01/pY1X0X1X2.json',
 'CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.05/pY1X0X1X2.json',
 'CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.1/pY1X0X1X2.json']

...

['CM_AmE_destressed_aligned_w_LTR_Buckeye_pseudocount0.01/pY1X0X1X2.json',
 'CM_AmE_destressed_aligned_w_LTR_Buckeye_pseudocount0.05/pY1X0X1X2.json',
 'CM_AmE_destressed_aligned_w_LTR_Buckeye_pseudocount0.1/pY1X0X1X2.json']

['CM_AmE_destressed_aligned_w_LTR_Buckeye_pseudocount0.01/LTR_Buckeye_aligned_CM_filtered_LM_filtered_pY1X0X1X2.json',
 'CM_AmE_destressed_aligned_w_LTR_Buckeye_pseudocount0.05/LTR_Buckeye_aligned_CM_filtered_LM_filtered_pY1X0X1X2.json',
 'CM_AmE_destressed_aligned_w_LTR_Buckeye_pseudocount0.1/LTR_Buckeye_aligned_CM_filtered_LM_filtered_pY1X0X1X2.json']

'LTR_NXT_swbd_destressed_aligned_w_GD_AmE_destressed/LTR_NXT_swbd_destressed_aligned_CM_filtered_LM_filtered.tsv'

['CM_AmE_destressed_aligned_w_LTR_NXT_swbd_destressed_pseudocount0.01/pY1X0X1X2.json',
 'CM_AmE_destressed_aligned_w_LTR_NXT_swbd_destressed_pseudocount0.05/pY1X0X1X2.json',
 'CM_AmE_destressed_aligned_w_LTR_NXT_swbd_destressed_pseudocount0.1/pY1X0X1X2.json']

['CM_AmE_destressed_aligned_w_LTR_NXT_swbd_destressed_pseudocount0.01/LTR_NXT_swbd_destressed_aligned_CM_filtered_LM_filtered_pY1X0X1X2.json',
 'CM_AmE_destressed_aligned_w_LTR_NXT_swbd_destressed_pseudocount0.05/LTR_NXT_swbd_destressed_aligned_CM_filtered_LM_filtered_pY1X0X1X2.json',
 'CM_AmE_destressed_aligned_w_LTR_NXT_swbd_destressed_pseudocount0.1/LTR_NXT_swbd_destressed_aligned_CM_filtered_LM_filtered_pY1X0X1X2.json']

In [78]:
trimmed_LTR_CM_triples = []
for each in aligned_LTRs_and_CM:
    assert len(each['matching_CMs']) == len(each['matching_trimmed_CMs'])
    
    for i in range(len(each['matching_CMs'])):
        args = dict()
        args['l'] = each['LTR_trimmed_fp']
        args['c'] = each['matching_CMs'][i]
        args['o'] = each['matching_trimmed_CMs'][i]
        args['nb_fp'] = path.join(path.dirname(args['c']),
                                  f"Filter {path.dirname(args['c'])} against {path.basename(args['l'])[:-4]}.ipynb")
        args
        trimmed_LTR_CM_triples.append(args)
        print('')


{'l': 'LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered.tsv',
 'c': 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.01/pY1X0X1X2.json',
 'o': 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.01/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered_pY1X0X1X2.json',
 'nb_fp': 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.01/Filter CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.01 against LTR_newdic_destressed_aligned_CM_filtered_LM_filtered.ipynb'}

{'l': 'LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered.tsv',
 'c': 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.05/pY1X0X1X2.json',
 'o': 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.05/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered_pY1X0X1X2.json',
 'nb_fp': 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.05/Filter CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.05 against LTR_newdic_destressed_aligned_CM_filtered_LM_filtered.ipynb'}

{'l': 'LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered.tsv',
 'c': 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.1/pY1X0X1X2.json',
 'o': 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.1/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered_pY1X0X1X2.json',
 'nb_fp': 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.1/Filter CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.1 against LTR_newdic_destressed_aligned_CM_filtered_LM_filtered.ipynb'}

{'l': 'LTR_CMU_destressed_aligned_w_GD_AmE_destressed/LTR_CMU_destressed_aligned_CM_filtered_LM_filtered.tsv',
 'c': 'CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.01/pY1X0X1X2.json',
 'o': 'CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.01/LTR_CMU_destressed_aligned_CM_filtered_LM_filtered_pY1X0X1X2.json',
 'nb_fp': 'CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.01/Filter CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.01 against LTR_CMU_destressed_aligned_CM_filtered_LM_filtered.ipynb'}

{'l': 'LTR_CMU_destressed_aligned_w_GD_AmE_destressed/LTR_CMU_destressed_aligned_CM_filtered_LM_filtered.tsv',
 'c': 'CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.05/pY1X0X1X2.json',
 'o': 'CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.05/LTR_CMU_destressed_aligned_CM_filtered_LM_filtered_pY1X0X1X2.json',
 'nb_fp': 'CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.05/Filter CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.05 against LTR_CMU_destressed_aligned_CM_filtered_LM_filtered.ipynb'}

...

{'l': 'LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_CM_filtered_LM_filtered.tsv',
 'c': 'CM_AmE_destressed_aligned_w_LTR_Buckeye_pseudocount0.05/pY1X0X1X2.json',
 'o': 'CM_AmE_destressed_aligned_w_LTR_Buckeye_pseudocount0.05/LTR_Buckeye_aligned_CM_filtered_LM_filtered_pY1X0X1X2.json',
 'nb_fp': 'CM_AmE_destressed_aligned_w_LTR_Buckeye_pseudocount0.05/Filter CM_AmE_destressed_aligned_w_LTR_Buckeye_pseudocount0.05 against LTR_Buckeye_aligned_CM_filtered_LM_filtered.ipynb'}

{'l': 'LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_CM_filtered_LM_filtered.tsv',
 'c': 'CM_AmE_destressed_aligned_w_LTR_Buckeye_pseudocount0.1/pY1X0X1X2.json',
 'o': 'CM_AmE_destressed_aligned_w_LTR_Buckeye_pseudocount0.1/LTR_Buckeye_aligned_CM_filtered_LM_filtered_pY1X0X1X2.json',
 'nb_fp': 'CM_AmE_destressed_aligned_w_LTR_Buckeye_pseudocount0.1/Filter CM_AmE_destressed_aligned_w_LTR_Buckeye_pseudocount0.1 against LTR_Buckeye_aligned_CM_filtered_LM_filtered.ipynb'}

{'l': 'LTR_NXT_swbd_destressed_aligned_w_GD_AmE_destressed/LTR_NXT_swbd_destressed_aligned_CM_filtered_LM_filtered.tsv',
 'c': 'CM_AmE_destressed_aligned_w_LTR_NXT_swbd_destressed_pseudocount0.01/pY1X0X1X2.json',
 'o': 'CM_AmE_destressed_aligned_w_LTR_NXT_swbd_destressed_pseudocount0.01/LTR_NXT_swbd_destressed_aligned_CM_filtered_LM_filtered_pY1X0X1X2.json',
 'nb_fp': 'CM_AmE_destressed_aligned_w_LTR_NXT_swbd_destressed_pseudocount0.01/Filter CM_AmE_destressed_aligned_w_LTR_NXT_swbd_destressed_pseudocount0.01 against LTR_NXT_swbd_destressed_aligned_CM_filtered_LM_filtered.ipynb'}

{'l': 'LTR_NXT_swbd_destressed_aligned_w_GD_AmE_destressed/LTR_NXT_swbd_destressed_aligned_CM_filtered_LM_filtered.tsv',
 'c': 'CM_AmE_destressed_aligned_w_LTR_NXT_swbd_destressed_pseudocount0.05/pY1X0X1X2.json',
 'o': 'CM_AmE_destressed_aligned_w_LTR_NXT_swbd_destressed_pseudocount0.05/LTR_NXT_swbd_destressed_aligned_CM_filtered_LM_filtered_pY1X0X1X2.json',
 'nb_fp': 'CM_AmE_destressed_aligned_w_LTR_NXT_swbd_destressed_pseudocount0.05/Filter CM_AmE_destressed_aligned_w_LTR_NXT_swbd_destressed_pseudocount0.05 against LTR_NXT_swbd_destressed_aligned_CM_filtered_LM_filtered.ipynb'}

{'l': 'LTR_NXT_swbd_destressed_aligned_w_GD_AmE_destressed/LTR_NXT_swbd_destressed_aligned_CM_filtered_LM_filtered.tsv',
 'c': 'CM_AmE_destressed_aligned_w_LTR_NXT_swbd_destressed_pseudocount0.1/pY1X0X1X2.json',
 'o': 'CM_AmE_destressed_aligned_w_LTR_NXT_swbd_destressed_pseudocount0.1/LTR_NXT_swbd_destressed_aligned_CM_filtered_LM_filtered_pY1X0X1X2.json',
 'nb_fp': 'CM_AmE_destressed_aligned_w_LTR_NXT_swbd_destressed_pseudocount0.1/Filter CM_AmE_destressed_aligned_w_LTR_NXT_swbd_destressed_pseudocount0.1 against LTR_NXT_swbd_destressed_aligned_CM_filtered_LM_filtered.ipynb'}

In [79]:
if '3c' in permittedSteps:
    # takes about 120s on wittgenstein
    for each in trimmed_LTR_CM_triples:
        output_dir = path.dirname(each['o'])
        ensure_dir_exists(output_dir)
    #     if not path.exists(output_dir):
    #         print(f"Creating output path '{output_dir}'")
    #         makedirs(output_dir)

        progress_report(each['nb_fp'],
                        dict(l = each['l'],
                             c = each['c'],
                             o = each['o']))

        nb = pm.execute_notebook(
        'Filter channel model by transcription lexicon.ipynb',
        each['nb_fp'],
        parameters=dict(l = each['l'],
                        c = each['c'],
                        o = each['o'])
        )
        endNote()
        print('\n')

## Step 3d: For each (filtered) transcribed lexicon relation, define the relevant contextual lexicon distributions over orthographic wordforms

**Dependencies**
 - **Step 2b**: `...pV_C`
 - **Step 3b**: `LTR_..._aligned_CM_filtered_LM_filtered.tsv`

In [80]:
LTR_CM_filtered_LM_filtered

['LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered.tsv',
 'LTR_CMU_destressed_aligned_w_GD_AmE_destressed/LTR_CMU_destressed_aligned_CM_filtered_LM_filtered.tsv',
 'LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_CM_filtered_LM_filtered.tsv',
 'LTR_NXT_swbd_destressed_aligned_w_GD_AmE_destressed/LTR_NXT_swbd_destressed_aligned_CM_filtered_LM_filtered.tsv']

In [81]:
LDs

[{'corpus': 'Buckeye',
  'context_fn': 'buckeye_contexts_following_1_filtered.txt',
  'context_fp': 'C_Buckeye/buckeye_contexts_following_1_filtered.txt',
  'lm_fn': 'fisher_utterances_main_rev_2gram.mmap',
  'lm_fp': 'LM_Fisher/fisher_utterances_main_rev_2gram.mmap',
  'LD_dir': 'LD_Fisher_vocab_in_Buckeye_following_contexts_2gram_model',
  'o_fn_stem': 'LD_Fisher_vocab_in_Buckeye_following_contexts_2gram_model',
  'o': 'LD_Fisher_vocab_in_Buckeye_following_contexts_2gram_model/LD_Fisher_vocab_in_Buckeye_following_contexts_2gram_model',
  'm': 'LM_Fisher/fisher_utterances_main_rev_2gram.mmap',
  'v': 'LM_Fisher/fisher_vocabulary_main.txt',
  'c': 'C_Buckeye/buckeye_contexts_following_1_filtered.txt',
  'nb_fp': 'LD_Fisher_vocab_in_Buckeye_following_contexts_2gram_model/Producing Fisher vocab in Buckeye following contexts 2gram model contextual distributions.ipynb'},
 {'corpus': 'Buckeye',
  'context_fn': 'buckeye_contexts_following_2_filtered.txt',
  'context_fp': 'C_Buckeye/buckeye_c

In [82]:
lmap(partial(omit, keys=('corpus','context_fn', 'lm_fn')), LDs)

[{'context_fp': 'C_Buckeye/buckeye_contexts_following_1_filtered.txt',
  'lm_fp': 'LM_Fisher/fisher_utterances_main_rev_2gram.mmap',
  'LD_dir': 'LD_Fisher_vocab_in_Buckeye_following_contexts_2gram_model',
  'o_fn_stem': 'LD_Fisher_vocab_in_Buckeye_following_contexts_2gram_model',
  'o': 'LD_Fisher_vocab_in_Buckeye_following_contexts_2gram_model/LD_Fisher_vocab_in_Buckeye_following_contexts_2gram_model',
  'm': 'LM_Fisher/fisher_utterances_main_rev_2gram.mmap',
  'v': 'LM_Fisher/fisher_vocabulary_main.txt',
  'c': 'C_Buckeye/buckeye_contexts_following_1_filtered.txt',
  'nb_fp': 'LD_Fisher_vocab_in_Buckeye_following_contexts_2gram_model/Producing Fisher vocab in Buckeye following contexts 2gram model contextual distributions.ipynb'},
 {'context_fp': 'C_Buckeye/buckeye_contexts_following_2_filtered.txt',
  'lm_fp': 'LM_Fisher/fisher_utterances_main_rev_3gram.mmap',
  'LD_dir': 'LD_Fisher_vocab_in_Buckeye_following_contexts_3gram_model',
  'o_fn_stem': 'LD_Fisher_vocab_in_Buckeye_followi

In [83]:
LD_projection_args = []
for ld in LDs:
    if ld['corpus'] == 'Buckeye':
        l = 'LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_CM_filtered_LM_filtered.tsv'
    elif ld['corpus'] == 'NXT_swbd':
        l = 'LTR_NXT_swbd_destressed_aligned_w_GD_AmE_destressed/LTR_NXT_swbd_destressed_aligned_CM_filtered_LM_filtered.tsv'
    else:
        raise Exception('Not currently supported...')
    
    lm_dir = path.dirname(ld['o'])
    lm_fn = path.basename(ld['o'])
    lm_stem = path.splitext(lm_fn)[0]
    
    l_fn = path.basename(l)
    l_stem = path.splitext(l_fn)[0]
    
    o = path.join(lm_dir, lm_dir + '_projected_' + l_stem) + '.pV_C'
    
    projection_ab = {
        'd':ld['o'] + '.pV_C',
        'v':fisher_lm_vocab_fp,
        'c':ld['c'],
        'l':l,
        'o':o,
        'f':'True',
        'nb_fp':path.join(lm_dir, 
                          'Filter ' + lm_stem + ' against ' + l_stem + '.ipynb')
    }
    LD_projection_args.append(projection_ab)

LD_projection_args

[{'d': 'LD_Fisher_vocab_in_Buckeye_following_contexts_2gram_model/LD_Fisher_vocab_in_Buckeye_following_contexts_2gram_model.pV_C',
  'v': 'LM_Fisher/fisher_vocabulary_main.txt',
  'c': 'C_Buckeye/buckeye_contexts_following_1_filtered.txt',
  'l': 'LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_CM_filtered_LM_filtered.tsv',
  'o': 'LD_Fisher_vocab_in_Buckeye_following_contexts_2gram_model/LD_Fisher_vocab_in_Buckeye_following_contexts_2gram_model_projected_LTR_Buckeye_aligned_CM_filtered_LM_filtered.pV_C',
  'f': 'True',
  'nb_fp': 'LD_Fisher_vocab_in_Buckeye_following_contexts_2gram_model/Filter LD_Fisher_vocab_in_Buckeye_following_contexts_2gram_model against LTR_Buckeye_aligned_CM_filtered_LM_filtered.ipynb'},
 {'d': 'LD_Fisher_vocab_in_Buckeye_following_contexts_3gram_model/LD_Fisher_vocab_in_Buckeye_following_contexts_3gram_model.pV_C',
  'v': 'LM_Fisher/fisher_vocabulary_main.txt',
  'c': 'C_Buckeye/buckeye_contexts_following_2_filtered.txt',
  'l': 'LTR_Buckeye_aligne

In [84]:
fisher_lm_vocab_fp

'LM_Fisher/fisher_vocabulary_main.txt'

In [85]:
# buckeye_contexts
# swbd2003_contexts

In [86]:
# LD_projection_args = [
#     {'d':'LD_Fisher_vocab_in_Buckeye_contexts/LD_fisher_vocab_in_buckeye_contexts.pV_C',
#      'v':fisher_lm_vocab_fp,
#      'c':buckeye_contexts,
#      'l':'LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_CM_filtered_LM_filtered.tsv',
#      'o':'LD_Fisher_vocab_in_Buckeye_contexts/LD_fisher_vocab_in_buckeye_contexts' + '_projected_' + 'LTR_Buckeye' + '.pV_C',
#      'f':'True',
#      'nb_fp':path.join('LD_Fisher_vocab_in_Buckeye_contexts',
#                        'Filter ' + 'LD_fisher_vocab_in_buckeye_contexts' + ' against ' + 'LTR_Buckeye_aligned_CM_filtered_LM_filtered' + '.ipynb')},
#     {'d':,
#      'v':,
#      'c':,
#      'l':,
#      'o':,
#      'f':,
#      'nb_fp':}
# #     {'d':'LD_Fisher_vocab_in_swbd2003_contexts/LD_fisher_vocab_in_swbd2003_contexts.pV_C',
# #      'v':fisher_lm_vocab_fp,
# #      'c':swbd2003_contexts,
# #      'l':'LTR_CMU_destressed_aligned_w_GD_AmE_destressed/LTR_CMU_destressed_aligned_CM_filtered_LM_filtered.tsv',
# #      'o':'LD_Fisher_vocab_in_swbd2003_contexts/LD_fisher_vocab_in_swbd2003_contexts' + '_projected_' + 'LTR_CMU_destressed' + '.pV_C',
# #      'f':'True',
# #      'nb_fp':path.join('LD_Fisher_vocab_in_swbd2003_contexts',
# #                        'Filter ' + 'LD_fisher_vocab_in_swbd2003_contexts' + ' against ' + 'LTR_CMU_destressed_aligned_CM_filtered_LM_filtered' + '.ipynb')},
# #     {'d':'LD_Fisher_vocab_in_swbd2003_contexts/LD_fisher_vocab_in_swbd2003_contexts.pV_C',
# #      'v':fisher_lm_vocab_fp,
# #      'c':swbd2003_contexts,
# #      'l':'LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered.tsv',
# #      'o':'LD_Fisher_vocab_in_swbd2003_contexts/LD_fisher_vocab_in_swbd2003_contexts' + '_projected_' + 'LTR_newdic_destressed' + '.pV_C',
# #      'f':'True',
# #      'nb_fp':path.join('LD_Fisher_vocab_in_swbd2003_contexts',
# #                        'Filter ' + 'LD_fisher_vocab_in_swbd2003_contexts' + ' against ' + 'LTR_newdic_destressed_aligned_CM_filtered_LM_filtered' + '.ipynb')}
# ]


In [87]:
if '3d' in permittedSteps:
    # used to take about ~10-15m on wittgenstein:
    #  ≤30s for Buckeye vocab + Buckeye contexts
    #  ≈6.5-7m for CMU_destressed vocab in swbd2003 contexts
    #  ≈3m for newdic_destressed vocab in swbd2003 contexts
    
    #takes 56.5m on wittgenstein
    
    for each in LD_projection_args:
        output_dir = path.dirname(each['o'])
        ensure_dir_exists(output_dir)

        progress_report(each['nb_fp'],
                        dict(d = each['d'],
                             v = each['v'],
                             c = each['c'],
                             l = each['l'],
                             o = each['o'],
                             f = each['f']))

        nb = pm.execute_notebook(
        'Filter contextual lexicon distribution by transcription lexicon.ipynb',
        each['nb_fp'],
        parameters=dict(d = each['d'],
                        v = each['v'],
                        c = each['c'],
                        l = each['l'],
                        o = each['o'],
                        f = each['f'])
        )
        endNote()
        print('\n')

## Step 3e: For each (filtered) transcribed lexicon relation, define a conditional distribution on segmental wordforms given an orthographic wordform

**Dependencies**
 - **Step 3b**: `LTR_..._aligned_CM_filtered_LM_filtered.tsv`

**Note**: this is the step where word edge symbols are added to segmental wordform representations.

In [88]:
LTR_CM_filtered_LM_filtered

['LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered.tsv',
 'LTR_CMU_destressed_aligned_w_GD_AmE_destressed/LTR_CMU_destressed_aligned_CM_filtered_LM_filtered.tsv',
 'LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_CM_filtered_LM_filtered.tsv',
 'LTR_NXT_swbd_destressed_aligned_w_GD_AmE_destressed/LTR_NXT_swbd_destressed_aligned_CM_filtered_LM_filtered.tsv']

In [89]:
pW_V_fp_bundles = []
for ltr_fp in LTR_CM_filtered_LM_filtered:
    LTR_n = path.basename( ltr_fp )[:-4]
    LTR_dir = path.dirname( ltr_fp )
    pW_V_fp_bundles.append({'LTR_fp':ltr_fp,
                           'pW_V_fp':ltr_fp[:-4],# + '.pW_V',
                           'nb_fp':path.join(LTR_dir,'Define pW_V given {0}.ipynb'.format(LTR_n))})
    pW_V_fp_bundles.append({'LTR_fp':ltr_fp,
                           'pW_V_fp':ltr_fp[:-4] + '_trim',# + '.pW_V',
                           'nb_fp':path.join(LTR_dir,'Define pW_V given {0}'.format(LTR_n) + '_trim' + '.ipynb'),
                           'r':'False'})
pW_V_fp_bundles

[{'LTR_fp': 'LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered.tsv',
  'pW_V_fp': 'LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered',
  'nb_fp': 'LTR_newdic_destressed_aligned_w_GD_AmE_destressed/Define pW_V given LTR_newdic_destressed_aligned_CM_filtered_LM_filtered.ipynb'},
 {'LTR_fp': 'LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered.tsv',
  'pW_V_fp': 'LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered_trim',
  'nb_fp': 'LTR_newdic_destressed_aligned_w_GD_AmE_destressed/Define pW_V given LTR_newdic_destressed_aligned_CM_filtered_LM_filtered_trim.ipynb',
  'r': 'False'},
 {'LTR_fp': 'LTR_CMU_destressed_aligned_w_GD_AmE_destressed/LTR_CMU_destressed_aligned_CM_filtered_LM_filtered.tsv',
  'pW_V_fp': 'LTR_CMU_destressed_aligned_w_GD_AmE_destressed/LTR_CMU_destressed_align

In [90]:
if '3e' in permittedSteps:
    # used to take about ~30s on wittgenstein
    
    #takes ≈90s on wittgenstein 
    for each in pW_V_fp_bundles:
        pW_V_fp_output_dir = path.dirname(each['pW_V_fp'])
        ensure_dir_exists(pW_V_fp_output_dir)
        
        if not overwrite and path.exists(each['nb_fp']):
#         if not overwrite and path.exists(path.join(ab['cm_dir'], ab['nb_output_name'])):
            print('{0} already exists. Skipping...'.format(path.join(each['nb_fp'])))
#             print('{0} already exists. Skipping...'.format(path.join(ab['cm_dir'], ab['nb_output_name'])))
            endNote()
            continue

        progress_report(each['nb_fp'],
                        dict(l = each['LTR_fp'],
                             o = each['pW_V_fp']))

        nb = pm.execute_notebook(
        'Define a conditional distribution on segmental wordforms given an orthographic one.ipynb',
        each['nb_fp'],
        parameters=dict(l = each['LTR_fp'],
                        o = each['pW_V_fp'])
        )
        endNote()
        print('\n')

# Step 4: Pre-calculate remaining forward model components and meta-data

Note that none of these steps need actually be ordered with respect to each other: the ordering below is arbitrary.

## Step 4a: Generate triphone lexicon distributions for every triphone channel model

**Dependencies**
 - **Step 3c**: `LTR_..._aligned_CM_filtered_LM_filtered_pY1X0X1X2.json`

In [91]:
def get_immediate_subdirectories(a_dir):
    return [name for name in listdir(a_dir)
            if path.isdir(path.join(a_dir, name))]

In [92]:
subdirs = get_immediate_subdirectories('.')
len(subdirs)

60

In [93]:
channel_model_fps = []
for d in subdirs:
    files = listdir(d)
    is_triph_channel_model = lambda fn: 'pY1X0X1X2.json' in fn
    CM_files = list(filter(is_triph_channel_model,
                           files))
    for CM_file in CM_files:
        if 'old' not in d:
            channel_model_fps.append(path.join(d, CM_file))
len(channel_model_fps)
channel_model_fps

27

['CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.05/pY1X0X1X2.json',
 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.05/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered_pY1X0X1X2.json',
 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.1/pY1X0X1X2.json',
 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.1/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered_pY1X0X1X2.json',
 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.01/pY1X0X1X2.json',
 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.01/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered_pY1X0X1X2.json',
 'CM_AmE_destressed_unaligned_pseudocount0.01/pY1X0X1X2.json',
 'CM_AmE_destressed_aligned_w_LTR_Buckeye_pseudocount0.01/pY1X0X1X2.json',
 'CM_AmE_destressed_aligned_w_LTR_Buckeye_pseudocount0.01/LTR_Buckeye_aligned_CM_filtered_LM_filtered_pY1X0X1X2.json',
 'CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.01/LTR_CMU_

In [94]:
triph_lex_bundles = []
for cm_fp in channel_model_fps:
    bundle = dict()
    bundle['c'] = cm_fp
    bundle['output_dir'] = path.dirname(cm_fp)
    bundle['c_fn'] = path.basename(cm_fp)
    bundle['o_fn_prefix'] = bundle['c_fn'].split('pY1X0X1X2')[0] + 'pX0X1X2'
    bundle['o'] = path.join(bundle['output_dir'],
                            bundle['o_fn_prefix'])
    bundle['nb_fp'] = path.join(bundle['output_dir'],
                                f"Generating {bundle['c_fn'].split('pY1X0X1X2.json')[0][:-1]} uniform triphone lexicon dist.ipynb")
    bundle
    print(' ')
    triph_lex_bundles.append(bundle)

{'c': 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.05/pY1X0X1X2.json',
 'output_dir': 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.05',
 'c_fn': 'pY1X0X1X2.json',
 'o_fn_prefix': 'pX0X1X2',
 'o': 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.05/pX0X1X2',
 'nb_fp': 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.05/Generating  uniform triphone lexicon dist.ipynb'}

{'c': 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.05/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered_pY1X0X1X2.json',
 'output_dir': 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.05',
 'c_fn': 'LTR_newdic_destressed_aligned_CM_filtered_LM_filtered_pY1X0X1X2.json',
 'o_fn_prefix': 'LTR_newdic_destressed_aligned_CM_filtered_LM_filtered_pX0X1X2',
 'o': 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.05/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered_pX0X1X2',
 'nb_fp': 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.05/Generating LTR_newdic_destressed_aligned_CM_filtered_LM_filtered uniform triphone lexicon dist.ipynb'}

{'c': 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.1/pY1X0X1X2.json',
 'output_dir': 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.1',
 'c_fn': 'pY1X0X1X2.json',
 'o_fn_prefix': 'pX0X1X2',
 'o': 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.1/pX0X1X2',
 'nb_fp': 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.1/Generating  uniform triphone lexicon dist.ipynb'}

{'c': 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.1/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered_pY1X0X1X2.json',
 'output_dir': 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.1',
 'c_fn': 'LTR_newdic_destressed_aligned_CM_filtered_LM_filtered_pY1X0X1X2.json',
 'o_fn_prefix': 'LTR_newdic_destressed_aligned_CM_filtered_LM_filtered_pX0X1X2',
 'o': 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.1/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered_pX0X1X2',
 'nb_fp': 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.1/Generating LTR_newdic_destressed_aligned_CM_filtered_LM_filtered uniform triphone lexicon dist.ipynb'}

{'c': 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.01/pY1X0X1X2.json',
 'output_dir': 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.01',
 'c_fn': 'pY1X0X1X2.json',
 'o_fn_prefix': 'pX0X1X2',
 'o': 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.01/pX0X1X2',
 'nb_fp': 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.01/Generating  uniform triphone lexicon dist.ipynb'}

...

{'c': 'CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.05/pY1X0X1X2.json',
 'output_dir': 'CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.05',
 'c_fn': 'pY1X0X1X2.json',
 'o_fn_prefix': 'pX0X1X2',
 'o': 'CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.05/pX0X1X2',
 'nb_fp': 'CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.05/Generating  uniform triphone lexicon dist.ipynb'}

{'c': 'CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.05/LTR_CMU_destressed_aligned_CM_filtered_LM_filtered_pY1X0X1X2.json',
 'output_dir': 'CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.05',
 'c_fn': 'LTR_CMU_destressed_aligned_CM_filtered_LM_filtered_pY1X0X1X2.json',
 'o_fn_prefix': 'LTR_CMU_destressed_aligned_CM_filtered_LM_filtered_pX0X1X2',
 'o': 'CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.05/LTR_CMU_destressed_aligned_CM_filtered_LM_filtered_pX0X1X2',
 'nb_fp': 'CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.05/Generating LTR_CMU_destressed_aligned_CM_filtered_LM_filtered uniform triphone lexicon dist.ipynb'}

{'c': 'CM_AmE_destressed_aligned_w_LTR_Buckeye_pseudocount0.05/pY1X0X1X2.json',
 'output_dir': 'CM_AmE_destressed_aligned_w_LTR_Buckeye_pseudocount0.05',
 'c_fn': 'pY1X0X1X2.json',
 'o_fn_prefix': 'pX0X1X2',
 'o': 'CM_AmE_destressed_aligned_w_LTR_Buckeye_pseudocount0.05/pX0X1X2',
 'nb_fp': 'CM_AmE_destressed_aligned_w_LTR_Buckeye_pseudocount0.05/Generating  uniform triphone lexicon dist.ipynb'}

{'c': 'CM_AmE_destressed_aligned_w_LTR_Buckeye_pseudocount0.05/LTR_Buckeye_aligned_CM_filtered_LM_filtered_pY1X0X1X2.json',
 'output_dir': 'CM_AmE_destressed_aligned_w_LTR_Buckeye_pseudocount0.05',
 'c_fn': 'LTR_Buckeye_aligned_CM_filtered_LM_filtered_pY1X0X1X2.json',
 'o_fn_prefix': 'LTR_Buckeye_aligned_CM_filtered_LM_filtered_pX0X1X2',
 'o': 'CM_AmE_destressed_aligned_w_LTR_Buckeye_pseudocount0.05/LTR_Buckeye_aligned_CM_filtered_LM_filtered_pX0X1X2',
 'nb_fp': 'CM_AmE_destressed_aligned_w_LTR_Buckeye_pseudocount0.05/Generating LTR_Buckeye_aligned_CM_filtered_LM_filtered uniform triphone lexicon dist.ipynb'}

{'c': 'CM_AmE_destressed_unaligned_pseudocount0.05/pY1X0X1X2.json',
 'output_dir': 'CM_AmE_destressed_unaligned_pseudocount0.05',
 'c_fn': 'pY1X0X1X2.json',
 'o_fn_prefix': 'pX0X1X2',
 'o': 'CM_AmE_destressed_unaligned_pseudocount0.05/pX0X1X2',
 'nb_fp': 'CM_AmE_destressed_unaligned_pseudocount0.05/Generating  uniform triphone lexicon dist.ipynb'}

In [95]:
if '4a' in permittedSteps:
    # used to take about ~1m on wittgenstein
    
    #takes 90s on wittgenstein
    for bundle in triph_lex_bundles:
        output_dir = bundle['output_dir']
        if not path.exists(output_dir):
            print(f"Making output path {output_dir}")
            makedirs(output_dir)

        if not overwrite and path.exists(bundle['nb_fp']):
#         if not overwrite and path.exists(path.join(ab['cm_dir'], ab['nb_output_name'])):
            print('{0} already exists. Skipping...'.format(path.join(bundle['nb_fp'])))
#             print('{0} already exists. Skipping...'.format(path.join(ab['cm_dir'], ab['nb_output_name'])))
            endNote()
            continue
            
        progress_report(bundle['nb_fp'],
                        dict(c = bundle['c'],
                             o = bundle['o']))

        nb = pm.execute_notebook(
        'Generate triphone lexicon distribution from channel model.ipynb',
        bundle['nb_fp'],
        parameters=dict(c = bundle['c'],
                        o = bundle['o'])
        )
        endNote()
        print('\n')

## Step 4b: Pre-calculate prefix relation, $k$-cousins, and $k$-spheres for each segmental lexicon

**Dependencies**
 - **Step 3e**: `...pW_V.json`

(This is comparable to the `Metadata` generation step in 2a above.)

In [96]:
pW_V_fps = [each['pW_V_fp'] + '.pW_V.json' for each in pW_V_fp_bundles]
pW_V_fps

['LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered.pW_V.json',
 'LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered_trim.pW_V.json',
 'LTR_CMU_destressed_aligned_w_GD_AmE_destressed/LTR_CMU_destressed_aligned_CM_filtered_LM_filtered.pW_V.json',
 'LTR_CMU_destressed_aligned_w_GD_AmE_destressed/LTR_CMU_destressed_aligned_CM_filtered_LM_filtered_trim.pW_V.json',
 'LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_CM_filtered_LM_filtered.pW_V.json',
 'LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_CM_filtered_LM_filtered_trim.pW_V.json',
 'LTR_NXT_swbd_destressed_aligned_w_GD_AmE_destressed/LTR_NXT_swbd_destressed_aligned_CM_filtered_LM_filtered.pW_V.json',
 'LTR_NXT_swbd_destressed_aligned_w_GD_AmE_destressed/LTR_NXT_swbd_destressed_aligned_CM_filtered_LM_filtered_trim.pW_V.json']

In [97]:
# pW_V_fps = []
# for d in subdirs:
#     files = listdir(d)
#     is_pW_V = lambda fn: 'pW_V.json' in fn
#     pW_V_files = list(filter(is_pW_V,
#                            files))
#     for pW_V_file in pW_V_files:
#         pW_V_fps.append(path.join(d, pW_V_file))
# len(pW_V_fps)
# pW_V_fps

In [98]:
lexicon_md_bundles = []
for pW_V_fp in pW_V_fps:
    
    
    bundle = dict()
    bundle['p'] = pW_V_fp
    bundle['lex_name'] = path.basename(pW_V_fp).split('.pW_V.json')[0]
    bundle['o'] = path.join(path.dirname(pW_V_fp), path.basename(pW_V_fp).split('.pW_V.json')[0] )
    
    output_dir = path.dirname(pW_V_fp)
    if not path.exists(output_dir):
        print(f"Making output path '{output_dir}'")
        makedirs(output_dir)
    
    bundle['nb_fp'] = path.join(path.dirname(pW_V_fp),
                                f"Calculate word-prefix relation, Hamming distances, and k-cousin relation for {bundle['lex_name']}.ipynb")
#                                 f"Calculate prefix data, k-cousins, and k-spheres for {bundle['lex_name']}.ipynb")
    bundle
    print(' ')
    lexicon_md_bundles.append(bundle)

{'p': 'LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered.pW_V.json',
 'lex_name': 'LTR_newdic_destressed_aligned_CM_filtered_LM_filtered',
 'o': 'LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered',
 'nb_fp': 'LTR_newdic_destressed_aligned_w_GD_AmE_destressed/Calculate word-prefix relation, Hamming distances, and k-cousin relation for LTR_newdic_destressed_aligned_CM_filtered_LM_filtered.ipynb'}

{'p': 'LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered_trim.pW_V.json',
 'lex_name': 'LTR_newdic_destressed_aligned_CM_filtered_LM_filtered_trim',
 'o': 'LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered_trim',
 'nb_fp': 'LTR_newdic_destressed_aligned_w_GD_AmE_destressed/Calculate word-prefix relation, Hamming distances, and k-cousin relation for LTR_newdic_destressed_aligned_CM_filtered_LM_filtered_trim.ipynb'}

{'p': 'LTR_CMU_destressed_aligned_w_GD_AmE_destressed/LTR_CMU_destressed_aligned_CM_filtered_LM_filtered.pW_V.json',
 'lex_name': 'LTR_CMU_destressed_aligned_CM_filtered_LM_filtered',
 'o': 'LTR_CMU_destressed_aligned_w_GD_AmE_destressed/LTR_CMU_destressed_aligned_CM_filtered_LM_filtered',
 'nb_fp': 'LTR_CMU_destressed_aligned_w_GD_AmE_destressed/Calculate word-prefix relation, Hamming distances, and k-cousin relation for LTR_CMU_destressed_aligned_CM_filtered_LM_filtered.ipynb'}

{'p': 'LTR_CMU_destressed_aligned_w_GD_AmE_destressed/LTR_CMU_destressed_aligned_CM_filtered_LM_filtered_trim.pW_V.json',
 'lex_name': 'LTR_CMU_destressed_aligned_CM_filtered_LM_filtered_trim',
 'o': 'LTR_CMU_destressed_aligned_w_GD_AmE_destressed/LTR_CMU_destressed_aligned_CM_filtered_LM_filtered_trim',
 'nb_fp': 'LTR_CMU_destressed_aligned_w_GD_AmE_destressed/Calculate word-prefix relation, Hamming distances, and k-cousin relation for LTR_CMU_destressed_aligned_CM_filtered_LM_filtered_trim.ipynb'}

{'p': 'LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_CM_filtered_LM_filtered.pW_V.json',
 'lex_name': 'LTR_Buckeye_aligned_CM_filtered_LM_filtered',
 'o': 'LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_CM_filtered_LM_filtered',
 'nb_fp': 'LTR_Buckeye_aligned_w_GD_AmE_destressed/Calculate word-prefix relation, Hamming distances, and k-cousin relation for LTR_Buckeye_aligned_CM_filtered_LM_filtered.ipynb'}

{'p': 'LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_CM_filtered_LM_filtered_trim.pW_V.json',
 'lex_name': 'LTR_Buckeye_aligned_CM_filtered_LM_filtered_trim',
 'o': 'LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_CM_filtered_LM_filtered_trim',
 'nb_fp': 'LTR_Buckeye_aligned_w_GD_AmE_destressed/Calculate word-prefix relation, Hamming distances, and k-cousin relation for LTR_Buckeye_aligned_CM_filtered_LM_filtered_trim.ipynb'}

{'p': 'LTR_NXT_swbd_destressed_aligned_w_GD_AmE_destressed/LTR_NXT_swbd_destressed_aligned_CM_filtered_LM_filtered.pW_V.json',
 'lex_name': 'LTR_NXT_swbd_destressed_aligned_CM_filtered_LM_filtered',
 'o': 'LTR_NXT_swbd_destressed_aligned_w_GD_AmE_destressed/LTR_NXT_swbd_destressed_aligned_CM_filtered_LM_filtered',
 'nb_fp': 'LTR_NXT_swbd_destressed_aligned_w_GD_AmE_destressed/Calculate word-prefix relation, Hamming distances, and k-cousin relation for LTR_NXT_swbd_destressed_aligned_CM_filtered_LM_filtered.ipynb'}

{'p': 'LTR_NXT_swbd_destressed_aligned_w_GD_AmE_destressed/LTR_NXT_swbd_destressed_aligned_CM_filtered_LM_filtered_trim.pW_V.json',
 'lex_name': 'LTR_NXT_swbd_destressed_aligned_CM_filtered_LM_filtered_trim',
 'o': 'LTR_NXT_swbd_destressed_aligned_w_GD_AmE_destressed/LTR_NXT_swbd_destressed_aligned_CM_filtered_LM_filtered_trim',
 'nb_fp': 'LTR_NXT_swbd_destressed_aligned_w_GD_AmE_destressed/Calculate word-prefix relation, Hamming distances, and k-cousin relation for LTR_NXT_swbd_destressed_aligned_CM_filtered_LM_filtered_trim.ipynb'}

In [99]:
if '4b' in permittedSteps:
    #oldest runtimes...
    # with J=-1, no CPU load, and no .npz export
    #  - newdic CM_filtered_LM_filtered takes ~45m (~168m on Solomonoff)
    #  - CMU CM_filtered_LM_filtered takes ~3.5-3.75h (14-15h? on Quine?)
    # - Buckeye CM_filtered_LM_filtered takes ~20m (~80m on Quine)

    #older runtimes...
    # with J=-1, no CPU load, *and* .npz export
    #  - newdic CM_filtered_LM_filtered takes ~45m
    #  - CMU CM_filtered_LM_filtered takes ~20.5h
    # - Buckeye CM_filtered_LM_filtered takes ~19.5h
    # (.npz representations are up to 20x smaller on disk 
    #  and are significantly smaller in memory when loaded)

    # with J=-1, no CPU load, *and* .npz export
    #  - newdic CM_filtered_LM_filtered takes ~?m
    #  - CMU CM_filtered_LM_filtered takes ~40h+ on wittgenstein
    # - Buckeye CM_filtered_LM_filtered takes ~?h
    # (.npz representations are up to 20x smaller on disk 
    #  and are significantly smaller in memory when loaded)
    
    #current runtime = 80m on wittgenstein
    # newdic takes 5.5m
    # CMU takes 60m / peak 55GB or 68GB mem usage
    # Buckeye takes 3.16m
    # NXT_swbd takes 9.3m
    for bundle in lexicon_md_bundles:
        
        output_dir = path.dirname(bundle['o'])
        if not path.exists(output_dir):
            print(f"Making output path {output_dir}")
            makedirs(output_dir)

        if not overwrite and path.exists(bundle['nb_fp']):
#         if not overwrite and path.exists(path.join(ab['cm_dir'], ab['nb_output_name'])):
            print('{0} already exists. Skipping...'.format(path.join(bundle['nb_fp'])))
#             print('{0} already exists. Skipping...'.format(path.join(ab['cm_dir'], ab['nb_output_name'])))
            endNote()
            continue
        
        progress_report(bundle['nb_fp'],
                        dict(p = bundle['p'],
                             o = bundle['o']))

        nb = pm.execute_notebook(
        'Calculate word-prefix relation, Hamming distances, and k-cousin relation.ipynb',
        bundle['nb_fp'],
        parameters=dict(p = bundle['p'],
                        o = bundle['o'])
        )
        endNote()
        print('\n')

## Step 4c: Calculate the marginal probability $p(W|C)$ of each segmental wordform $w$ given $n$-gram contexts $C$

**Dependencies**
 - **Step 3e**: `pW_V` matrix
 - **Step 3d**: `LD_fisher_vocab_in...contexts_projected_LTR...pV_C.npy` matrix

In [100]:
#gather pV_C, pW_V fp pairs as numpy arrays

In [101]:
def LTR_to_pW_Vs(LTR_fp):
    matching_bundles = list(filter(lambda bundle: bundle['LTR_fp'] == LTR_fp,
                                   pW_V_fp_bundles))
    matching_pW_V_fps = set(map(lambda bundle: bundle['pW_V_fp'],
                                matching_bundles))
    return matching_pW_V_fps

In [102]:
pW_V_fp_bundles

[{'LTR_fp': 'LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered.tsv',
  'pW_V_fp': 'LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered',
  'nb_fp': 'LTR_newdic_destressed_aligned_w_GD_AmE_destressed/Define pW_V given LTR_newdic_destressed_aligned_CM_filtered_LM_filtered.ipynb'},
 {'LTR_fp': 'LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered.tsv',
  'pW_V_fp': 'LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered_trim',
  'nb_fp': 'LTR_newdic_destressed_aligned_w_GD_AmE_destressed/Define pW_V given LTR_newdic_destressed_aligned_CM_filtered_LM_filtered_trim.ipynb',
  'r': 'False'},
 {'LTR_fp': 'LTR_CMU_destressed_aligned_w_GD_AmE_destressed/LTR_CMU_destressed_aligned_CM_filtered_LM_filtered.tsv',
  'pW_V_fp': 'LTR_CMU_destressed_aligned_w_GD_AmE_destressed/LTR_CMU_destressed_align

In [103]:
def LTR_to_LD(LTR_fp):
    matching_bundles = list(filter(lambda bundle: bundle['l'] == LTR_fp,
                                   LD_projection_args))
    matching_pV_C_fps = set(map(lambda bundle: bundle['o'],
                                matching_bundles))
    return matching_pV_C_fps

In [104]:
LD_projection_args

[{'d': 'LD_Fisher_vocab_in_Buckeye_following_contexts_2gram_model/LD_Fisher_vocab_in_Buckeye_following_contexts_2gram_model.pV_C',
  'v': 'LM_Fisher/fisher_vocabulary_main.txt',
  'c': 'C_Buckeye/buckeye_contexts_following_1_filtered.txt',
  'l': 'LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_CM_filtered_LM_filtered.tsv',
  'o': 'LD_Fisher_vocab_in_Buckeye_following_contexts_2gram_model/LD_Fisher_vocab_in_Buckeye_following_contexts_2gram_model_projected_LTR_Buckeye_aligned_CM_filtered_LM_filtered.pV_C',
  'f': 'True',
  'nb_fp': 'LD_Fisher_vocab_in_Buckeye_following_contexts_2gram_model/Filter LD_Fisher_vocab_in_Buckeye_following_contexts_2gram_model against LTR_Buckeye_aligned_CM_filtered_LM_filtered.ipynb'},
 {'d': 'LD_Fisher_vocab_in_Buckeye_following_contexts_3gram_model/LD_Fisher_vocab_in_Buckeye_following_contexts_3gram_model.pV_C',
  'v': 'LM_Fisher/fisher_vocabulary_main.txt',
  'c': 'C_Buckeye/buckeye_contexts_following_2_filtered.txt',
  'l': 'LTR_Buckeye_aligne

In [105]:
def get_matched_pW_V_LD_pairs(LTR_fp):
    matching_pW_V_fps = LTR_to_pW_Vs(LTR_fp)
    matching_pV_C_fps = LTR_to_LD(LTR_fp)
    
    return set(product(matching_pW_V_fps,
                       matching_pV_C_fps))

my_LTR_fps = list(map(lambda bundle: bundle['LTR_fp'],
                      pW_V_fp_bundles))

matched_pW_V_LD_pairs = union(map(get_matched_pW_V_LD_pairs,
                                  my_LTR_fps))
matched_pW_V_LD_pairs

{('LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_CM_filtered_LM_filtered',
  'LD_Fisher_vocab_in_Buckeye_following_contexts_2gram_model/LD_Fisher_vocab_in_Buckeye_following_contexts_2gram_model_projected_LTR_Buckeye_aligned_CM_filtered_LM_filtered.pV_C'),
 ('LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_CM_filtered_LM_filtered',
  'LD_Fisher_vocab_in_Buckeye_following_contexts_3gram_model/LD_Fisher_vocab_in_Buckeye_following_contexts_3gram_model_projected_LTR_Buckeye_aligned_CM_filtered_LM_filtered.pV_C'),
 ('LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_CM_filtered_LM_filtered',
  'LD_Fisher_vocab_in_Buckeye_following_contexts_4gram_model/LD_Fisher_vocab_in_Buckeye_following_contexts_4gram_model_projected_LTR_Buckeye_aligned_CM_filtered_LM_filtered.pV_C'),
 ('LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_CM_filtered_LM_filtered',
  'LD_Fisher_vocab_in_Buckeye_following_contexts_5gram_model/LD_Fisher_vocab_in_Buckeye_following_con

In [106]:
WD_bundles = []
for w,d in matched_pW_V_LD_pairs:
    bundle = dict()
    
    LTR_key = path.basename(w)
    LD_key = path.basename(d)
    C_key = LD_key.split('LD_Fisher_vocab')[1].split('_projected_')[0]
    
    output_dir = path.dirname(d)
    output_prefix = LTR_key + C_key + '.pW_C'
    
    bundle['d'] = d + '.npy'
    bundle['w'] = w + '.pW_V.npz'
    
    bundle['o'] = path.join(output_dir, output_prefix)
    
    bundle['nb_fp'] = path.join(output_dir, f"Calculate segmental wordform distribution for {LTR_key}{C_key.replace('_', ' ')}.ipynb")
    
    bundle
                                
    trim_bundle = deepcopy(bundle)
    trim_bundle['w'] = w + '_trim' + '.pW_V.npz'
    trim_bundle
    
    WD_bundles.append(bundle)

{'d': 'LD_Fisher_vocab_in_NXT_swbd_preceding_contexts_2gram_model/LD_Fisher_vocab_in_NXT_swbd_preceding_contexts_2gram_model_projected_LTR_NXT_swbd_destressed_aligned_CM_filtered_LM_filtered.pV_C.npy',
 'w': 'LTR_NXT_swbd_destressed_aligned_w_GD_AmE_destressed/LTR_NXT_swbd_destressed_aligned_CM_filtered_LM_filtered_trim.pW_V.npz',
 'o': 'LD_Fisher_vocab_in_NXT_swbd_preceding_contexts_2gram_model/LTR_NXT_swbd_destressed_aligned_CM_filtered_LM_filtered_trim_in_NXT_swbd_preceding_contexts_2gram_model.pW_C',
 'nb_fp': 'LD_Fisher_vocab_in_NXT_swbd_preceding_contexts_2gram_model/Calculate segmental wordform distribution for LTR_NXT_swbd_destressed_aligned_CM_filtered_LM_filtered_trim in NXT swbd preceding contexts 2gram model.ipynb'}

{'d': 'LD_Fisher_vocab_in_NXT_swbd_preceding_contexts_2gram_model/LD_Fisher_vocab_in_NXT_swbd_preceding_contexts_2gram_model_projected_LTR_NXT_swbd_destressed_aligned_CM_filtered_LM_filtered.pV_C.npy',
 'w': 'LTR_NXT_swbd_destressed_aligned_w_GD_AmE_destressed/LTR_NXT_swbd_destressed_aligned_CM_filtered_LM_filtered_trim_trim.pW_V.npz',
 'o': 'LD_Fisher_vocab_in_NXT_swbd_preceding_contexts_2gram_model/LTR_NXT_swbd_destressed_aligned_CM_filtered_LM_filtered_trim_in_NXT_swbd_preceding_contexts_2gram_model.pW_C',
 'nb_fp': 'LD_Fisher_vocab_in_NXT_swbd_preceding_contexts_2gram_model/Calculate segmental wordform distribution for LTR_NXT_swbd_destressed_aligned_CM_filtered_LM_filtered_trim in NXT swbd preceding contexts 2gram model.ipynb'}

{'d': 'LD_Fisher_vocab_in_Buckeye_preceding_contexts_2gram_model/LD_Fisher_vocab_in_Buckeye_preceding_contexts_2gram_model_projected_LTR_Buckeye_aligned_CM_filtered_LM_filtered.pV_C.npy',
 'w': 'LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_CM_filtered_LM_filtered_trim.pW_V.npz',
 'o': 'LD_Fisher_vocab_in_Buckeye_preceding_contexts_2gram_model/LTR_Buckeye_aligned_CM_filtered_LM_filtered_trim_in_Buckeye_preceding_contexts_2gram_model.pW_C',
 'nb_fp': 'LD_Fisher_vocab_in_Buckeye_preceding_contexts_2gram_model/Calculate segmental wordform distribution for LTR_Buckeye_aligned_CM_filtered_LM_filtered_trim in Buckeye preceding contexts 2gram model.ipynb'}

{'d': 'LD_Fisher_vocab_in_Buckeye_preceding_contexts_2gram_model/LD_Fisher_vocab_in_Buckeye_preceding_contexts_2gram_model_projected_LTR_Buckeye_aligned_CM_filtered_LM_filtered.pV_C.npy',
 'w': 'LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_CM_filtered_LM_filtered_trim_trim.pW_V.npz',
 'o': 'LD_Fisher_vocab_in_Buckeye_preceding_contexts_2gram_model/LTR_Buckeye_aligned_CM_filtered_LM_filtered_trim_in_Buckeye_preceding_contexts_2gram_model.pW_C',
 'nb_fp': 'LD_Fisher_vocab_in_Buckeye_preceding_contexts_2gram_model/Calculate segmental wordform distribution for LTR_Buckeye_aligned_CM_filtered_LM_filtered_trim in Buckeye preceding contexts 2gram model.ipynb'}

{'d': 'LD_Fisher_vocab_in_NXT_swbd_following_contexts_5gram_model/LD_Fisher_vocab_in_NXT_swbd_following_contexts_5gram_model_projected_LTR_NXT_swbd_destressed_aligned_CM_filtered_LM_filtered.pV_C.npy',
 'w': 'LTR_NXT_swbd_destressed_aligned_w_GD_AmE_destressed/LTR_NXT_swbd_destressed_aligned_CM_filtered_LM_filtered_trim.pW_V.npz',
 'o': 'LD_Fisher_vocab_in_NXT_swbd_following_contexts_5gram_model/LTR_NXT_swbd_destressed_aligned_CM_filtered_LM_filtered_trim_in_NXT_swbd_following_contexts_5gram_model.pW_C',
 'nb_fp': 'LD_Fisher_vocab_in_NXT_swbd_following_contexts_5gram_model/Calculate segmental wordform distribution for LTR_NXT_swbd_destressed_aligned_CM_filtered_LM_filtered_trim in NXT swbd following contexts 5gram model.ipynb'}

...

{'d': 'LD_Fisher_vocab_in_Buckeye_preceding_contexts_4gram_model/LD_Fisher_vocab_in_Buckeye_preceding_contexts_4gram_model_projected_LTR_Buckeye_aligned_CM_filtered_LM_filtered.pV_C.npy',
 'w': 'LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_CM_filtered_LM_filtered_trim.pW_V.npz',
 'o': 'LD_Fisher_vocab_in_Buckeye_preceding_contexts_4gram_model/LTR_Buckeye_aligned_CM_filtered_LM_filtered_in_Buckeye_preceding_contexts_4gram_model.pW_C',
 'nb_fp': 'LD_Fisher_vocab_in_Buckeye_preceding_contexts_4gram_model/Calculate segmental wordform distribution for LTR_Buckeye_aligned_CM_filtered_LM_filtered in Buckeye preceding contexts 4gram model.ipynb'}

{'d': 'LD_Fisher_vocab_in_Buckeye_following_contexts_3gram_model/LD_Fisher_vocab_in_Buckeye_following_contexts_3gram_model_projected_LTR_Buckeye_aligned_CM_filtered_LM_filtered.pV_C.npy',
 'w': 'LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_CM_filtered_LM_filtered.pW_V.npz',
 'o': 'LD_Fisher_vocab_in_Buckeye_following_contexts_3gram_model/LTR_Buckeye_aligned_CM_filtered_LM_filtered_in_Buckeye_following_contexts_3gram_model.pW_C',
 'nb_fp': 'LD_Fisher_vocab_in_Buckeye_following_contexts_3gram_model/Calculate segmental wordform distribution for LTR_Buckeye_aligned_CM_filtered_LM_filtered in Buckeye following contexts 3gram model.ipynb'}

{'d': 'LD_Fisher_vocab_in_Buckeye_following_contexts_3gram_model/LD_Fisher_vocab_in_Buckeye_following_contexts_3gram_model_projected_LTR_Buckeye_aligned_CM_filtered_LM_filtered.pV_C.npy',
 'w': 'LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_CM_filtered_LM_filtered_trim.pW_V.npz',
 'o': 'LD_Fisher_vocab_in_Buckeye_following_contexts_3gram_model/LTR_Buckeye_aligned_CM_filtered_LM_filtered_in_Buckeye_following_contexts_3gram_model.pW_C',
 'nb_fp': 'LD_Fisher_vocab_in_Buckeye_following_contexts_3gram_model/Calculate segmental wordform distribution for LTR_Buckeye_aligned_CM_filtered_LM_filtered in Buckeye following contexts 3gram model.ipynb'}

{'d': 'LD_Fisher_vocab_in_Buckeye_following_contexts_5gram_model/LD_Fisher_vocab_in_Buckeye_following_contexts_5gram_model_projected_LTR_Buckeye_aligned_CM_filtered_LM_filtered.pV_C.npy',
 'w': 'LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_CM_filtered_LM_filtered_trim.pW_V.npz',
 'o': 'LD_Fisher_vocab_in_Buckeye_following_contexts_5gram_model/LTR_Buckeye_aligned_CM_filtered_LM_filtered_trim_in_Buckeye_following_contexts_5gram_model.pW_C',
 'nb_fp': 'LD_Fisher_vocab_in_Buckeye_following_contexts_5gram_model/Calculate segmental wordform distribution for LTR_Buckeye_aligned_CM_filtered_LM_filtered_trim in Buckeye following contexts 5gram model.ipynb'}

{'d': 'LD_Fisher_vocab_in_Buckeye_following_contexts_5gram_model/LD_Fisher_vocab_in_Buckeye_following_contexts_5gram_model_projected_LTR_Buckeye_aligned_CM_filtered_LM_filtered.pV_C.npy',
 'w': 'LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_CM_filtered_LM_filtered_trim_trim.pW_V.npz',
 'o': 'LD_Fisher_vocab_in_Buckeye_following_contexts_5gram_model/LTR_Buckeye_aligned_CM_filtered_LM_filtered_trim_in_Buckeye_following_contexts_5gram_model.pW_C',
 'nb_fp': 'LD_Fisher_vocab_in_Buckeye_following_contexts_5gram_model/Calculate segmental wordform distribution for LTR_Buckeye_aligned_CM_filtered_LM_filtered_trim in Buckeye following contexts 5gram model.ipynb'}

In [107]:
if '4c' in permittedSteps:
    # used to take ≈5-10m on wittgenstein with no background load
    #  ≈8-10m for filtered LTR_CMU_destressed in swbd2003 contexts
    #  ≤30s for filtered LTR_Buckeye in buckeye contexts
    #  ≈2-3m for filtered LTR_newdic_destressed in swbd2003 contexts
    
    #now takes 18m on wittgenstein
    for bundle in WD_bundles:
        ensure_dir_exists(path.dirname(bundle['o']))

        if not overwrite and path.exists(bundle['nb_fp']):
#         if not overwrite and path.exists(path.join(ab['cm_dir'], ab['nb_output_name'])):
            print('{0} already exists. Skipping...'.format(path.join(bundle['nb_fp'])))
#             print('{0} already exists. Skipping...'.format(path.join(ab['cm_dir'], ab['nb_output_name'])))
            endNote()
            continue
        
        progress_report(bundle['nb_fp'],
                        dict(d = bundle['d'],
                             w = bundle['w'],
                             o = bundle['o']))

        nb = pm.execute_notebook(
        'Calculate segmental wordform distribution given corpus contexts.ipynb',
        bundle['nb_fp'],
        parameters=dict(d = bundle['d'],
                        w = bundle['w'],
                        o = bundle['o'])
        )
        endNote()
        print('\n')

## Step 4d: Define observation distributions

**Dependencies**
 - **Step 3c**: `LTR_..._aligned_CM_filtered_LM_filtered_pY1X0X1X2.json`
 - **Step 3c**: `LTR_..._aligned_CM_filtered_LM_filtered_p3Y1X01.json`
 - **Step 3c**: `LTR_..._aligned_CM_filtered_LM_filtered_p6Y0X01.json`

In [108]:
# identify trimmed center (i.e. triphone) channel model fps defined earlier
trimmed_CM_bundles = [{'center':bundle['o']} for bundle in trimmed_LTR_CM_triples]
trimmed_CM_bundles

[{'center': 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.01/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered_pY1X0X1X2.json'},
 {'center': 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.05/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered_pY1X0X1X2.json'},
 {'center': 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.1/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered_pY1X0X1X2.json'},
 {'center': 'CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.01/LTR_CMU_destressed_aligned_CM_filtered_LM_filtered_pY1X0X1X2.json'},
 {'center': 'CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.05/LTR_CMU_destressed_aligned_CM_filtered_LM_filtered_pY1X0X1X2.json'},
 {'center': 'CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.1/LTR_CMU_destressed_aligned_CM_filtered_LM_filtered_pY1X0X1X2.json'},
 {'center': 'CM_AmE_destressed_aligned_w_LTR_Buckeye_pseudocount0.01/LTR_Buckeye_aligned_CM_filtered_LM_filtered_p

In [109]:
# add inferred (err, hardcoded-in-step-3c) fps for preview and postview distributions
for bundle in trimmed_CM_bundles:
    dirpath = path.dirname(bundle['center'])
    processing_prefix = path.basename(bundle['center']).split('pY1X0X1X2.json')[0]
    bundle['preview'] = path.join(dirpath, processing_prefix + 'p3Y1X01.json')
    bundle['postview'] = path.join(dirpath, processing_prefix + 'p6Y0X01.json')

In [110]:
observation_bundles = []
for bundle in trimmed_CM_bundles:
    new_bundle = dict()
    
    dirpath = path.dirname(bundle['center'])
    processing_prefix = path.basename(bundle['center']).split('pY1X0X1X2.json')[0]
    
    new_bundle['l'] = bundle['postview']
    new_bundle['c'] = bundle['center']
    new_bundle['r'] = bundle['preview']
    new_bundle['o'] = path.join(dirpath, processing_prefix + 'pC1X012')
    
#     pprintable_proc_pref = processing_prefix[:-1]
    new_bundle['nb_fp'] = path.join(dirpath, f"Calculate {processing_prefix[:-1]} observation distribution given channel models.ipynb")
    
    new_bundle
    observation_bundles.append(new_bundle)

{'l': 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.01/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered_p6Y0X01.json',
 'c': 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.01/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered_pY1X0X1X2.json',
 'r': 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.01/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered_p3Y1X01.json',
 'o': 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.01/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered_pC1X012',
 'nb_fp': 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.01/Calculate LTR_newdic_destressed_aligned_CM_filtered_LM_filtered observation distribution given channel models.ipynb'}

{'l': 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.05/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered_p6Y0X01.json',
 'c': 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.05/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered_pY1X0X1X2.json',
 'r': 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.05/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered_p3Y1X01.json',
 'o': 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.05/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered_pC1X012',
 'nb_fp': 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.05/Calculate LTR_newdic_destressed_aligned_CM_filtered_LM_filtered observation distribution given channel models.ipynb'}

{'l': 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.1/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered_p6Y0X01.json',
 'c': 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.1/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered_pY1X0X1X2.json',
 'r': 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.1/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered_p3Y1X01.json',
 'o': 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.1/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered_pC1X012',
 'nb_fp': 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.1/Calculate LTR_newdic_destressed_aligned_CM_filtered_LM_filtered observation distribution given channel models.ipynb'}

{'l': 'CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.01/LTR_CMU_destressed_aligned_CM_filtered_LM_filtered_p6Y0X01.json',
 'c': 'CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.01/LTR_CMU_destressed_aligned_CM_filtered_LM_filtered_pY1X0X1X2.json',
 'r': 'CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.01/LTR_CMU_destressed_aligned_CM_filtered_LM_filtered_p3Y1X01.json',
 'o': 'CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.01/LTR_CMU_destressed_aligned_CM_filtered_LM_filtered_pC1X012',
 'nb_fp': 'CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.01/Calculate LTR_CMU_destressed_aligned_CM_filtered_LM_filtered observation distribution given channel models.ipynb'}

{'l': 'CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.05/LTR_CMU_destressed_aligned_CM_filtered_LM_filtered_p6Y0X01.json',
 'c': 'CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.05/LTR_CMU_destressed_aligned_CM_filtered_LM_filtered_pY1X0X1X2.json',
 'r': 'CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.05/LTR_CMU_destressed_aligned_CM_filtered_LM_filtered_p3Y1X01.json',
 'o': 'CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.05/LTR_CMU_destressed_aligned_CM_filtered_LM_filtered_pC1X012',
 'nb_fp': 'CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.05/Calculate LTR_CMU_destressed_aligned_CM_filtered_LM_filtered observation distribution given channel models.ipynb'}

...

{'l': 'CM_AmE_destressed_aligned_w_LTR_Buckeye_pseudocount0.05/LTR_Buckeye_aligned_CM_filtered_LM_filtered_p6Y0X01.json',
 'c': 'CM_AmE_destressed_aligned_w_LTR_Buckeye_pseudocount0.05/LTR_Buckeye_aligned_CM_filtered_LM_filtered_pY1X0X1X2.json',
 'r': 'CM_AmE_destressed_aligned_w_LTR_Buckeye_pseudocount0.05/LTR_Buckeye_aligned_CM_filtered_LM_filtered_p3Y1X01.json',
 'o': 'CM_AmE_destressed_aligned_w_LTR_Buckeye_pseudocount0.05/LTR_Buckeye_aligned_CM_filtered_LM_filtered_pC1X012',
 'nb_fp': 'CM_AmE_destressed_aligned_w_LTR_Buckeye_pseudocount0.05/Calculate LTR_Buckeye_aligned_CM_filtered_LM_filtered observation distribution given channel models.ipynb'}

{'l': 'CM_AmE_destressed_aligned_w_LTR_Buckeye_pseudocount0.1/LTR_Buckeye_aligned_CM_filtered_LM_filtered_p6Y0X01.json',
 'c': 'CM_AmE_destressed_aligned_w_LTR_Buckeye_pseudocount0.1/LTR_Buckeye_aligned_CM_filtered_LM_filtered_pY1X0X1X2.json',
 'r': 'CM_AmE_destressed_aligned_w_LTR_Buckeye_pseudocount0.1/LTR_Buckeye_aligned_CM_filtered_LM_filtered_p3Y1X01.json',
 'o': 'CM_AmE_destressed_aligned_w_LTR_Buckeye_pseudocount0.1/LTR_Buckeye_aligned_CM_filtered_LM_filtered_pC1X012',
 'nb_fp': 'CM_AmE_destressed_aligned_w_LTR_Buckeye_pseudocount0.1/Calculate LTR_Buckeye_aligned_CM_filtered_LM_filtered observation distribution given channel models.ipynb'}

{'l': 'CM_AmE_destressed_aligned_w_LTR_NXT_swbd_destressed_pseudocount0.01/LTR_NXT_swbd_destressed_aligned_CM_filtered_LM_filtered_p6Y0X01.json',
 'c': 'CM_AmE_destressed_aligned_w_LTR_NXT_swbd_destressed_pseudocount0.01/LTR_NXT_swbd_destressed_aligned_CM_filtered_LM_filtered_pY1X0X1X2.json',
 'r': 'CM_AmE_destressed_aligned_w_LTR_NXT_swbd_destressed_pseudocount0.01/LTR_NXT_swbd_destressed_aligned_CM_filtered_LM_filtered_p3Y1X01.json',
 'o': 'CM_AmE_destressed_aligned_w_LTR_NXT_swbd_destressed_pseudocount0.01/LTR_NXT_swbd_destressed_aligned_CM_filtered_LM_filtered_pC1X012',
 'nb_fp': 'CM_AmE_destressed_aligned_w_LTR_NXT_swbd_destressed_pseudocount0.01/Calculate LTR_NXT_swbd_destressed_aligned_CM_filtered_LM_filtered observation distribution given channel models.ipynb'}

{'l': 'CM_AmE_destressed_aligned_w_LTR_NXT_swbd_destressed_pseudocount0.05/LTR_NXT_swbd_destressed_aligned_CM_filtered_LM_filtered_p6Y0X01.json',
 'c': 'CM_AmE_destressed_aligned_w_LTR_NXT_swbd_destressed_pseudocount0.05/LTR_NXT_swbd_destressed_aligned_CM_filtered_LM_filtered_pY1X0X1X2.json',
 'r': 'CM_AmE_destressed_aligned_w_LTR_NXT_swbd_destressed_pseudocount0.05/LTR_NXT_swbd_destressed_aligned_CM_filtered_LM_filtered_p3Y1X01.json',
 'o': 'CM_AmE_destressed_aligned_w_LTR_NXT_swbd_destressed_pseudocount0.05/LTR_NXT_swbd_destressed_aligned_CM_filtered_LM_filtered_pC1X012',
 'nb_fp': 'CM_AmE_destressed_aligned_w_LTR_NXT_swbd_destressed_pseudocount0.05/Calculate LTR_NXT_swbd_destressed_aligned_CM_filtered_LM_filtered observation distribution given channel models.ipynb'}

{'l': 'CM_AmE_destressed_aligned_w_LTR_NXT_swbd_destressed_pseudocount0.1/LTR_NXT_swbd_destressed_aligned_CM_filtered_LM_filtered_p6Y0X01.json',
 'c': 'CM_AmE_destressed_aligned_w_LTR_NXT_swbd_destressed_pseudocount0.1/LTR_NXT_swbd_destressed_aligned_CM_filtered_LM_filtered_pY1X0X1X2.json',
 'r': 'CM_AmE_destressed_aligned_w_LTR_NXT_swbd_destressed_pseudocount0.1/LTR_NXT_swbd_destressed_aligned_CM_filtered_LM_filtered_p3Y1X01.json',
 'o': 'CM_AmE_destressed_aligned_w_LTR_NXT_swbd_destressed_pseudocount0.1/LTR_NXT_swbd_destressed_aligned_CM_filtered_LM_filtered_pC1X012',
 'nb_fp': 'CM_AmE_destressed_aligned_w_LTR_NXT_swbd_destressed_pseudocount0.1/Calculate LTR_NXT_swbd_destressed_aligned_CM_filtered_LM_filtered observation distribution given channel models.ipynb'}

In [111]:
if '4d' in permittedSteps:
    # takes 5-6h on old sidious, iirc
    # takes 2h 40m on new sidious 
    
    # takes ≈100m on wittgenstein
    # (peak mem usage is ≈100+GB but <120GB on cmu)
    # ≈7.6m per newdic input
    # ≈11.3m per CMU input
    # ≈6m per Buckeye input
    # ≈8.2m per NXT_swbd input
    for bundle in observation_bundles:
        ensure_dir_exists(path.dirname(bundle['o']))

        if not overwrite and path.exists(bundle['nb_fp']):
#         if not overwrite and path.exists(path.join(ab['cm_dir'], ab['nb_output_name'])):
            print('{0} already exists. Skipping...'.format(path.join(bundle['nb_fp'])))
#             print('{0} already exists. Skipping...'.format(path.join(ab['cm_dir'], ab['nb_output_name'])))
            endNote()
            continue
        
        progress_report(bundle['nb_fp'],
                        dict(l = bundle['l'],
                             c = bundle['c'],
                             r = bundle['r'],
                             o = bundle['o']))

        nb = pm.execute_notebook(
        'Calculate observation distribution given channel models.ipynb',
        bundle['nb_fp'],
        parameters=dict(l = bundle['l'],
                        c = bundle['c'],
                        r = bundle['r'],
                        o = bundle['o'])
        )
        endNote()
        print('\n')

## Step 4e: Define channel distributions on a set of segmental wordforms(+prefixes)

**Dependencies**
 - **Step 3c**: `LTR_..._aligned_CM_filtered_LM_filtered_pY1X0X1X2.json`
 - **Step 3e**: `...pW_V.json`

In [112]:
# gather paired (....pW_V.json, ...pY1X0X1X2.json) p(W|V), p(Y_i|X_{i-1},X_i;X_{i+1}) distributions
# output complete wordform channel models into the channel model directory with the same prefix that's on the triphone channel distribution file

In [113]:
# def LTR_to_pW_Vs(LTR_fp):
#     matching_bundles = list(filter(lambda bundle: bundle['LTR_fp'] == LTR_fp,
#                                    pW_V_fp_bundles))
#     matching_pW_V_fps = set(map(lambda bundle: bundle['pW_V_fp'],
#                                 matching_bundles))
#     return matching_pW_V_fps

def LTR_to_TCMs(LTR_fp):
    matching_bundles = list(filter(lambda bundle: bundle['l'] == LTR_fp,
                                   trimmed_LTR_CM_triples))
    matching_TCM_fps = set(map(lambda bundle: bundle['o'],
                               matching_bundles))
    return matching_TCM_fps

def matched_pW_Vs_and_TCMs(LTR_fp):
    matching_pW_V_fps = LTR_to_pW_Vs(LTR_fp)
    matching_TCM_fps = LTR_to_TCMs(LTR_fp)
    return {'LW_V_fps':matching_pW_V_fps,
            'TCM_fps':matching_TCM_fps}

def get_matched_pairs(LTR_fp):
    matching_TCM_fps = LTR_to_TCMs(LTR_fp)
    matching_pW_V_fps = LTR_to_pW_Vs(LTR_fp)
    matched_pairs = set(product(matching_TCM_fps,
                                matching_pW_V_fps))
    return matched_pairs

my_LTR_fps = list(map(lambda bundle: bundle['LTR_fp'],
                      pW_V_fp_bundles))

LCM_bundles = []
for c,w in union(map(get_matched_pairs,
                     my_LTR_fps)):
#     if '_trim' in w:
#         print(f'Skipping w = {w}')
#         continue
    
    bundle = dict()
    bundle['c'] = c
    if '_trim' in w:
        bundle['b'] = c.split('pY1X0X1X2.json')[0] + 'pC1X012.npy'
    bundle['w'] = w + '.pW_V.json'
    output_dir = path.dirname(c)
    output_suffix = 'trim_' if '_trim' in w else ''
    output_prefix = path.basename(c).split('pY1X0X1X2.json')[0] + output_suffix
    bundle['o'] = path.join(output_dir, output_prefix)
    
    bundle['nb_fp'] = path.join(output_dir, f'Calculate wordform channel matrices for {path.basename(w)}.ipynb')
    
    bundle
    LCM_bundles.append(bundle)

{'c': 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.1/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered_pY1X0X1X2.json',
 'w': 'LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered.pW_V.json',
 'o': 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.1/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered_',
 'nb_fp': 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.1/Calculate wordform channel matrices for LTR_newdic_destressed_aligned_CM_filtered_LM_filtered.ipynb'}

{'c': 'CM_AmE_destressed_aligned_w_LTR_NXT_swbd_destressed_pseudocount0.1/LTR_NXT_swbd_destressed_aligned_CM_filtered_LM_filtered_pY1X0X1X2.json',
 'b': 'CM_AmE_destressed_aligned_w_LTR_NXT_swbd_destressed_pseudocount0.1/LTR_NXT_swbd_destressed_aligned_CM_filtered_LM_filtered_pC1X012.npy',
 'w': 'LTR_NXT_swbd_destressed_aligned_w_GD_AmE_destressed/LTR_NXT_swbd_destressed_aligned_CM_filtered_LM_filtered_trim.pW_V.json',
 'o': 'CM_AmE_destressed_aligned_w_LTR_NXT_swbd_destressed_pseudocount0.1/LTR_NXT_swbd_destressed_aligned_CM_filtered_LM_filtered_trim_',
 'nb_fp': 'CM_AmE_destressed_aligned_w_LTR_NXT_swbd_destressed_pseudocount0.1/Calculate wordform channel matrices for LTR_NXT_swbd_destressed_aligned_CM_filtered_LM_filtered_trim.ipynb'}

{'c': 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.01/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered_pY1X0X1X2.json',
 'b': 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.01/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered_pC1X012.npy',
 'w': 'LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered_trim.pW_V.json',
 'o': 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.01/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered_trim_',
 'nb_fp': 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.01/Calculate wordform channel matrices for LTR_newdic_destressed_aligned_CM_filtered_LM_filtered_trim.ipynb'}

{'c': 'CM_AmE_destressed_aligned_w_LTR_NXT_swbd_destressed_pseudocount0.01/LTR_NXT_swbd_destressed_aligned_CM_filtered_LM_filtered_pY1X0X1X2.json',
 'w': 'LTR_NXT_swbd_destressed_aligned_w_GD_AmE_destressed/LTR_NXT_swbd_destressed_aligned_CM_filtered_LM_filtered.pW_V.json',
 'o': 'CM_AmE_destressed_aligned_w_LTR_NXT_swbd_destressed_pseudocount0.01/LTR_NXT_swbd_destressed_aligned_CM_filtered_LM_filtered_',
 'nb_fp': 'CM_AmE_destressed_aligned_w_LTR_NXT_swbd_destressed_pseudocount0.01/Calculate wordform channel matrices for LTR_NXT_swbd_destressed_aligned_CM_filtered_LM_filtered.ipynb'}

{'c': 'CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.1/LTR_CMU_destressed_aligned_CM_filtered_LM_filtered_pY1X0X1X2.json',
 'w': 'LTR_CMU_destressed_aligned_w_GD_AmE_destressed/LTR_CMU_destressed_aligned_CM_filtered_LM_filtered.pW_V.json',
 'o': 'CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.1/LTR_CMU_destressed_aligned_CM_filtered_LM_filtered_',
 'nb_fp': 'CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.1/Calculate wordform channel matrices for LTR_CMU_destressed_aligned_CM_filtered_LM_filtered.ipynb'}

...

{'c': 'CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.1/LTR_CMU_destressed_aligned_CM_filtered_LM_filtered_pY1X0X1X2.json',
 'b': 'CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.1/LTR_CMU_destressed_aligned_CM_filtered_LM_filtered_pC1X012.npy',
 'w': 'LTR_CMU_destressed_aligned_w_GD_AmE_destressed/LTR_CMU_destressed_aligned_CM_filtered_LM_filtered_trim.pW_V.json',
 'o': 'CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.1/LTR_CMU_destressed_aligned_CM_filtered_LM_filtered_trim_',
 'nb_fp': 'CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.1/Calculate wordform channel matrices for LTR_CMU_destressed_aligned_CM_filtered_LM_filtered_trim.ipynb'}

{'c': 'CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.05/LTR_CMU_destressed_aligned_CM_filtered_LM_filtered_pY1X0X1X2.json',
 'b': 'CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.05/LTR_CMU_destressed_aligned_CM_filtered_LM_filtered_pC1X012.npy',
 'w': 'LTR_CMU_destressed_aligned_w_GD_AmE_destressed/LTR_CMU_destressed_aligned_CM_filtered_LM_filtered_trim.pW_V.json',
 'o': 'CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.05/LTR_CMU_destressed_aligned_CM_filtered_LM_filtered_trim_',
 'nb_fp': 'CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.05/Calculate wordform channel matrices for LTR_CMU_destressed_aligned_CM_filtered_LM_filtered_trim.ipynb'}

{'c': 'CM_AmE_destressed_aligned_w_LTR_Buckeye_pseudocount0.05/LTR_Buckeye_aligned_CM_filtered_LM_filtered_pY1X0X1X2.json',
 'w': 'LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_CM_filtered_LM_filtered.pW_V.json',
 'o': 'CM_AmE_destressed_aligned_w_LTR_Buckeye_pseudocount0.05/LTR_Buckeye_aligned_CM_filtered_LM_filtered_',
 'nb_fp': 'CM_AmE_destressed_aligned_w_LTR_Buckeye_pseudocount0.05/Calculate wordform channel matrices for LTR_Buckeye_aligned_CM_filtered_LM_filtered.ipynb'}

{'c': 'CM_AmE_destressed_aligned_w_LTR_Buckeye_pseudocount0.05/LTR_Buckeye_aligned_CM_filtered_LM_filtered_pY1X0X1X2.json',
 'b': 'CM_AmE_destressed_aligned_w_LTR_Buckeye_pseudocount0.05/LTR_Buckeye_aligned_CM_filtered_LM_filtered_pC1X012.npy',
 'w': 'LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_CM_filtered_LM_filtered_trim.pW_V.json',
 'o': 'CM_AmE_destressed_aligned_w_LTR_Buckeye_pseudocount0.05/LTR_Buckeye_aligned_CM_filtered_LM_filtered_trim_',
 'nb_fp': 'CM_AmE_destressed_aligned_w_LTR_Buckeye_pseudocount0.05/Calculate wordform channel matrices for LTR_Buckeye_aligned_CM_filtered_LM_filtered_trim.ipynb'}

{'c': 'CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.01/LTR_CMU_destressed_aligned_CM_filtered_LM_filtered_pY1X0X1X2.json',
 'w': 'LTR_CMU_destressed_aligned_w_GD_AmE_destressed/LTR_CMU_destressed_aligned_CM_filtered_LM_filtered.pW_V.json',
 'o': 'CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.01/LTR_CMU_destressed_aligned_CM_filtered_LM_filtered_',
 'nb_fp': 'CM_AmE_destressed_aligned_w_LTR_CMU_destressed_pseudocount0.01/Calculate wordform channel matrices for LTR_CMU_destressed_aligned_CM_filtered_LM_filtered.ipynb'}

In [114]:
skip_trim

False

In [115]:
# skip_trim = False

if '4e' in permittedSteps:
    #old runtime info
    # takes ~2m on wittgenstein with no background load and no .npz files
    # takes ~12m on wittgenstein with no background load *and* .npz files
    # takes ~15m on (new) sidious with no background load *and* .npz files *and* assertion checking
    # takes ~20m on (new) sidious with no background load *and* .npz files *and* assertion checking *and* 'exact wordform' calculations
    
    #current runtime info
    # ? on untrimmed inputs (wittgenstein)
    #   ≈?m on newdic
    #   ≈?m on Buckeye
    #   ≈?m on CMU
    #   ≈?m on NXT_swbd
    # ≈160m on trimmed inputs (tarski)
    #   ≈10m on newdic inputs
    #   ≈6m on Buckeye inputs
    #   ≈43m on CMU inputs
    #   ≈14m on NXT_swbd inputs
    for bundle in LCM_bundles:
        ensure_dir_exists(path.dirname(bundle['o']))

        if '_trim' in bundle['w'] and skip_trim:
            bundle_w = bundle['w']
            print(f'Skipping bundle containing w = {bundle_w}')
            continue
        
        if not overwrite and path.exists(bundle['nb_fp']):
#         if not overwrite and path.exists(path.join(ab['cm_dir'], ab['nb_output_name'])):
            print('{0} already exists. Skipping...'.format(path.join(bundle['nb_fp'])))
#             print('{0} already exists. Skipping...'.format(path.join(ab['cm_dir'], ab['nb_output_name'])))
            endNote()
            continue
        
        
        if '_trim' in bundle['w']:
            progress_report(bundle['nb_fp'], 
                            dict(c = bundle['c'],
                                 b = bundle['b'],
                                 w = bundle['w'],
                                 o = bundle['o']))
            nb = pm.execute_notebook(
                'Calculate segmental wordform and prefix channel matrices - OD.ipynb',
                bundle['nb_fp'],
                parameters=dict(c = bundle['c'],
                                b = bundle['b'],
                                w = bundle['w'],
                                o = bundle['o'])
            )
        else:
            progress_report(bundle['nb_fp'], 
                            dict(c = bundle['c'],
                                 w = bundle['w'],
                                 o = bundle['o']))
            nb = pm.execute_notebook(
                'Calculate segmental wordform and prefix channel matrices.ipynb',
                bundle['nb_fp'],
                parameters=dict(c = bundle['c'],
                                w = bundle['w'],
                                o = bundle['o'])
            )
        endNote()
        print('\n')

# Step 5: Calculate posterior probabilities

## Step 5a: Calculate $p(V|W, C)$

**Dependencies**
 - **Step 4c**: `pW_C` matrix
 - **Step 3e**: `pW_V` matrix
 - **Step 3d**: `LD_fisher_vocab_in...contexts_projected_LTR...pV_C.npy` matrix

$p(\hat{V} = v^*|\hat{X}_0^f = x_0^{'f}, c) = \frac{p(x_0^{'f}|v^*)p(v^*|c)}{p(x_0^{'f}|c)}$ 

In [116]:
#gather aligned triples of filepaths defining
# - $p(V|C)$
# - $p(W|C)$
# - $p(W|V)$
#plus associated (and crucially appropriately ordered!) metadata detailing 
# - C
# - V
# - W
# - the mapping between V and W
# construct the output filename and location (probably in LD?)
# construct output notebook filepaths

In [120]:
posterior_WD_bundles = []
for bundle in WD_bundles:
    output_dir = path.dirname(bundle['d'])
    LTR_key = path.basename(bundle['w']).split('.pW_V.npz')[0]
    contexts_key = path.basename(bundle['o']).split('LM_filtered')[1].split('.pW_C')[0].replace('_', ' ')
    output_base_prefix = LTR_key.replace('_trim', '') + contexts_key.replace(' ', '_') + '.pV_WC'
    
    
    new_bundle = dict()
    new_bundle['d'] = bundle['d']          #p(V|C) as .npy
    new_bundle['w'] = bundle['w']          #p(W|V) as .npz
    new_bundle['m'] = bundle['o'] + '.npy' #p(W|C) as .npy
    
    # c = arg pointing to file specifying C
#     LM_dir = path.dirname(new_bundle['d'])
#     LM_name = '_'.join(LM_dir.lower().split('_')[-2:])
#     contexts_ext = '.txt'
#     contexts_fn = 'LM_filtered_' + LM_name + contexts_ext
#     contexts_fp = path.join(LM_dir, contexts_fn)
#     new_bundle['c'] = contexts_fp
    
#     # v = arg pointing to file specifying V
#     # l = arg pointing to file specifying W
#     vlt_prefix = bundle['w'].split('.pW_V.npz')[0]
#     vocabulary_fp = vlt_prefix + '_Orthographic_Wordforms' + '.txt'
#     lexicon_fp = vlt_prefix + '_Transcriptions' + '.txt'
#     LTR_fp = vlt_prefix + '.tsv'
#     new_bundle['v'] = vocabulary_fp
#     new_bundle['l'] = lexicon_fp
#     new_bundle['t'] = LTR_fp
    
    new_bundle['o'] = path.join(output_dir, output_base_prefix)
    new_bundle['nb_fp'] = path.join(output_dir, f'Calculate orthographic posterior given segmental wordform + context for {LTR_key.replace("_trim", "")}{contexts_key}' + '.ipynb')
    new_bundle
    posterior_WD_bundles.append(new_bundle)
    print('\n')

{'d': 'LD_Fisher_vocab_in_NXT_swbd_preceding_contexts_2gram_model/LD_Fisher_vocab_in_NXT_swbd_preceding_contexts_2gram_model_projected_LTR_NXT_swbd_destressed_aligned_CM_filtered_LM_filtered.pV_C.npy',
 'w': 'LTR_NXT_swbd_destressed_aligned_w_GD_AmE_destressed/LTR_NXT_swbd_destressed_aligned_CM_filtered_LM_filtered_trim.pW_V.npz',
 'm': 'LD_Fisher_vocab_in_NXT_swbd_preceding_contexts_2gram_model/LTR_NXT_swbd_destressed_aligned_CM_filtered_LM_filtered_trim_in_NXT_swbd_preceding_contexts_2gram_model.pW_C.npy',
 'o': 'LD_Fisher_vocab_in_NXT_swbd_preceding_contexts_2gram_model/LTR_NXT_swbd_destressed_aligned_CM_filtered_LM_filtered_trim_in_NXT_swbd_preceding_contexts_2gram_model.pV_WC',
 'nb_fp': 'LD_Fisher_vocab_in_NXT_swbd_preceding_contexts_2gram_model/Calculate orthographic posterior given segmental wordform + context for LTR_NXT_swbd_destressed_aligned_CM_filtered_LM_filtered trim in NXT swbd preceding contexts 2gram model.ipynb'}

{'d': 'LD_Fisher_vocab_in_Buckeye_preceding_contexts_2gram_model/LD_Fisher_vocab_in_Buckeye_preceding_contexts_2gram_model_projected_LTR_Buckeye_aligned_CM_filtered_LM_filtered.pV_C.npy',
 'w': 'LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_CM_filtered_LM_filtered_trim.pW_V.npz',
 'm': 'LD_Fisher_vocab_in_Buckeye_preceding_contexts_2gram_model/LTR_Buckeye_aligned_CM_filtered_LM_filtered_trim_in_Buckeye_preceding_contexts_2gram_model.pW_C.npy',
 'o': 'LD_Fisher_vocab_in_Buckeye_preceding_contexts_2gram_model/LTR_Buckeye_aligned_CM_filtered_LM_filtered_trim_in_Buckeye_preceding_contexts_2gram_model.pV_WC',
 'nb_fp': 'LD_Fisher_vocab_in_Buckeye_preceding_contexts_2gram_model/Calculate orthographic posterior given segmental wordform + context for LTR_Buckeye_aligned_CM_filtered_LM_filtered trim in Buckeye preceding contexts 2gram model.ipynb'}

{'d': 'LD_Fisher_vocab_in_NXT_swbd_following_contexts_5gram_model/LD_Fisher_vocab_in_NXT_swbd_following_contexts_5gram_model_projected_LTR_NXT_swbd_destressed_aligned_CM_filtered_LM_filtered.pV_C.npy',
 'w': 'LTR_NXT_swbd_destressed_aligned_w_GD_AmE_destressed/LTR_NXT_swbd_destressed_aligned_CM_filtered_LM_filtered_trim.pW_V.npz',
 'm': 'LD_Fisher_vocab_in_NXT_swbd_following_contexts_5gram_model/LTR_NXT_swbd_destressed_aligned_CM_filtered_LM_filtered_trim_in_NXT_swbd_following_contexts_5gram_model.pW_C.npy',
 'o': 'LD_Fisher_vocab_in_NXT_swbd_following_contexts_5gram_model/LTR_NXT_swbd_destressed_aligned_CM_filtered_LM_filtered_trim_in_NXT_swbd_following_contexts_5gram_model.pV_WC',
 'nb_fp': 'LD_Fisher_vocab_in_NXT_swbd_following_contexts_5gram_model/Calculate orthographic posterior given segmental wordform + context for LTR_NXT_swbd_destressed_aligned_CM_filtered_LM_filtered trim in NXT swbd following contexts 5gram model.ipynb'}

{'d': 'LD_Fisher_vocab_in_NXT_swbd_following_contexts_3gram_model/LD_Fisher_vocab_in_NXT_swbd_following_contexts_3gram_model_projected_LTR_NXT_swbd_destressed_aligned_CM_filtered_LM_filtered.pV_C.npy',
 'w': 'LTR_NXT_swbd_destressed_aligned_w_GD_AmE_destressed/LTR_NXT_swbd_destressed_aligned_CM_filtered_LM_filtered.pW_V.npz',
 'm': 'LD_Fisher_vocab_in_NXT_swbd_following_contexts_3gram_model/LTR_NXT_swbd_destressed_aligned_CM_filtered_LM_filtered_in_NXT_swbd_following_contexts_3gram_model.pW_C.npy',
 'o': 'LD_Fisher_vocab_in_NXT_swbd_following_contexts_3gram_model/LTR_NXT_swbd_destressed_aligned_CM_filtered_LM_filtered_in_NXT_swbd_following_contexts_3gram_model.pV_WC',
 'nb_fp': 'LD_Fisher_vocab_in_NXT_swbd_following_contexts_3gram_model/Calculate orthographic posterior given segmental wordform + context for LTR_NXT_swbd_destressed_aligned_CM_filtered_LM_filtered in NXT swbd following contexts 3gram model.ipynb'}

{'d': 'LD_Fisher_vocab_in_NXT_swbd_following_contexts_3gram_model/LD_Fisher_vocab_in_NXT_swbd_following_contexts_3gram_model_projected_LTR_NXT_swbd_destressed_aligned_CM_filtered_LM_filtered.pV_C.npy',
 'w': 'LTR_NXT_swbd_destressed_aligned_w_GD_AmE_destressed/LTR_NXT_swbd_destressed_aligned_CM_filtered_LM_filtered_trim.pW_V.npz',
 'm': 'LD_Fisher_vocab_in_NXT_swbd_following_contexts_3gram_model/LTR_NXT_swbd_destressed_aligned_CM_filtered_LM_filtered_trim_in_NXT_swbd_following_contexts_3gram_model.pW_C.npy',
 'o': 'LD_Fisher_vocab_in_NXT_swbd_following_contexts_3gram_model/LTR_NXT_swbd_destressed_aligned_CM_filtered_LM_filtered_trim_in_NXT_swbd_following_contexts_3gram_model.pV_WC',
 'nb_fp': 'LD_Fisher_vocab_in_NXT_swbd_following_contexts_3gram_model/Calculate orthographic posterior given segmental wordform + context for LTR_NXT_swbd_destressed_aligned_CM_filtered_LM_filtered trim in NXT swbd following contexts 3gram model.ipynb'}

...

{'d': 'LD_Fisher_vocab_in_Buckeye_following_contexts_3gram_model/LD_Fisher_vocab_in_Buckeye_following_contexts_3gram_model_projected_LTR_Buckeye_aligned_CM_filtered_LM_filtered.pV_C.npy',
 'w': 'LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_CM_filtered_LM_filtered_trim.pW_V.npz',
 'm': 'LD_Fisher_vocab_in_Buckeye_following_contexts_3gram_model/LTR_Buckeye_aligned_CM_filtered_LM_filtered_trim_in_Buckeye_following_contexts_3gram_model.pW_C.npy',
 'o': 'LD_Fisher_vocab_in_Buckeye_following_contexts_3gram_model/LTR_Buckeye_aligned_CM_filtered_LM_filtered_trim_in_Buckeye_following_contexts_3gram_model.pV_WC',
 'nb_fp': 'LD_Fisher_vocab_in_Buckeye_following_contexts_3gram_model/Calculate orthographic posterior given segmental wordform + context for LTR_Buckeye_aligned_CM_filtered_LM_filtered trim in Buckeye following contexts 3gram model.ipynb'}

{'d': 'LD_Fisher_vocab_in_NXT_swbd_preceding_contexts_5gram_model/LD_Fisher_vocab_in_NXT_swbd_preceding_contexts_5gram_model_projected_LTR_NXT_swbd_destressed_aligned_CM_filtered_LM_filtered.pV_C.npy',
 'w': 'LTR_NXT_swbd_destressed_aligned_w_GD_AmE_destressed/LTR_NXT_swbd_destressed_aligned_CM_filtered_LM_filtered_trim.pW_V.npz',
 'm': 'LD_Fisher_vocab_in_NXT_swbd_preceding_contexts_5gram_model/LTR_NXT_swbd_destressed_aligned_CM_filtered_LM_filtered_trim_in_NXT_swbd_preceding_contexts_5gram_model.pW_C.npy',
 'o': 'LD_Fisher_vocab_in_NXT_swbd_preceding_contexts_5gram_model/LTR_NXT_swbd_destressed_aligned_CM_filtered_LM_filtered_trim_in_NXT_swbd_preceding_contexts_5gram_model.pV_WC',
 'nb_fp': 'LD_Fisher_vocab_in_NXT_swbd_preceding_contexts_5gram_model/Calculate orthographic posterior given segmental wordform + context for LTR_NXT_swbd_destressed_aligned_CM_filtered_LM_filtered trim in NXT swbd preceding contexts 5gram model.ipynb'}

{'d': 'LD_Fisher_vocab_in_Buckeye_preceding_contexts_4gram_model/LD_Fisher_vocab_in_Buckeye_preceding_contexts_4gram_model_projected_LTR_Buckeye_aligned_CM_filtered_LM_filtered.pV_C.npy',
 'w': 'LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_CM_filtered_LM_filtered.pW_V.npz',
 'm': 'LD_Fisher_vocab_in_Buckeye_preceding_contexts_4gram_model/LTR_Buckeye_aligned_CM_filtered_LM_filtered_in_Buckeye_preceding_contexts_4gram_model.pW_C.npy',
 'o': 'LD_Fisher_vocab_in_Buckeye_preceding_contexts_4gram_model/LTR_Buckeye_aligned_CM_filtered_LM_filtered_in_Buckeye_preceding_contexts_4gram_model.pV_WC',
 'nb_fp': 'LD_Fisher_vocab_in_Buckeye_preceding_contexts_4gram_model/Calculate orthographic posterior given segmental wordform + context for LTR_Buckeye_aligned_CM_filtered_LM_filtered in Buckeye preceding contexts 4gram model.ipynb'}

{'d': 'LD_Fisher_vocab_in_Buckeye_following_contexts_3gram_model/LD_Fisher_vocab_in_Buckeye_following_contexts_3gram_model_projected_LTR_Buckeye_aligned_CM_filtered_LM_filtered.pV_C.npy',
 'w': 'LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_CM_filtered_LM_filtered.pW_V.npz',
 'm': 'LD_Fisher_vocab_in_Buckeye_following_contexts_3gram_model/LTR_Buckeye_aligned_CM_filtered_LM_filtered_in_Buckeye_following_contexts_3gram_model.pW_C.npy',
 'o': 'LD_Fisher_vocab_in_Buckeye_following_contexts_3gram_model/LTR_Buckeye_aligned_CM_filtered_LM_filtered_in_Buckeye_following_contexts_3gram_model.pV_WC',
 'nb_fp': 'LD_Fisher_vocab_in_Buckeye_following_contexts_3gram_model/Calculate orthographic posterior given segmental wordform + context for LTR_Buckeye_aligned_CM_filtered_LM_filtered in Buckeye following contexts 3gram model.ipynb'}

{'d': 'LD_Fisher_vocab_in_Buckeye_following_contexts_5gram_model/LD_Fisher_vocab_in_Buckeye_following_contexts_5gram_model_projected_LTR_Buckeye_aligned_CM_filtered_LM_filtered.pV_C.npy',
 'w': 'LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_CM_filtered_LM_filtered_trim.pW_V.npz',
 'm': 'LD_Fisher_vocab_in_Buckeye_following_contexts_5gram_model/LTR_Buckeye_aligned_CM_filtered_LM_filtered_trim_in_Buckeye_following_contexts_5gram_model.pW_C.npy',
 'o': 'LD_Fisher_vocab_in_Buckeye_following_contexts_5gram_model/LTR_Buckeye_aligned_CM_filtered_LM_filtered_trim_in_Buckeye_following_contexts_5gram_model.pV_WC',
 'nb_fp': 'LD_Fisher_vocab_in_Buckeye_following_contexts_5gram_model/Calculate orthographic posterior given segmental wordform + context for LTR_Buckeye_aligned_CM_filtered_LM_filtered trim in Buckeye following contexts 5gram model.ipynb'}

In [120]:
#NB for reasons I haven't tried to figure out, this cell dumps a bunch of notebook metadata
# into the cell output...

if '5a' in permittedSteps:
    # takes ~360m for old sidious w/ no background load and J=-1
    
    # takes 135m for wittgenstein w/ no background load and J=-1
    for bundle in posterior_WD_bundles:
        output_dir = path.dirname(bundle['o'])
        ensure_dir_exists(output_dir)

        if not overwrite and path.exists(bundle['nb_fp']):
#         if not overwrite and path.exists(path.join(ab['cm_dir'], ab['nb_output_name'])):
            print('{0} already exists. Skipping...'.format(path.join(bundle['nb_fp'])))
#             print('{0} already exists. Skipping...'.format(path.join(ab['cm_dir'], ab['nb_output_name'])))
            endNote()
            continue
        
        progress_report(bundle['nb_fp'],
                        dict(d = bundle['d'],
                             w = bundle['w'],
                             m = bundle['m'],
                             o = bundle['o']))

        pm.execute_notebook(
            'Calculate orthographic posterior given segmental wordform + context.ipynb',
            bundle['nb_fp'],
            parameters=dict(d = bundle['d'],
                            w = bundle['w'],
                            m = bundle['m'],
#                             c = bundle['c'],
#                             v = bundle['v'],
#                             l = bundle['l'],
#                             t = bundle['t'],
                            o = bundle['o'])
        )
        endNote()
        print('\n')

Start  @ 21:05:13
Running notebook:
	Calculate orthographic posterior given segmental wordform + context for LTR_Buckeye_aligned_CM_filtered_LM_filtered in Buckeye preceding contexts 3gram model.ipynb
Output directory:
	LD_Fisher_vocab_in_Buckeye_preceding_contexts_3gram_model
Arguments:
{
 "d": "LD_Fisher_vocab_in_Buckeye_preceding_contexts_3gram_model/LD_Fisher_vocab_in_Buckeye_preceding_contexts_3gram_model_projected_LTR_Buckeye_aligned_CM_filtered_LM_filtered.pV_C.npy",
 "w": "LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_CM_filtered_LM_filtered.pW_V.npz",
 "m": "LD_Fisher_vocab_in_Buckeye_preceding_contexts_3gram_model/LTR_Buckeye_aligned_CM_filtered_LM_filtered_in_Buckeye_preceding_contexts_3gram_model.pW_C.npy",
 "o": "LD_Fisher_vocab_in_Buckeye_preceding_contexts_3gram_model/LTR_Buckeye_aligned_CM_filtered_LM_filtered_in_Buckeye_preceding_contexts_3gram_model.pV_WC"
}


{'cells': [{'cell_type': 'code',
   'execution_count': 1,
   'metadata': {'ExecuteTime': {'end_time': '2019-07-16T19:53:06.337260Z',
     'start_time': '2019-07-16T19:53:06.311888Z'},
    'tags': [],
    'papermill': {'exception': False,
     'start_time': '2019-09-12T04:05:14.858483',
     'end_time': '2019-09-12T04:05:14.900856',
     'duration': 0.042373,
     'status': 'completed'}},
   'outputs': [],
   'source': '#Prints **all** console output, not just last item in cell \nfrom IPython.core.interactiveshell import InteractiveShell\nInteractiveShell.ast_node_interactivity = "all"'},
  {'cell_type': 'markdown',
   'metadata': {'tags': [],
    'papermill': {'exception': False,
     'start_time': '2019-09-12T04:05:14.930573',
     'end_time': '2019-09-12T04:05:14.959725',
     'duration': 0.029152,
     'status': 'completed'}},
   'source': '**Eric Meinhardt / emeinhardt@ucsd.edu**'},
  {'cell_type': 'markdown',
   'metadata': {'toc': True,
    'tags': [],
    'papermill': {'exceptio

End  @ 21:07:33


Start  @ 21:07:33
Running notebook:
	Calculate orthographic posterior given segmental wordform + context for LTR_Buckeye_aligned_CM_filtered_LM_filtered in Buckeye following contexts 4gram model.ipynb
Output directory:
	LD_Fisher_vocab_in_Buckeye_following_contexts_4gram_model
Arguments:
{
 "d": "LD_Fisher_vocab_in_Buckeye_following_contexts_4gram_model/LD_Fisher_vocab_in_Buckeye_following_contexts_4gram_model_projected_LTR_Buckeye_aligned_CM_filtered_LM_filtered.pV_C.npy",
 "w": "LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_CM_filtered_LM_filtered.pW_V.npz",
 "m": "LD_Fisher_vocab_in_Buckeye_following_contexts_4gram_model/LTR_Buckeye_aligned_CM_filtered_LM_filtered_in_Buckeye_following_contexts_4gram_model.pW_C.npy",
 "o": "LD_Fisher_vocab_in_Buckeye_following_contexts_4gram_model/LTR_Buckeye_aligned_CM_filtered_LM_filtered_in_Buckeye_following_contexts_4gram_model.pV_WC"
}


{'cells': [{'cell_type': 'code',
   'execution_count': 1,
   'metadata': {'ExecuteTime': {'end_time': '2019-07-16T19:53:06.337260Z',
     'start_time': '2019-07-16T19:53:06.311888Z'},
    'tags': [],
    'papermill': {'exception': False,
     'start_time': '2019-09-12T04:07:34.201460',
     'end_time': '2019-09-12T04:07:34.242563',
     'duration': 0.041103,
     'status': 'completed'}},
   'outputs': [],
   'source': '#Prints **all** console output, not just last item in cell \nfrom IPython.core.interactiveshell import InteractiveShell\nInteractiveShell.ast_node_interactivity = "all"'},
  {'cell_type': 'markdown',
   'metadata': {'tags': [],
    'papermill': {'exception': False,
     'start_time': '2019-09-12T04:07:34.275628',
     'end_time': '2019-09-12T04:07:34.304831',
     'duration': 0.029203,
     'status': 'completed'}},
   'source': '**Eric Meinhardt / emeinhardt@ucsd.edu**'},
  {'cell_type': 'markdown',
   'metadata': {'toc': True,
    'tags': [],
    'papermill': {'exceptio

End  @ 21:12:07


Start  @ 21:12:07
Running notebook:
	Calculate orthographic posterior given segmental wordform + context for LTR_Buckeye_aligned_CM_filtered_LM_filtered in Buckeye preceding contexts 5gram model.ipynb
Output directory:
	LD_Fisher_vocab_in_Buckeye_preceding_contexts_5gram_model
Arguments:
{
 "d": "LD_Fisher_vocab_in_Buckeye_preceding_contexts_5gram_model/LD_Fisher_vocab_in_Buckeye_preceding_contexts_5gram_model_projected_LTR_Buckeye_aligned_CM_filtered_LM_filtered.pV_C.npy",
 "w": "LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_CM_filtered_LM_filtered.pW_V.npz",
 "m": "LD_Fisher_vocab_in_Buckeye_preceding_contexts_5gram_model/LTR_Buckeye_aligned_CM_filtered_LM_filtered_in_Buckeye_preceding_contexts_5gram_model.pW_C.npy",
 "o": "LD_Fisher_vocab_in_Buckeye_preceding_contexts_5gram_model/LTR_Buckeye_aligned_CM_filtered_LM_filtered_in_Buckeye_preceding_contexts_5gram_model.pV_WC"
}


...

End  @ 22:47:51


Start  @ 22:47:51
Running notebook:
	Calculate orthographic posterior given segmental wordform + context for LTR_NXT_swbd_destressed_aligned_CM_filtered_LM_filtered in NXT swbd preceding contexts 3gram model.ipynb
Output directory:
	LD_Fisher_vocab_in_NXT_swbd_preceding_contexts_3gram_model
Arguments:
{
 "d": "LD_Fisher_vocab_in_NXT_swbd_preceding_contexts_3gram_model/LD_Fisher_vocab_in_NXT_swbd_preceding_contexts_3gram_model_projected_LTR_NXT_swbd_destressed_aligned_CM_filtered_LM_filtered.pV_C.npy",
 "w": "LTR_NXT_swbd_destressed_aligned_w_GD_AmE_destressed/LTR_NXT_swbd_destressed_aligned_CM_filtered_LM_filtered.pW_V.npz",
 "m": "LD_Fisher_vocab_in_NXT_swbd_preceding_contexts_3gram_model/LTR_NXT_swbd_destressed_aligned_CM_filtered_LM_filtered_in_NXT_swbd_preceding_contexts_3gram_model.pW_C.npy",
 "o": "LD_Fisher_vocab_in_NXT_swbd_preceding_contexts_3gram_model/LTR_NXT_swbd_destressed_aligned_CM_filtered_LM_filtered_in_NXT_swbd_preceding_contexts_3gram_model.pV_WC"
}

{'cells': [{'cell_type': 'code',
   'execution_count': 1,
   'metadata': {'ExecuteTime': {'end_time': '2019-07-16T19:53:06.337260Z',
     'start_time': '2019-07-16T19:53:06.311888Z'},
    'tags': [],
    'papermill': {'exception': False,
     'start_time': '2019-09-12T05:47:52.270509',
     'end_time': '2019-09-12T05:47:52.314869',
     'duration': 0.04436,
     'status': 'completed'}},
   'outputs': [],
   'source': '#Prints **all** console output, not just last item in cell \nfrom IPython.core.interactiveshell import InteractiveShell\nInteractiveShell.ast_node_interactivity = "all"'},
  {'cell_type': 'markdown',
   'metadata': {'tags': [],
    'papermill': {'exception': False,
     'start_time': '2019-09-12T05:47:52.353304',
     'end_time': '2019-09-12T05:47:52.383820',
     'duration': 0.030516,
     'status': 'completed'}},
   'source': '**Eric Meinhardt / emeinhardt@ucsd.edu**'},
  {'cell_type': 'markdown',
   'metadata': {'toc': True,
    'tags': [],
    'papermill': {'exception

End  @ 22:56:31


Start  @ 22:56:31
Running notebook:
	Calculate orthographic posterior given segmental wordform + context for LTR_NXT_swbd_destressed_aligned_CM_filtered_LM_filtered in NXT swbd preceding contexts 5gram model.ipynb
Output directory:
	LD_Fisher_vocab_in_NXT_swbd_preceding_contexts_5gram_model
Arguments:
{
 "d": "LD_Fisher_vocab_in_NXT_swbd_preceding_contexts_5gram_model/LD_Fisher_vocab_in_NXT_swbd_preceding_contexts_5gram_model_projected_LTR_NXT_swbd_destressed_aligned_CM_filtered_LM_filtered.pV_C.npy",
 "w": "LTR_NXT_swbd_destressed_aligned_w_GD_AmE_destressed/LTR_NXT_swbd_destressed_aligned_CM_filtered_LM_filtered.pW_V.npz",
 "m": "LD_Fisher_vocab_in_NXT_swbd_preceding_contexts_5gram_model/LTR_NXT_swbd_destressed_aligned_CM_filtered_LM_filtered_in_NXT_swbd_preceding_contexts_5gram_model.pW_C.npy",
 "o": "LD_Fisher_vocab_in_NXT_swbd_preceding_contexts_5gram_model/LTR_NXT_swbd_destressed_aligned_CM_filtered_LM_filtered_in_NXT_swbd_preceding_contexts_5gram_model.pV_WC"
}

{'cells': [{'cell_type': 'code',
   'execution_count': 1,
   'metadata': {'ExecuteTime': {'end_time': '2019-07-16T19:53:06.337260Z',
     'start_time': '2019-07-16T19:53:06.311888Z'},
    'tags': [],
    'papermill': {'exception': False,
     'start_time': '2019-09-12T05:56:32.890463',
     'end_time': '2019-09-12T05:56:32.933751',
     'duration': 0.043288,
     'status': 'completed'}},
   'outputs': [],
   'source': '#Prints **all** console output, not just last item in cell \nfrom IPython.core.interactiveshell import InteractiveShell\nInteractiveShell.ast_node_interactivity = "all"'},
  {'cell_type': 'markdown',
   'metadata': {'tags': [],
    'papermill': {'exception': False,
     'start_time': '2019-09-12T05:56:32.964148',
     'end_time': '2019-09-12T05:56:32.993635',
     'duration': 0.029487,
     'status': 'completed'}},
   'source': '**Eric Meinhardt / emeinhardt@ucsd.edu**'},
  {'cell_type': 'markdown',
   'metadata': {'toc': True,
    'tags': [],
    'papermill': {'exceptio

End  @ 23:19:48




## Step 5b: Calculate $p(\hat{X}_0^f|X_0^f, C)$

**Dependencies**
 - **Step 4e**: `CM_AmE_destressed_aligned_w_LTR_..._pseudocount0.01/LTR_..._aligned_CM_filtered_LM_filtered_CMs_by_length_by_prefix_index.pickle` list of matrices
 - **Step 4c**: `LD_Fisher_vocab_in_..._contexts/LTR_..._aligned_CM_filtered_LM_filtered_in_..._contexts.pW_C.npy` matrix
 - **Step ?**: `LTR_..._aligned_w_GD_AmE_destressed` metadata directory
 - **Step 3e**: `LTR_..._aligned_w_GD_AmE_destressed/LTR_..._aligned_CM_filtered_LM_filtered.pW_V.json` dist (sanity check)
 - **Step 3c**: `CM_AmE_destressed_aligned_w_LTR_..._pseudocount0.01/LTR_..._aligned_CM_filtered_LM_filtered_pY1X0X1X2.json` dist (sanity check)
 - **Step ?**: `LD_Fisher_vocab_in_..._contexts/LM_filtered_..._contexts.txt` (sanity check)

Given a choice of parameters $\epsilon$ and $n$, and given
 - wordform channel matrices $p(Y_0^f|X_0^f)$
 - a contextual distribution on segmental wordforms $p(X_0^f|C)$
 - segmental lexicon metadata pre-calculating $k$-cousins/$k$-spheres up to $k=4$
 
Calculate

$$\hat{p}(\hat{X}_0^f = x_0^{'f}|X_0^f = x_0^{*f}, c) = \frac{1}{n} \sum\limits_{y_0^f \in S} p(\hat{X}_0^f = x_0^{'f}|y_0^f, c)$$
 where 
  - edit distance $d(x_0^{'f}, x_0^{*f}) \leq 4$
  - $S = $ a set of $n$ samples from $p(Y_0^f|x_0^{*f})$. In practice an $n \approx 50$ seems to result in estimates that are within $10^{-6}$ of the true estimate. 
  - $p(\hat{X}_0^f = x_0^{'f}|Y_0^f = y_0^f, c) = \frac{p(y_0^f|x_0^{'f})p(x_0^{'f}|c)}{p(y_0^f | c)}$
  - $p(y_0^f| c) = \sum\limits_{v', x_0^{''f}} p(y_0^f|x_0^{''f})p(x_0^{''f}|v')p(v'|c) = \sum\limits_{x_0^{''f}} p(y_0^f|x_0^{''f})p(x_0^{''f}|c)$

In [ ]:
# where do wordform channel matrices live?
#  where are they bundled?
#   - LCM_bundles
# where do contextual distributions on segmental wordforms live?
#   where are they bundled?
#   - WD_bundles
# where do lexicon metadata live?
#   where are they bundled?
#   - lexicon_md_bundles

In [ ]:
'Calculate segmental posterior given segmental wordform + context'

In [ ]:
#plan:
# 1. Check if there are components of the posterior calculation that are worth pre-calculating; if so, calculate them.
#    - e.g. pW_C, pV_W-hat,C
# 2. Next notebook should have a flag for calculating just p(V-hat = v* | V = v*) ∀v* vs. the full p(V-hat | V = v*) ∀v*
#    - Consider facilitating SLURM cluster jobs by 
#       - making posterior calculation deterministic (unlike P4bnt2) 
#       - adding/supporting optional notebook arguments that specify which parts of the posterior dist to calculate

## Step 5c: Calculate $p(\hat{V} = v^*| V = v^*, C)$

In [ ]:
#FIXME

# Step 6: Generating analysis measures